#### Read data

In [ ]:
val paths: List[String] = List("dbfs:/FileStore/shared_uploads/hugower@kth.se/sequences_oceania.fasta",
                               "dbfs:/FileStore/shared_uploads/hugower@kth.se/sequences_northamerica.fasta",
                               "dbfs:/FileStore/shared_uploads/hugower@kth.se/sequences_southamerica.fasta",
                               "dbfs:/FileStore/shared_uploads/hugower@kth.se/sequences_europe.fasta",
                               "dbfs:/FileStore/shared_uploads/hugower@kth.se/sequences_africa.fasta",
                               "dbfs:/FileStore/shared_uploads/hugower@kth.se/sequences_asia.fasta")

  

>     paths: List[String] = List(dbfs:/FileStore/shared_uploads/hugower@kth.se/sequences_oceania.fasta, dbfs:/FileStore/shared_uploads/hugower@kth.se/sequences_northamerica.fasta, dbfs:/FileStore/shared_uploads/hugower@kth.se/sequences_southamerica.fasta, dbfs:/FileStore/shared_uploads/hugower@kth.se/sequences_europe.fasta, dbfs:/FileStore/shared_uploads/hugower@kth.se/sequences_africa.fasta, dbfs:/FileStore/shared_uploads/hugower@kth.se/sequences_asia.fasta)

In [ ]:
import scala.util.matching.Regex

val pattern: Regex = "/[a-zA-Z]+_([a-zA-Z]+)\\.".r

def read_datasets(paths:List[String]): List[RDD[(String,String)]] = {
  if (paths.size < 1) {
    return List.fill(0) (sc.emptyRDD)
  }
  else {
    pattern.findFirstMatchIn(paths.head) match {
      case Some(x) => {
        val label:String = x.group(1)
        return (sc.textFile(paths.head).filter(_ != "").map(_.trim()).map(s => (s,label)))::read_datasets(paths.tail)
      }
      case None => throw new RuntimeException("no label found")
    }
  }
}
// read data and cache for optimization
sc.hadoopConfiguration.set("textinputformat.record.delimiter",">")
val datasets = read_datasets(paths)
val data = datasets(2)

  

>     import scala.util.matching.Regex
>     pattern: scala.util.matching.Regex = /[a-zA-Z]+_([a-zA-Z]+)\.
>     read_datasets: (paths: List[String])List[org.apache.spark.rdd.RDD[(String, String)]]
>     datasets: List[org.apache.spark.rdd.RDD[(String, String)]] = List(MapPartitionsRDD[643] at map at command-2185505652307040:13, MapPartitionsRDD[648] at map at command-2185505652307040:13, MapPartitionsRDD[653] at map at command-2185505652307040:13, MapPartitionsRDD[658] at map at command-2185505652307040:13, MapPartitionsRDD[663] at map at command-2185505652307040:13, MapPartitionsRDD[672] at map at command-2185505652307040:13)
>     data: org.apache.spark.rdd.RDD[(String, String)] = MapPartitionsRDD[653] at map at command-2185505652307040:13

In [ ]:
data.count

  

>     res6: Long = 135

In [ ]:
// val filtered_s = data.filter( {case (genome,label) => genome.split("\n").head.split('|').last == "complete"})
val samples = data.map( {case (genome,label) => (genome.split("\n").tail.mkString(""), label)}).cache()
samples.count

  

>     samples: org.apache.spark.rdd.RDD[(String, String)] = MapPartitionsRDD[720] at map at command-2185505652307550:2
>     res7: Long = 135

In [ ]:
// A tail recursive overlapping subsequence function
def subsequence_str( sequence:String, k:Int ): String = {
  def helper(seq:String, acc:String): String = {
    if (seq.length < k ) {
      return acc
    }
    else {
      val sub = seq.substring(0,k)
//       helper(seq.substring(k), acc + sub + " ")
      helper(seq.tail, acc + sub + " ")
    }
  }
  return helper(sequence, "")
}

  

>     subsequence_str: (sequence: String, k: Int)String

In [ ]:
val k_mers = samples.map( {case (genome,label) => (subsequence_str(genome, 3),label)} ).cache()

  

>     k_mers: org.apache.spark.rdd.RDD[(String, String)] = MapPartitionsRDD[787] at map at command-2185505652307569:1

In [ ]:
k_mers.take(10)

  

>     res8: Array[(String, String)] = Array(("ATG TGG GGA GAG AGA GAG AGC GCC CCT CTT TTG TGT GTC TCC CCC CCT CTG TGG GGT GTT TTT TTC TCA CAA AAC ACG CGA GAG AGA GAA AAA AAA AAC ACA CAC ACA CAC ACG CGT GTC TCC CCA CAA AAC ACT CTC TCA CAG AGT GTT TTT TTG TGC GCC CCT CTG TGT GTT TTT TTT TTA TAC ACA CAG AGG GGT GTT TTC TCG CGC GCG CGA GAC ACG CGT GTG TGC GCT CTC TCG CGT GTA TAC ACG CGT GTG TGG GGC GCT CTT TTT TTG TGG GGA GAG AGA GAC ACT CTC TCC CCG CGT GTG TGG GGA GAG AGG GGA GAG AGG GGT GTC TCT CTT TTA TAT ATC TCA CAG AGA GAG AGG GGC GCA CAC ACG CGT GTC TCA CAA AAC ACA CAT ATC TCT CTT TTA TAA AAA AAG AGA GAT ATG TGG GGC GCA CAC ACT CTT TTG TGT GTG TGG GGC GCT CTT TTA TAG AGT GTA TAG AGA GAA AAG AGT GTT TTG TGA GAA AAA AAA AAA AAG AGG GGC GCG CGT GTT TTT TTT TTG TGC GCC CCT CTC TCA CAA AAC ACT CTT TTG TGA GAA AAC ACA CAG AGC GCC CCC CCT CTA TAT ATG TGT GTG TGT GTT TTC TCA CAT ATC TCA CAA AAA AAC ACG CGT GTT TTC TCG CGG GGA GAT ATG TGC GCT CTC TCG CGA GAA AAC ACT CTG TGC GCA CAC ACC CCT CTC TCA CAT ATG TGG GGT GTC TCA CAT ATG TGT GTT TTA TAT ATG TGG GGT GTT TTG TGA GAG AGC GCT CTG TGG GGT GTA TAG AGC GCA CAG AGA GAA AAC ACT CTC TCG CGA GAA AAG AGG GGC GCA CAT ATT TTC TCA CAG AGT GTA TAC ACG CGG GGT GTC TCG CGT GTA TAG AGT GTG TGG GGT GTG TGA GAG AGA GAC ACA CAC ACT CTT TTG TGG GGT GTG TGT GTC TCC CCT CTT TTG TGT GTC TCC CCC CCT CTC TCA CAT ATG TGT GTG TGG GGG GGC GCG CGA GAA AAA AAT ATA TAC ACC CCA CAG AGT GTG TGG GGC GCT CTT TTA TAC ACC CCG CGC GCA CAA AAG AGG GGT GTT TTC TCT CTT TTC TCT CTT TTC TCG CGT GTA TAA AAG AGA GAA AAC ACG CGG GGT GTA TAA AAT ATA TAA AAA AAG AGG GGA GAG AGC GCT CTG TGG GGT GTG TGG GGC GCC CCA CAT ATA TAG AGT GTT TTA TAC ACG CGG GGC GCG CGC GCC CCG CGA GAT ATC TCT CTA TAA AAA AAG AGT GTC TCA CAT ATT TTT TTG TGA GAC ACT CTT TTA TAG AGG GGC GCG CGA GAC ACG CGA GAG AGC GCT CTT TTG TGG GGC GCA CAC ACT CTG TGA GAT ATC TCC CCT CTT TTA TAT ATG TGA GAA AAG AGA GAT ATT TTT TTT TTC TCA CAA AAG AGA GAA AAA AAA AAC ACT CTG TGG GGA GAA AAC ACA CAC ACT CTA TAA AAA AAC ACA CAT ATA TAG AGC GCA CAG AGT GTG TGG GGT GTG TGT GTT TTA TAC ACC CCC CCG CGT GTG TGA GAA AAC ACT CTC TCA CAT ATG TGC GCG CGT GTG TGA GAG AGC GCT CTT TTA TAA AAC ACG CGG GGA GAG AGG GGG GGG GGC GCA CAT ATA TAC ACA CAC ACT CTC TCG CGC GCT CTA TAT ATG TGT GTC TCG CGA GAT ATA TAA AAC ACA CAA AAC ACT CTT TTC TCT CTG TGT GTG TGG GGC GCC CCC CCT CTG TGA GAT ATG TGG GGC GCT CTA TAC ACC CCC CCT CTC TCT CTT TTG TGA GAG AGT GTG TGC GCA CAT ATT TTA TAA AAA AAG AGA GAC ACC CCT CTT TTC TCT CTA TAG AGC GCA CAC ACG CGT GTG TGC GCT CTG TGG GGT GTA TAA AAA AAG AGC GCT CTT TTC TCA CAT ATG TGC GCA CAC ACT CTT TTT TTG TGT GTC TCC CCG CGA GAA AAC ACA CAA AAC ACT CTG TGG GGA GAC ACT CTT TTT TTA TAT ATT TTG TGA GAC ACA CAC ACT CTA TAA AAG AGA GAG AGG GGG GGG GGT GTG TGT GTA TAT ATA TAC ACT CTG TGC GCT CTG TGC GCC CCG CGT GTG TGA GAA AAC ACA CAT ATG TGA GAG AGC GCA CAT ATG TGA GAA AAA AAT ATT TTG TGC GCT CTT TTG TGG GGT GTA TAC ACA CAC ACG CGG GGA GAA AAC ACG CGT GTT TTC TCT CTG TGA GAA AAA AAA AAG AGA GAG AGC GCT CTA TAT ATG TGA GAA AAT ATT TTG TGC GCA CAG AGA GAC ACA CAC ACC CCT CTT TTT TTT TTG TGA GAA AAA AAT ATT TTA TAA AAA AAT ATT TTG TGG GGC GCA CAA AAA AAG AGA GAA AAA AAT ATT TTT TTG TGA GAC ACA CAC ACC CCT CTT TTC TCA CAA AAT ATG TGG GGG GGG GGA GAA AAT ATG TGT GTC TCC CCA CAA AAA AAT ATT TTT TTT TTG TGT GTA TAT ATT TTT TTC TCC CCC CCT CTT TTA TAA AAA AAT ATT TTC TCC CCA CAT ATA TAA AAT ATC TCA CAA AAG AGA GAC ACT CTA TAT ATT TTC TCA CAA AAC ACC CCA CAA AAG AGG GGG GGT GTT TTG TGA GAA AAA AAA AAG AGA GAA AAA AAA AAA AAG AGC GCT CTT TTG TGA GAT ATG TGG GGC GCT CTT TTT TTA TAT ATG TGG GGG GGT GTA TAG AGA GAA AAT ATT TTC TCG CGA GAT ATC TCT CTG TGT GTC TCT CTA TAT ATC TCC CCA CAG AGT GTT TTG TGC GCG CGT GTC TCA CAC ACC CCA CAA AAA AAT ATG TGA GAA AAT ATG TGC GCA CAA AAC ACC CCA CAA AAA AAT ATG TGT GTG TGC GCC CCT CTT TTT TTC TCA CAA AAC ACT CTC TCT CTC TCA CAT ATG TGA GAA AAG AGT GTG TGT GTG TGA GAT ATC TCA CAT ATT TTG TGT GTG TGG GGT GTG TGA GAA AAA AAC ACT CTT TTC TCA CAT ATG TGG GGC GCA CAG AGA GAC ACG CGG GGG GGC GCG CGA GAT ATT TTT TTT TTG TGT GTT TTA TAA AAA AAG AGC GCC CCA CAC ACT CTT TTG TGC GCG CGA GAA AAT ATT TTT TTT TTG TGT GTG TGG GGC GCA CAC ACT CTG TGA GAG AGA GAA AAT ATT TTT TTG TGA GAC ACT CTA TAA AAA AAG AGA GAA AAG AGG GGT GTG TGC GCC CCA CAC ACT CTA TAC ACT CTT TTG TGT GTG TGG GGT GTT TTA TAC ACT CTT TTA TAC ACC CCC CCC CCA CAA AAA AAA AAT ATG TGC GCT CTG TGT GTT TTG TGT GTT TTA TAA AAA AAA AAT ATT TTT TTA TAT ATT TTG TGT GTC TCC CCA CAG AGC GCA CAT ATG TGT GTC TCA CAC ACA CAA AAT ATT TTC TCA CAG AGA GAA AAG AGT GTA TAG AGG GGA GAC ACC CCT CTG TGA GAG AGC GCA CAT ATA TAG AGT GTC TCT CTT TTG TGC GCC CCG CGA GAA AAT ATA TAC ACC CCA CAT ATA TAA AAT ATG TGA GAA AAT ATC TCT CTG TGG GGC GCT CTT TTG TGA GAA AAA AAA AAC ACC CCA CAT ATT TTC TCT CTT TTC TCG CGT GTA TAA AAG AGG GGG GGT GTG TGG GGT GTC TCG CGC GCA CAC ACT CTA TAT ATT TTG TGC GCC CCT CTT TTT TTG TGG GGA GAG AGG GGC GCT CTG TGT GTG TGT GTG TGT GTT TTC TCT CTC TCT CTT TTA TAT ATG TGT GTT TTG TGG GGT GTT TTG TGC GCC CCA CAT ATA TAA AAC ACA CAA AAG AGT GTG TGT GTG TGC GCC CCT CTA TAT ATT TTG TGG GGG GGT GTT TTC TCC CCA CAC ACG CGT GTG TGC GCT CTA TAG AGC GCG CGC GCT CTA TAA AAC ACA CAT ATA TAG AGG GGT GTT TTG TGT GTA TAA AAC ACC CCA CAT ATA TAC ACA CAG AGG GGT GTG TGT GTT TTG TGT GTT TTG TGG GGA GAG AGA GAA AAG AGG GGT GTT TTC TCC CCG CGA GAA AAG AGG GGT GTC TCT CTT TTA TAA AAT ATG TGA GAC ACA CAA AAC ACC CCT CTT TTC TCT CTT TTG TGA GAA AAA AAT ATA TAC ACT CTC TCC CCA CAA AAA AAA AAA AAG AGA GAG AGA GAA AAA AAG AGT GTC TCA CAA AAC ACA CAT ATC TCA CAA AAT ATA TAT ATT TTG TGT GTT TTG TGG GGT GTG TGA GAC ACT CTT TTT TTA TAA AAA AAC ACT CTT TTA TAA AAT ATG TGA GAA AAG AGA GAG AGA GAT ATC TCG CGC GCC CCA CAT ATT TTA TAT ATT TTT TTT TTG TGG GGC GCA CAT ATC TCT CTT TTT TTT TTT TTC TCT CTG TGC GCT CTT TTC TCC CCA CAC ACA CAA AAG AGT GTG TGC GCT CTT TTT TTT TTG TGT GTG TGG GGA GAA AAA AAC ACT CTG TGT GTG TGA GAA AAA AAG AGG GGT GTT TTT TTG TGG GGA GAT ATT TTA TAT ATA TAA AAA AAG AGC GCA CAT ATT TTC TCA CAA AAA AAC ACA CAA AAA AAT ATT TTG TGT GTT TTG TGA GAA AAT ATC TCC CCT CTG TGT GTG TGG GGT GTA TAA AAT ATT TTT TTT TTA TAA AAA AAG AGT GTT TTA TAC ACA CAA AAA AAA AAG AGG GGA GAA AAA AAA AAG AGC GCT CTA TAA AAA AAA AAA AAA AAG AGG GGT GTG TGC GCC CCT CTG TGG GGA GAA AAT ATA TAT ATT TTG TGG GGT GTG TGA GAA AAC ACA CAG AGA GAA AAA AAT ATC TCA CAA AAT ATA TAC ACT CTG TGA GAG AGT GTC TCC CCT CTC TCT CTT TTT TTA TAT ATG TGC GCA CAT ATT TTT TTG TGC GCA CAT ATC TCA CAG AGA GAG AGG GGC GCT CTG TGC GCT CTC TCG CGT GTG TGT GTT TTG TGT GTA TAC ACG CGA GAT ATC TCA CAA AAT ATT TTT TTT TTC TCT CTC TCC CCC CCG CGC GCA CAC ACT CTC TCT CTT TTG TGA GAA AAA AAC ACT CTG TGC GCT CTC TCA CAA AAA AAA AAT ATT TTC TCT CTG TGT GTG TGC GCG CGT GTG TGT GTT TTT TTT TTA TAC ACA CAG AGA GAA AAG AGG GGC GCC CCG CGC GCT CTA TAT ATA TAA AAC ACA CAA AAT ATA TAC ACT CTA TAG AGA GAT ATG TGG GGA GAA AAT ATT TTT TTC TCA CAC ACA CAG AGT GTA TAT ATT TTC TCA CAC ACT CTG TGA GAG AGA GAC ACT CTC TCA CAT ATT TTG TGA GAT ATG TGC GCT CTA TAT ATG TGA GAT ATG TGT GTT TTC TCA CAC ACA CAT ATC TCT CTG TGA GAT ATT TTT TTG TGG GGC GCT CTA TAC ACT CTA TAA AAC ACA CAA AAT ATC TCT CTA TAG AGT GTT TTG TGT GTA TAA AAT ATG TGG GGC GCC CCT CTA TAC ACA CAT ATT TTA TAC ACA CAG AGG GGT GTG TGG GGT GTG TGT GTT TTG TGT GTT TTC TCA CAG AGT GTT TTG TGA GAC ACT CTT TTC TCG CGC GCA CAG AGT GTG TGG GGC GCT CTA TAA AAC ACT CTA TAA AAC ACA CAT ATC TCT CTT TTT TTG TGG GGC GCA CAC ACT CTG TGT GTT TTT TTA TAT ATG TGA GAA AAA AAA AAA AAC ACT CTC TCA CAA AAA AAC ACC CCC CCG CGT GTC TCC CCT CTT TTG TGA GAT ATT TTG TGG GGC GCT CTT TTG TGA GAA AAG AGA GAG AGA GAA AAG AGT GTT TTT TTA TAA AAG AGG GGA GAA AAG AGG GGT GTG TGT GTA TAG AGA GAG AGT GTT TTT TTC TCT CTT TTA TAG AGA GAG AGA GAC ACG CGG GGT GTT TTG TGG GGG GGA GAA AAA AAT ATT TTG TGT GTT TTA TAA AAA AAT ATT TTT TTA TAT ATC TCT CTC TCA CAA AAC ACC CCT CTG TGT GTG TGC GCT CTT TTG TGT GTG TGA GAA AAA AAT ATT TTG TGT GTC TCG CGG GGT GTG TGG GGA GAC ACA CAA AAA AAT ATT TTG TGT GTC TCA CAC ACC CCT CTG TGT GTG TGC GCA CAA AAA AAG AGG GGA GAA AAA AAT ATT TTA TAA AAG AGG GGA GAG AGA GAG AGT GTG TGT GTT TTC TCA CAG AGA GAC ACA CAT ATT TTC TCT CTT TTT TTA TAA AAG AGC GCT CTT TTG TGT GTA TAA AAA AAT ATA TAA AAA AAT ATT TTT TTT TTT TTG TGG GGC GCT CTT TTT TTG TGT GTG TGT GTG TGC GCT CTG TGA GAC ACT CTC TCT CTA TAT ATC TCA CAT ATT TTA TAT ATT TTG TGG GGT GTG TGG GGA GAG AGC GCT CTA TAA AAA AAC ACT CTT TTA TAA AAA AAG AGC GCC CCT CTT TTG TGA GAA AAT ATT TTT TTA TAG AGG GGT GTG TGA GAA AAA AAC ACA CAT ATT TTT TTG TGT GTC TCA CAC ACG CGC GCA CAC ACT CTC TCA CAA AAA AAG AGG GGG GGA GAT ATT TTG TGT GTA TAC ACA CAG AGA GAA AAA AAG AGT GTG TGT GTG TGT GTT TTA TAA AAA AAT ATC TCC CCA CAG AGA GAG AGA GAA AAG AGA GAA AAA AAC ACT CTG TGG GGC GCC CCT CTA TAC ACT CTC TCA CAT ATG TGC GCC CCT CTC TCT CTA TAA AAA AAA AAG AGC GCC CCC CCC CCA CAA AAA AAA AAG AGA GAA AAA AAT ATT TTA TAT ATC TCT CTT TTC TCT CTT TTA TAG AGA GAG AGG GGG GGA GAG AGA GAA AAA AAC ACA CAC ACT CTT TTC TCC CCC CCA CAC ACA CAG AGA GAA AAG AGT GTG TGT GTT TTA TAA AAC ACA CAG AGA GAG AGG GGA GAA AAG AGT GTT TTG TGT GTC TCT CTT TTG TGA GAA AAA AAA AAC ACT CTG TGG GGT GTG TGA GAT ATT TTT TTA TAC ACA CAA AAC ACC CCA CAT ATT TTA TAG AGA GAA AAC ACA CAA AAC ACC CCT CTA TAC ACT CTA TAG AGT GTG TGA GAA AAG AGC GCT CTG TGT GTT TTG TGA GAA AAG AGC GCT CTC TCC CCA CAT ATT TTG TGG GGT GTT TTG TGG GGT GTA TAC ACA CAC ACC CCA CAG AGT GTT TTT TTG TGT GTA TAT ATT TTA TAA AAC ACG CGG GGG GGC GCT CTT TTA TAT ATG TGT GTT TTG TGC GCT CTC TCG CGA GAA AAA AAT ATC TCA CAA AAA AAG AGA GAC ACA CAC ACA CAG AGA GAA AAA AAA AAG AGT GTA TAC ACT CTG TGT GTG TGC GCC CCC CCT CTT TTG TGC GCA CAC ACC CCT CTA TAA AAT ATA TAT ATG TGA GAT ATG TGG GGT GTA TAA AAC ACA CAA AAA AAC ACA CAA AAT ATA TAC ACC CCT CTT TTC TCA CAC ACA CAC ACT CTC TCA CAA AAA AAG AGG GGC GCG CGG GGT GTG TGC GCA CAC ACC CCA CAA AAC ACA CAA AAA AAG AGG GGT GTT TTA TAC ACT CTT TTT TTT TTG TGG GGT GTG TGA GAT ATG TGA GAC ACA CAC ACT CTG TGT GTG TGA GAT ATA TAG AGA GAA AAG AGT GTG TGC GCA CAA AAG AGG GGT GTT TTA TAC ACA CAA AAG AGA GAG AGT GTG TGT GTG TGA GAA AAT ATA TAT ATC TCA CAC ACT CTT TTT TTT TTG TGA GAA AAC ACT CTT TTG TGA GAT ATG TGA GAA AAA AAG AGG GGA GAT ATT TTG TGA GAT ATA TAA AAA AAG AGT GTA TAC ACT CTT TTA TAA AAT ATG TGA GAG AGA GAA AAG AGT GTG TGC GCT CTC TCT CTG TGC GCC CCT CTA TAT ATA TAC ACA CAG AGT GTT TTG TGA GAA AAC ACT CTC TCG CGG GGT GTA TAC ACA CAG AGA GAA AAG AGT GTA TAA AAA AAT ATG TGA GAG AGT GTT TTC TCG CGC GCC CCT CTG TGT GTG TGT GTT TTG TGT GTG TGG GGC GCA CAG AGA GAT ATG TGC GCT CTG TGT GTC TCA CAT ATA TAA AAA AAA AAA AAC ACT CTT TTT TTG TGC GCA CAA AAC ACC CCA CAG AGT GTA TAT ATC TCT CTG TGA GAA AAT ATT TTA TAC ACT CTT TTA TAC ACA CAC ACC CCA CAC ACT CTG TGG GGG GGC GCA CAT ATT TTG TGA GAT ATT TTT TTA TAG AGA GAT ATG TGA GAG AGT GTG TGG GGA GAG AGT GTA TAT ATG TGG GGC GCT CTA TAC ACA CAT ATA TAC ACT CTA TAC ACT CTT TTA TAT ATT TTT TTG TGA GAT ATG TGA GAG AGT GTC TCT CTG TGG GGT GTG TGA GAG AGT GTT TTT TTA TAA AAA AAT ATT TTG TGG GGC GCT CTT TTC TCA CAC ACA CAT ATA TAT ATG TGT GTA TAT ATT TTG TGT GTT TTC TCT CTT TTT TTT TTT TTA TAC ACC CCC CCT CTC TCC CCA CAG AGA GAT ATG TGA GAG AGG GGA GAT ATG TGA GAA AAG AGA GAA AAG AGA GAA AAG AGG GGT GTG TGA GAT ATT TTG TGT GTG TGA GAA AAG AGA GAA AAG AGA GAA AAG AGA GAG AGT GTT TTT TTG TGA GAG AGC GCC CCA CAT ATC TCA CAA AAC ACT CTC TCA CAA AAT ATA TAT ATG TGA GAG AGT GTA TAT ATG TGG GGT GTA TAC ACT CTG TGA GAA AAG AGA GAT ATG TGA GAT ATT TTA TAC ACC CCA CAA AAG AGG GGT GTA TAA AAA AAC ACC CCT CTT TTT TTG TGG GGA GAA AAT ATT TTT TTG TGG GGT GTG TGC GCC CCA CAC ACT CTT TTC TCT CTG TGC GCT CTG TGC GCT CTC TCT CTT TTC TCA CAA AAC ACC CCT CTG TGA GAA AAG AGA GAA AAG AGA GAG AGC GCA CAA AAG AGA GAA AAG AGA GAA AAG AGA GAT ATT TTG TGG GGT GTT TTA TAG AGA GAT ATG TGA GAT ATG TGA GAT ATA TAG AGT GTC TCA CAA AAC ACA CAA AAA AAC ACT CTG TGT GTT TTG TGG GGT GTC TCA CAA AAC ACA CAA AAG AGA GAC ACG CGG GGC GCA CAG AGT GTG TGA GAG AGG GGA GAC ACA CAA AAT ATC TCA CAG AGA GAC ACA CAA AAC ACT CTA TAC ACT CTA TAT ATT TTC TCA CAA AAA AAC ACA CAA AAT ATT TTG TGT GTT TTG TGA GAG AGG GGT GTT TTC TCA CAA AAC ACC CCT CTC TCA CAA AAT ATT TTA TAG AGA GAG AGA GAT ATG TGG GGA GAA AAC ACT CTT TTA TAC ACA CAC ACC CCA CAG AGT GTT TTG TGT GTT TTC TCA CAG AGA GAC ACT CTA TAT ATT TTG TGA GAA AAG AGT GTG TGA GAA AAT ATA TAG AGT GTT TTT TTT TTA TAG AGT GTG TGG GGT GTT TTA TAT ATT TTT TTA TAA AAA AAA AAC ACT CTT TTA TAC ACT CTG TGA GAC ACA CAA AAT ATG TGT GTA TAT ATA TAC ACA CAT ATT TTA TAA AAA AAA AAA AAT ATG TGC GCA CAG AGA GAC ACA CAT ATT TTG TGT GTG TGG GGA GAA AAG AGA GAA AAG AGC GCT CTA TAA AAA AAA AAA AAG AGG GGT GTA TAA AAA AAA AAC ACC CCA CAA AAC ACA CAG AGT GTG TGG GGT GTT TTG TGT GTT TTA TAA AAT ATG TGC GCA CAG AGC GCC CCA CAA AAT ATG TGT GTT TTT TTA TAC ACC CCT CTT TTA TAA AAA AAC ACA CAT ATG TGG GGA GAG AGG GGA GAG AGG GGT GTG TGT GTT TTG TGC GCA CAG AGG GGA GAG AGC GCC CCT CTT TTA TAA AAA AAT ATA TAA AAG AGG GGC GCT CTA TAC ACT CTA TAA AAC ACA CAA AAT ATG TGC GCC CCA CAT ATG TGC GCA CAA AAG AGT GTT TTG TGA GAA AAT ATC TCT CTG TGA GAT ATG TGA GAT ATT TTA TAC ACA CAT ATA TAG AGC GCT CTA TAC ACT CTA TAA AAT ATG TGG GGA GAC ACC CCA CAC ACT CTT TTA TAA AAA AAG AGT GTG TGG GGG GGT GTG TGG GGT GTA TAG AGT GTT TTG TGT GTG TGT GTT TTT TTT TTA TAA AAG AGC GCG CGG GGA GAC ACA CAC ACA CAA AAT ATC TCT CTT TTG TGC GCT CTA TAA AAA AAC ACA CAC ACT CTG TGT GTC TCT CTT TTC TCA CAT ATG TGT GTT TTG TGT GTC TCG CGG GGC GCC CCC CCA CAA AAA AAT ATG TGT GTT TTA TAA AAC ACA CAA AAA AAG AGG GGT GTG TGA GAA AAG AGA GAC ACA CAT ATT TTC TCA CAA AAC ACT CTT TTC TCT CTT TTA TAA AAG AGA GAG AGT GTG TGC GCT CTT TTA TAT ATG TGA GAA AAA AAA AAT ATT TTT TTT TTA TAA AAT ATC TCA CAG AGC GCA CAC ACG CGA GAA AAG AGT GTT TTC TCT CTA TAC ACT CTT TTG TGC GCA CAC ACC CCA CAT ATT TTA TAT ATT TTA TAT ATC TCA CAG AGC GCT CTG TGG GGT GTA TAT ATT TTT TTT TTT TTG TGG GGT GTG TGC GCT CTG TGA GAC ACC CCC CCT CTA TAT ATA TAC ACA CAT ATT TTC TCT CTT TTT TTA TAA AAG AGA GAG AGT GTT TTT TTG TGT GTG TGT GTA TAG AGA GAT ATA TAC ACT CTG TGT GTT TTC TCG CGC GCA CAC ACA CAA AAA AAT ATG TGT GTC TCT CTA TAC ACT CTT TTA TAG AGC GCT CTG TGT GTC TCT CTT TTT TTG TGA GAT ATA TAA AAA AAA AAA AAT ATC TCT CTC TCT CTA TAT ATG TGA GAC ACA CAA AAA AAC ACT CTT TTG TGT GTT TTT TTC TCA CAA AAG AGC GCT CTT TTT TTT TTT TTG TGG GGA GAA AAA AAT ATG TGA GAA AAG AGA GAG AGT GTG TGA GAA AAA AAA AAG AGC GCA CAA AAG AGT GTT TTG TGA GAA AAC ACA CAA AAA AAA AAG AGA GAT ATC TCG CGC GCT CTG TGA GAG AGA GAT ATT TTC TCC CCT CTA TAA AAA AAG AGA GAG AGG GGA GAA AAG AGT GTT TTA TAA AAG AGC GCC CCA CAT ATT TTT TTA TAT ATA TAA AAC ACT CTG TGA GAA AAA AAG AGT GTA TAA AAA AAC ACC CCT CTT TTC TCA CAG AGT GTT TTG TGA GAA AAC ACA CAG AGA GAG AGA GAA AAA AAA AAC ACA CAA AAG AGA GAT ATG TGA GAT ATA TAA AAG AGA GAA AAA AAA AAT ATC TCA CAA AAA AAG AGC GCT CTT TTG TGT GTG TGT GTT TTG TGA GAA AAG AGA GAA AAG AGT GTT TTA TAC ACA CAA AAC ACA CAA AAC ACT CTC TCT CTG TGG GGA GAA AAG AGA GAA AAA AAC ACT CTA TAA AAG AGT GTT TTC TCC CCT CTC TCA CAC ACA CAG AGA GAA AAA AAA AAC ACT CTT TTG TGT GTT TTA TAC ACT CTT TTT TTA TAT ATA TAT ATT TTG TGA GAC ACA CAT ATT TTA TAA AAT ATG TGG GGC GCA CAA AAT ATC TCT CTT TTC TCA CAT ATC TCC CCA CAG AGA GAT ATT TTC TCT CTG TGC GCC CCA CAC ACT CTC TCT CTT TTG TGT GTT TTA TAG AGT GTG TGA GAC ACA CAT ATT TTG TGA GAC ACA CAT ATC TCA CAC ACT CTT TTT TTC TCT CTT TTA TAA AAA AAG AGA GAA AAA AAG AGA GAT ATG TGC GCT CTC TCC CCA CAT ATA TAT ATA TAT ATA TAG AGT GTG TGG GGG GGT GTG TGA GAT ATG TGT GTT TTG TGT GTT TTC TCA CAA AAG AGA GAG AGG GGG GGT GTG TGT GTT TTT TTT TTA TAA AAC ACT CTG TGC GCT CTG TGT GTG TGG GGT GTT TTA TAT ATA TAC ACC CCT CTA TAC ACT CTA TAA AAA AAA AAA AAG AGG GGC GCT CTG TGG GGT GTG TGG GGC GCA CAC ACT CTA TAC ACT CTG TGA GAA AAA AAT ATG TGC GCT CTA TAG AGC GCG CGA GAA AAA AAG AGC GCT CTT TTT TTG TGA GAG AGA GAA AAA AAA AAG AGT GTG TGC GCC CCA CAA AAC ACA CAG AGA GAC ACA CAA AAT ATT TTA TAT ATA TAT ATA TAA AAC ACC CCA CAC ACT CTT TTA TAC ACC CCC CCG CGG GGG GGT GTC TCA CAG AGG GGG GGT GTT TTT TTA TAA AAA AAT ATG TGG GGT GTT TTA TAC ACA CAC ACT CTG TGT GTA TAG AGA GAG AGG GGA GAG AGG GGC GCA CAA AAA AAG AGA GAC ACA CAG AGT GTG TGC GCT CTT TTA TAA AAA AAA AAA AAG AGT GTG TGT GTA TAA AAA AAA AAG AGT GTG TGC GCC CCT CTT TTT TTT TTA TAC ACA CAT ATT TTC TCT CTA TAC ACC CCA CAT ATC TCT CTA TAT ATT TTA TAT ATC TCT CTC TCT CTA TAA AAT ATG TGA GAG AGA GAA AAG AGC GCA CAA AAG AGA GAA AAA AAT ATT TTC TCT CTT TTG TGG GGA GAA AAC ACT CTG TGT GTT TTT TTC TCT CTT TTG TGG GGA GAA AAT ATT TTT TTG TGC GCG CGA GAG AGA GAA AAA AAT ATG TGC GCT CTT TTG TGC GCA CAC ACA CAT ATG TGC GCA CAG AGA GAA AAG AGA GAA AAA AAC ACA CAC ACG CGC GCA CAA AAA AAT ATT TTA TAA AAT ATG TGC GCC CCT CTG TGT GTC TCT CTG TGT GTG TGT GTG TGG GGA GAA AAA AAC ACT CTA TAA AAA AAG AGC GCC CCA CAT ATA TAG AGT GTT TTT TTC TCA CAA AAC ACT CTA TAT ATA TAC ACA CAG AGC GCG CGT GTA TAA AAA AAT ATA TAT ATA TAA AAG AGG GGG GGT GTA TAT ATT TTA TAA AAA AAA AAT ATA TAC ACA CAA AAG AGA GAG AGG GGG GGT GTG TGT GTG TGG GGT GTT TTG TGA GAT ATT TTA TAT ATG TGG GGT GTG TGC GCT CTA TAG AGA GAT ATT TTT TTT TTA TAC ACT CTT TTT TTT TTA TAC ACA CAC ACC CCA CAG AGT GTA TAA AAA AAA AAC ACA CAA AAC ACT CTG TGT GTA TAG AGC GCG CGT GTC TCA CAC ACT CTT TTA TAT ATC TCA CAA AAC ACA CAC ACA CAC ACT CTT TTA TAA AAC ACG CGA GAT ATC TCT CTA TAA AAA AAT ATG TGA GAA AAA AAC ACT CTC TCT CTT TTG TGT GTT TTA TAC ACA CAA AAT ATG TGC GCC CCA CAC ACT CTT TTG TGG GGC GCT CTA TAT ATG TGT GTA TAA AAC ACA CAC ACA CAT ATG TGG GGC GCT CTT TTA TAA AAA AAT ATT TTT TTG TGG GGA GAA AAG AGA GAA AAG AGC GCT CTG TGC GCT CTC TCG CGG GGT GTA TAT ATA TAT ATG TGA GAG AGA GAT ATC TCT CTC TCT CTC TCA CAA AAA AAG AGT GTG TGC GCC CCA CAG AGC GCT CTA TAC ACA CAG AGT GTT TTT TTC TCT CTG TGT GTT TTT TTC TCT CTT TTC TCA CAC ACC CCT CTG TGA GAT ATG TGC GCT CTG TGT GTT TTA TAC ACA CAG AGC GCG CGT GTA TAT ATA TAA AAT ATG TGG GGT GTT TTA TAT ATC TCT CTT TTA TAC ACT CTT TTC TCT CTT TTC TCT CTT TTC TCT CTA TAA AAA AAA AAC ACA CAC ACC CCT CTG TGA GAA AAG AGA GAA AAC ACA CAT ATT TTT TTT TTA TAT ATT TTG TGA GAA AAA AAC ACC CCA CAT ATC TCT CTC TCA CAC ACT CTT TTG TGC GCT CTG TGG GGT GTT TTC TCC CCT CTA TAT ATA TAA AAA AAG AGA GAT ATT TTG TGG GGT GTC TCC CCT CTA TAT ATT TTC TCT CTG TGG GGA GAC ACA CAA AAT ATC TCT CTA TAC ACA CAC ACA CAA AAC ACT CTA TAG AGG GGT GTA TAT ATA TAG AGA GAA AAT ATT TTT TTC TCT CTT TTA TAA AAG AGA GAG AGA GAG AGG GGT GTG TGA GAT ATA TAA AAA AAA AAG AGT GTG TGT GTA TAT ATA TAT ATT TTA TAC ACA CAC ACT CTA TAG AGT GTA TAA AAT ATC TCC CCT CTA TAC ACC CCA CAC ACA CAT ATT TTC TCC CCA CAC ACC CCT CTA TAG AGA GAT ATG TGG GGT GTG TGA GAA AAG AGT GTT TTA TAT ATC TCA CAC ACC CCT CTT TTT TTG TGA GAC ACA CAA AAT ATC TCT CTT TTA TAA AAG AGA GAC ACA CAC ACT CTT TTC TCT CTT TTT TTC TCT CTT TTT TTG TGA GAG AGA GAG AGA GAA AAG AGT GTG TGA GAG AGG GGA GAC ACT CTA TAT ATT TTA TAA AAG AGG GGT GTG TGT GTT TTT TTA TAC ACA CAA AAC ACA CAG AGT GTA TAG AGA GAC ACA CAA AAC ACA CAT ATT TTA TAA AAC ACC CCT CTC TCC CCA CAC ACA CAC ACG CGC GCA CAA AAG AGT GTT TTG TGT GTG TGG GGA GAC ACA CAT ATG TGT GTC TCA CAA AAT ATG TGA GAC ACA CAT ATA TAT ATG TGG GGA GAC ACA CAA AAC ACA CAG AGT GTT TTT TTG TGG GGT GTC TCC CCA CAA AAC ACT CTT TTA TAT ATT TTT TTG TGG GGA GAT ATG TGG GGA GAG AGC GCT CTG TGA GAT ATG TGT GTT TTA TAC ACT CTA TAA AAA AAA AAT ATA TAA AAA AAA AAC ACC CCT CTC TCA CAT ATA TAA AAT ATT TTC TCA CAC ACA CAT ATG TGA GAA AAG AGG GGT GTA TAA AAA AAA AAC ACA CAT ATT TTT TTT TTA TAT ATG TGT GTT TTT TTT TTA TAC ACC CCT CTA TAA AAT ATG TGA GAT ATG TGA GAC ACA CAC ACT CTC TCT CTA TAC ACG CGT GTG TGT GTT TTG TGA GAG AGG GGC GCT CTT TTT TTT TTG TGA GAG AGT GTA TAC ACT CTA TAC ACC CCA CAC ACA CAC ACA CAA AAC ACT CTG TGA GAT ATC TCC CCT CTA TAG AGT GTT TTT TTT TTC TCT CTG TGG GGG GGT GTA TAG AGG GGT GTA TAC ACA CAT ATG TGT GTC TCA CAG AGC GCA CAT ATT TTA TAA AAA AAT ATC TCA CAC ACA CAC ACT CTA TAA AAA AAA AAA AAG AGT GTG TGG GGA GAA AAA AAT ATA TAC ACC CCC CCA CAC ACA CAA AAG AGT GTT TTA TAA AAT ATG TGG GGT GTT TTT TTA TAA AAC ACT CTT TTC TCT CTA TAT ATT TTA TAA AAA AAT ATG TGG GGG GGC GCA CAG AGA GAT ATA TAA AAC ACA CAA AAC ACT CTG TGT GTT TTA TAT ATC TCT CTT TTG TGC GCC CCA CAC ACT CTG TGC GCA CAT ATT TTG TGT GTT TTA TAA AAC ACA CAC ACT CTC TCC CCA CAA AAC ACA CAA AAA AAT ATA TAG AGA GAG AGT GTT TTG TGA GAA AAG AGT GTT TTT TTA TAA AAT ATC TCC CCA CAC ACC CCT CTG TGC GCT CTC TCT CTA TAC ACA CAA AAG AGA GAT ATG TGC GCT CTT TTA TAT ATT TTA TAC ACA CAG AGA GAG AGC GCA CAA AAG AGG GGG GGC GCT CTG TGG GGT GTG TGA GAA AAG AGC GCT CTG TGC GCT CTA TAA AAC ACT CTT TTT TTT TTG TGT GTG TGC GCA CAC ACT CTT TTA TAT ATC TCT CTT TTA TAG AGC GCC CCT CTA TAC ACT CTG TGT GTA TAA AAT ATA TAA AAG AGA GAC ACA CAG AGT GTA TAG AGG GGT GTG TGA GAG AGT GTT TTA TAG AGG GGT GTG TGA GAT ATG TGT GTT TTA TAG AGA GAG AGA GAA AAA AAC ACA CAA AAT ATG TGA GAG AGT GTT TTA TAC ACT CTT TTG TGT GTT TTT TTC TCA CAA AAC ACA CAT ATG TGC GCC CCA CAA AAT ATT TTT TTA TAG AGA GAT ATT TTC TCT CTT TTG TGC GCA CAA AAA AAA AAG AGA GAG AGT GTC TCT CTT TTG TGA GAA AAC ACG CGT GTG TGG GGT GTG TGT GTG TGT GTA TAA AAA AAA AAC ACT CTT TTG TGT GTG TGG GGA GAC ACA CAA AAC ACA CAG AGC GCA CAG AGA GAC ACA CAA AAC ACC CCC CCT CTT TTA TAA AAG AGG GGG GGT GTG TGT GTA TAG AGA GAA AAG AGC GCT CTG TGT GTT TTA TAT ATG TGT GTA TAC ACA CAT ATG TGG GGG GGC GCA CAC ACA CAC ACT CTT TTT TTC TCT CTT TTA TAT ATG TGA GAA AAC ACA CAA AAT ATT TTT TTA TAA AAG AGA GAA AAA AAG AGG GGT GTG TGT GTT TTC TCA CAG AGA GAT ATA TAC ACC CCT CTT TTG TGT GTA TAC ACG CGT GTG TGT GTG TGG GGT GTA TAA AAA AAC ACA CAA AAG AGC GCT CTA TAC ACA CAA AAA AAA AAT ATA TAT ATC TCT CTA TAG AGT GTA TAC ACA CAA AAC ACA CAG AGG GGA GAG AGT GTC TCA CAC ACC CCT CTT TTT TTT TTG TGT GTT TTA TAT ATG TGA GAT ATG TGT GTC TCA CAG AGC GCA CAC ACC CCA CAC ACC CCT CTG TGC GCT CTC TCA CAG AGT GTA TAT ATG TGA GAA AAC ACT CTT TTA TAA AAG AGC GCA CAT ATG TGG GGT GTA TAC ACA CAT ATT TTT TTA TAC ACT CTT TTG TGT GTG TGC GCT CTA TAG AGT GTG TGA GAG AGT GTA TAC ACA CAC ACT CTG TGG GGT GTA TAA AAT ATT TTA TAC ACC CCA CAG AGT GTG TGT GTG TGG GGT GTC TCA CAC ACT CTA TAT ATA TAA AAA AAC ACA CAT ATA TAT ATA TAA AAC ACT CTT TTC TCT CTA TAA AAA AAG AGA GAA AAA AAC ACT CTT TTT TTG TGT GTA TAT ATT TTG TGC GCA CAT ATA TAG AGA GAC ACG CGG GGT GTG TGC GCT CTT TTT TTA TAC ACT CTT TTA TAC ACA CAA AAA AAG AGT GTC TCC CCT CTC TCA CAG AGA GAA AAT ATA TAC ACA CAA AAA AAG AGG GGT GTC TCC CCT CTA TAT ATT TTA TAC ACG CGG GGA GAT ATG TGT GTT TTT TTT TTC TCT CTA TAC ACA CAA AAA AAG AGA GAA AAA AAA AAC ACA CAG AGT GTT TTA TAC ACA CAC ACA CAA AAC ACA CAA AAC ACC CCA CAT ATA TAA AAA AAA AAC ACC CCA CAG AGT GTT TTA TAC ACT CTT TTA TAT ATA TAA AAA AAT ATT TTG TGG GGA GAT ATG TGG GGT GTG TGT GTT TTG TGT GTT TTT TTG TGT GTA TAC ACA CAG AGA GAA AAA AAT ATT TTG TGA GAC ACC CCC CCT CTA TAA AAG AGT GTT TTG TGG GGA GAC ACA CAA AAT ATT TTA TAT ATT TTA TAT ATA TAA AAG AGA GAA AAA AAG AGA GAC ACA CAA AAT ATT TTC TCT CTT TTA TAT ATT TTT TTC TCA CAC ACA CAG AGA GAG AGC GCA CAA AAC ACC CCA CAA AAT ATT TTG TGA GAT ATC TCT CTT TTG TGT GTA TAC ACC CCA CAA AAA AAC ACC CCA CAA AAC ACC CCA CAT ATA TAT ATC TCC CCA CAA AAA AAC ACG CGC GCA CAA AAG AGC GCT CTT TTC TCG CGA GAT ATA TAA AAT ATT TTT TTT TTA TAA AAG AGT GTT TTT TTG TGT GTA TAT ATG TGT GTG TGA GAT ATA TAA AAT ATA TAT ATC TCA CAA AAA AAT ATT TTT TTG TGC GCT CTG TGA GAT ATG TGA GAT ATT TTT TTA TAA AAA AAC ACC CCA CAG AGT GTT TTA TAA AAC ACT CTG TGG GGT GTT TTA TAT ATA TAA AAG AGA GAA AAA AAC ACC CCT CTG TGC GCT CTT TTC TCA CAA AAG AGA GAG AGA GAG AGC GCT CTT TTA TAA AAA AAG AGT GTT TTA TAC ACA CAT ATT TTT TTT TTT TTC TCC CCC CCT CTG TGA GAC ACT CTT TTA TAA AAA AAT ATG TGG GGT GTG TGA GAT ATG TGT GTG TGG GGT GTG TGG GGC GCT CTA TAT ATT TTG TGA GAT ATT TTA TAT ATA TAA AAA AAC ACA CAC ACT CTA TAC ACA CAC ACA CAC ACC CCC CCT CTC TCT CTT TTT TTT TTA TAA AAG AGA GAA AAA AAG AGG GGA GAG AGC GCT CTA TAA AAA AAT ATT TTG TGT GTT TTA TAC ACA CAT ATA TAA AAA AAC ACC CCT CTA TAT ATT TTG TGT GTT TTT TTG TGG GGC GCA CAT ATG TGT GTT TTA TAA AAC ACA CAA AAT ATG TGC GCA CAA AAC ACT CTA TAA AAT ATA TAA AAA AAG AGC GCC CCA CAC ACG CGT GTA TAT ATA TAA AAA AAC ACC CCA CAA AAA AAT ATA TAC ACC CCT CTG TGG GGT GTG TGT GTA TAT ATA TAC ACG CGT GTT TTG TGT GTC TCT CTT TTT TTG TGG GGA GAG AGC GCA CAC ACA CAA AAA AAA AAC ACC CCA CAG AGT GTT TTG TGA GAA AAA AAC ACA CAT ATC TCA CAA AAA AAT ATT TTC TCG CGT GTT TTT TTG TGA GAT ATG TGT GTA TAC ACT CTG TGA GAA AAG AGT GTC TCA CAG AGA GAG AGG GGA GAC ACG CGC GCG CGC GCA CAG AGG GGG GGA GAA AAT ATG TGG GGA GAT ATA TAA AAT ATC TCT CTT TTG TGC GCC CCT CTG TGC GCG CGA GAA AAG AGA GAT ATC TCT CTA TAA AAA AAA AAC ACC CCA CAG AGT GTC TCT CTC TCT CTG TGA GAA AAG AGA GAA AAG AGT GTA TAG AGT GTG TGG GGA GAA AAA AAA AAT ATC TCC CCT CTA TAC ACC CCA CAT ATA TAC ACA CAG AGA GAA AAA AAG AGA GAC ACG CGT GTT TTC TCT CTT TTG TGA GAG AGT GTG TGT GTA TAA AAT ATG TGT GTG TGA GAA AAA AAA AAC ACT CTA TAC ACC CCG CGA GAA AAG AGT GTT TTG TGT GTA TAG AGG GGA GAG AGA GAC ACA CAT ATT TTA TAT ATA TAC ACT CTT TTA TAA AAA AAC ACC CCA CAG AGC GCA CAA AAA AAT ATA TAA AAT ATA TAG AGT GTT TTT TTA TAA AAA AAA AAA AAT ATT TTA TAC ACA CAG AGA GAA AAG AGA GAG AGG GGT GTT TTG TGG GGC GCC CCA CAC ACA CAC ACA CAG AGA GAT ATC TCT CTA TAA AAT ATG TGG GGC GCT CTG TGC GCT CTT TTA TAT ATG TGT GTA TAG AGA GAC ACA CAA AAT ATT TTC TCT CTA TAG AGT GTC TCT CTT TTA TAC ACT CTA TAT ATT TTA TAA AAG AGA GAA AAA AAC ACC CCT CTA TAA AAT ATG TGA GAA AAT ATT TTA TAT ATC TCT CTA TAG AGA GAG AGT GTA TAT ATT TTA TAG AGG GGT GTT TTT TTG TGA GAA AAA AAA AAC ACC CCC CCT CTT TTG TGC GCT CTA TAC ACT CTC TCA CAT ATG TGG GGT GTT TTT TTA TAG AGC GCT CTG TGC GCT CTG TGT GTT TTA TAA AAT ATA TAG AGT GTG TGT GTC TCC CCC CCT CTT TTG TGG GGG GGA GAT ATA TAC ACT CTA TAT ATA TAG AGC GCT CTA TAA AAT ATT TTA TAT ATG TGC GCT CTA TAA AAG AGC GCC CCT CTT TTT TTT TTC TCT CTT TTA TAA AAC ACA CAA AAA AAG AGT GTT TTG TGT GTT TTA TAG AGT GTA TAC ACA CAA AAC ACT CTA TAC ACT CTA TAA AAC ACA CAT ATA TAG AGT GTT TTA TAC ACA CAC ACG CGG GGT GTG TGT GTT TTT TTA TAA AAA AAC ACC CCG CGT GTG TGT GTT TTT TTG TGT GTA TAC ACT CTA TAA AAT ATT TTA TAT ATA TAT ATG TGC GCC CCT CTT TTA TAT ATT TTT TTC TCT CTT TTT TTA TAC ACT CTT TTT TTA TAT ATT TTG TGC GCT CTA TAC ACA CAA AAT ATT TTG TGT GTG TGT GTA TAC ACT CTT TTT TTT TTA TAC ACT CTA TAG AGA GAA AAG AGT GTA TAC ACA CAA AAA AAT ATT TTC TCT CTA TAG AGA GAA AAT ATT TTA TAA AAA AAG AGC GCA CAT ATC TCT CTA TAT ATG TGC GCC CCG CGA GAC ACT CTA TAC ACT CTA TAT ATA TAG AGC GCA CAA AAA AAG AGA GAA AAT ATA TAC ACT CTG TGT GTT TTA TAA AAG AGA GAG AGT GTG TGT GTC TCG CGG GGT GTA TAA AAA AAT ATT TTT TTT TTG TGT GTC TCT CTA TAG AGA GAG AGG GGC GCT CTT TTC TCA CAT ATT TTT TTA TAA AAT ATT TTA TAT ATT TTT TTG TGA GAA AAG AGT GTC TCA CAC ACC CCT CTA TAA AAT ATT TTT TTT TTT TTC TCT CTA TAA AAA AAC ACT CTG TGA GAT ATA TAA AAA AAT ATA TAT ATT TTA TAT ATA TAA AAT ATT TTT TTG TGG GGT GTT TTT TTT TTT TTA TAC ACT CTA TAT ATT TTA TAA AAG AGT GTG TGT GTT TTT TTG TGC GCC CCT CTA TAG AGG GGT GTT TTC TCT CTT TTT TTA TAA AAT ATC TCT CTA TAC ACT CTC TCA CAA AAC ACC CCG CGC GCT CTG TGC GCT CTT TTT TTA TAG AGG GGT GTG TGT GTT TTT TTT TTA TAA AAT ATG TGT GTC TCT CTA TAA AAT ATT TTT TTA TAG AGG GGC GCA CAT ATG TGC GCC CCT CTT TTC TCT CTT TTA TAC ACT CTG TGT GTA TAC ACT CTG TGG GGT GTT TTA TAC ACA CAG AGA GAG AGA GAA AAG AGG GGC GCT CTA TAT ATT TTT TTG TGA GAA AAC ACT CTC TCT CTA TAC ACT CTA TAA AAT ATG TGT GTC TCA CAC ACT CTA TAT ATT TTG TGC GCA CAA AAC ACC CCT CTA TAC ACT CTG TGT GTA TAC ACT CTG TGG GGT GTT TTC TCT CTA TAT ATA TAC ACC CCT CTT TTG TGT GTA TAG AGT GTG TGT GTT TTT TTG TGT GTC TCT CTT TTA TAG AGT GTG TGG GGT GTT TTT TTA TAG AGA GAT ATT TTC TCT CTT TTT TTA TAG AGA GAC ACA CAC ACC CCT CTA TAT ATC TCC CCT CTT TTC TCT CTT TTT TTA TAG AGA GAA AAA AAC ACT CTA TAT ATA TAC ACA CAA AAA AAT ATT TTA TAC ACC CCA CAT ATT TTT TTC TCA CAT ATC TCT CTT TTT TTT TTA TAA AAA AAT ATG TGG GGG GGA GAT ATT TTT TTA TAA AAC ACT CTG TGC GCT CTT TTT TTT TTG TGG GGC GCT CTT TTA TAG AGT GTT TTG TGC GCA CAG AGA GAG AGT GTG TGG GGT GTT TTT TTT TTT TTG TGG GGC GCA CAT ATA TAT ATA TAT ATT TTC TCT CTT TTT TTT TTC TCA CAC ACT CTA TAG AGG GGT GTT TTT TTT TTT TTC TCT CTA TAT ATG TGT GTA TAC ACT CTT TTG TGG GGA GAT ATT TTG TGG GGC GCT CTG TGC GCA CAA AAT ATC TCA CAT ATG TGC GCA CAA AAT ATT TTG TGT GTT TTT TTT TTT TTC TCA CAG AGC GCT CTA TAT ATT TTT TTT TTG TGC GCA CAG AGT GTA TAC ACA CAT ATT TTT TTT TTA TAT ATT TTA TAG AGT GTA TAA AAT ATT TTC TCT CTT TTG TGG GGC GCT CTT TTA TAT ATG TGT GTG TGG GGT GTT TTA TAA AAT ATA TAA AAT ATT TTA TAA AAT ATC TCT CTT TTG TGT GTA TAC ACA CAA AAA AAT ATG TGG GGC GCC CCC CCC CCG CGA GAT ATT TTT TTC TCA CAG AGC GCT CTA TAT ATG TGG GGT GTT TTA TAG AGA GAA AAT ATG TGT GTA TAC ACA CAT ATC TCT CTT TTC TCT CTT TTT TTG TGC GCA CAT ATC TCA CAT ATT TTT TTT TTA TAT ATT TTA TAT ATG TGT GTA TAT ATG TGG GGA GAA AAA AAA AAG AGT GTT TTA TAT ATG TGT GTG TGC GCA CAT ATG TGT GTT TTG TGT GTA TAG AGA GAC ACG CGG GGT GTT TTG TGT GTA TAA AAT ATT TTC TCA CAT ATC TCA CAA AAC ACT CTT TTG TGT GTA TAT ATG TGA GAT ATG TGT GTG TGT GTT TTA TAC ACA CAA AAA AAC ACG CGT GTA TAA AAT ATA TAG AGA GAG AGC GCA CAA AAC ACA CAA AAG AGA GAG AGT GTC TCG CGA GAA AAT ATG TGT GTA TAC ACA CAA AAC ACT CTA TAT ATT TTG TGT GTT TTA TAA AAT ATG TGG GGT GTG TGT GTT TTA TAG AGA GAA AAG AGG GGT GTC TCC CCT CTT TTT TTT TTA TAT ATG TGT GTC TCT CTA TAT ATG TGC GCT CTA TAA AAT ATG TGG GGA GAG AGG GGT GTA TAA AAA AAG AGG GGC GCT CTT TTT TTT TTG TGC GCA CAA AAA AAC ACT CTA TAC ACA CAC ACA CAA AAT ATT TTG TGG GGA GAA AAT ATT TTG TGT GTG TGT GTT TTA TAA AAT ATT TTG TGT GTG TGA GAT ATA TAC ACA CAT ATT TTC TCT CTG TGT GTG TGC GCT CTG TGG GGT GTA TAG AGT GTA TAC ACA CAT ATT TTT TTA TAT ATT TTA TAG AGT GTG TGA GAT ATG TGA GAA AAG AGT GTT TTG TGC GCG CGA GAG AGA GAG AGA GAC ACT CTT TTG TGT GTC TCA CAC ACT CTA TAC ACA CAG AGT GTT TTT TTA TAA AAA AAA AAG AGA GAC ACC CCA CAA AAT ATA TAA AAA AAT ATC TCC CCT CTA TAC ACT CTG TGA GAC ACC CCA CAG AGT GTC TCT CTT TTC TCT CTT TTA TAC ACA CAT ATC TCG CGT GTT TTG TGA GAT ATA TAG AGT GTG TGT GTT TTA TAC ACA CAG AGT GTG TGA GAA AAG AGA GAA AAT ATG TGG GGT GTT TTC TCC CCA CAT ATC TCC CCA CAT ATC TCT CTT TTT TTA TAC ACT CTT TTT TTG TGA GAT ATA TAA AAA AAG AGC GCT CTG TGG GGT GTC TCA CAA AAA AAA AAG AGA GAC ACT CTT TTA TAT ATG TGA GAA AAA AAG AGA GAC ACA CAT ATT TTC TCT CTC TCT CTC TCT CTC TCT CTC TCA CAT ATT TTT TTT TTG TGT GTT TTA TAA AAC ACT CTT TTA TAG AGA GAC ACA CAA AAC ACC CCT CTG TGA GAG AGA GAG AGC GCT CTA TAA AAT ATA TAA AAC ACA CAC ACT CTA TAA AAA AAG AGG GGT GTT TTC TCA CAT ATT TTG TGC GCC CCT CTA TAT ATT TTA TAA AAT ATG TGT GTT TTA TAT ATA TAG AGT GTT TTT TTT TTT TTG TGA GAT ATG TGG GGT GTA TAA AAA AAT ATC TCA CAA AAA AAA AAT ATG TGT GTG TGA GAA AAG AGA GAA AAT ATC TCA CAT ATC TCT CTG TGC GCA CAA AAA AAA AAT ATC TCA CAG AGC GCG CGT GTC TCT CTG TGT GTT TTT TTA TAC ACT CTA TAC ACA CAG AGT GTC TCA CAG AGC GCT CTT TTA TAT ATG TGT GTG TGT GTC TCA CAA AAC ACC CCT CTA TAT ATA TAC ACT CTG TGT GTT TTA TAC ACT CTA TAG AGA GAT ATC TCA CAG AGG GGC GCA CAT ATT TTA TAG AGT GTG TGT GTC TCT CTG TGA GAT ATG TGT GTT TTG TGG GGT GTG TGA GAT ATA TAG AGT GTG TGC GCG CGG GGA GAA AAG AGT GTT TTG TGC GCA CAG AGT GTT TTA TAA AAA AAA AAT ATG TGT GTT TTT TTG TGA GAT ATG TGC GCT CTT TTA TAC ACG CGT GTT TTA TAA AAT ATA TAC ACG CGT GTT TTT TTT TTC TCA CAT ATC TCA CAA AAC ACT CTT TTT TTT TTA TAA AAC ACG CGT GTA TAC ACC CCA CAA AAT ATG TGG GGA GAA AAA AAA AAA AAC ACT CTC TCA CAA AAA AAA AAC ACA CAC ACT CTA TAG AGT GTT TTG TGC GCA CAA AAC ACT CTG TGC GCA CAG AGA GAA AAG AGC GCT CTG TGA GAA AAC ACT CTT TTG TGC GCA CAA AAA AAG AGA GAA AAT ATG TGT GTG TGT GTC TCC CCT CTT TTA TAG AGA GAC ACA CAA AAT ATG TGT GTC TCT CTT TTA TAT ATC TCT CTA TAC ACT CTT TTT TTT TTA TAT ATT TTT TTC TCA CAG AGC GCA CAG AGC GCT CTC TCG CGG GGC GCA CAA AAG AGG GGG GGT GTT TTT TTG TGT GTT TTG TGA GAT ATT TTC TCA CAG AGA GAT ATG TGT GTA TAG AGA GAA AAA AAC ACT CTA TAA AAA AAG AGA GAT ATG TGT GTT TTG TGT GTT TTG TGA GAA AAT ATG TGT GTC TCT CTT TTA TAA AAA AAT ATT TTG TGT GTC TCA CAC ACA CAT ATC TCA CAA AAT ATC TCT CTG TGA GAC ACA CAT ATA TAG AGA GAA AAG AGT GTT TTA TAC ACT CTG TGG GGC GCG CGA GAT ATA TAG AGT GTT TTG TGT GTA TAA AAT ATA TAA AAC ACT CTA TAT ATA TAT ATG TGC GCT CTC TCA CAC ACC CCT CTA TAT ATA TAA AAC ACA CAA AAA AAG AGT GTT TTG TGA GAA AAA AAA AAC ACA CAT ATG TGA GAC ACA CAC ACC CCC CCC CCG CGT GTG TGA GAC ACC CCT CTT TTG TGG GGT GTG TGC GCT CTT TTG TGT GTA TAT ATT TTG TGA GAC ACT CTG TGT GTA TAG AGT GTG TGC GCG CGC GCG CGT GTC TCA CAT ATA TAT ATT TTA TAA AAT ATG TGC GCG CGC GCA CAG AGG GGT GTA TAG AGC GCA CAA AAA AAA AAA AAG AGT GTC TCA CAC ACA CAA AAC ACA CAT ATT TTG TGC GCT CTT TTT TTG TGA GAT ATA TAT ATG TGG GGA GAA AAC ACG CGT GTT TTA TAA AAA AAG AGA GAT ATT TTT TTC TCA CAT ATG TGT GTC TCA CAT ATT TTG TGT GTC TCT CTG TGA GAA AAC ACA CAA AAC ACT CTA TAC ACG CGA GAA AAA AAA AAC ACA CAA AAA AAT ATA TAC ACG CGT GTA TAG AGT GTG TGC GCT CTG TGC GCT CTA TAA AAA AAA AAA AAG AGA GAA AAT ATA TAA AAC ACT CTT TTA TAC ACC CCT CTT TTT TTT TTA TAA AAG AGT GTT TTG TGA GAC ACA CAT ATG TGT GTG TGC GCA CAA AAC ACT CTA TAC ACT CTA TAG AGA GAC ACA CAA AAG AGT GTT TTG TGT GTT TTA TAA AAT ATG TGT GTT TTG TGT GTA TAA AAC ACA CAA AAC ACA CAA AAA AAG AGA GAT ATA TAG AGC GCA CAC ACT CTT TTA TAA AAG AGG GGG GGT GTG TGG GGT GTA TAA AAA AAA AAT ATT TTG TGT GTT TTA TAA AAT ATA TAA AAT ATT TTG TGG GGT GTT TTG TGA GAA AAG AGC GCA CAG AGT GTT TTA TAA AAT ATT TTA TAA AAA AAG AGT GTT TTA TAC ACA CAC ACT CTT TTG TGT GTG TGT GTT TTC TCC CCT CTT TTT TTT TTT TTG TGT GTT TTG TGC GCT CTG TGC GCT CTA TAT ATT TTT TTT TTC TCT CTA TAT ATT TTT TTA TAA AAT ATA TAA AAC ACA CAC ACC CCT CTG TGT GTT TTC TCA CAT ATG TGT GTC TCA CAT ATG TGT GTC TCT CTA TAA AAA AAC ACA CAT ATA TAC ACT CTG TGA GAC ACT CTT TTT TTT TTC TCA CAA AAG AGT GTG TGA GAA AAA AAT ATC TCA CAT ATA TAG AGG GGA GAT ATA TAC ACA CAA AAG AGG GGC GCT CTA TAT ATT TTG TGA GAT ATG TGG GGT GTG TGG GGT GTG TGT GTC TCA CAC ACT CTC TCG CGT GTG TGA GAC ACA CAT ATA TAG AGC GCA CAT ATC TCT CTA TAC ACA CAG AGA GAT ATA TAC ACT CTT TTG TGT GTT TTT TTT TTG TGC GCT CTA TAA AAC ACA CAA AAA AAC ACA CAT ATG TGC GCT CTG TGA GAT ATT TTT TTT TTG TGA GAC ACA CAC ACA CAT ATG TGG GGT GTT TTT TTA TAG AGC GCC CCA CAG AGC GCG CGT GTG TGG GGT GTG TGG GGT GTA TAG AGT GTT TTA TAT ATA TAC ACT CTA TAA AAT ATG TGA GAC ACA CAA AAA AAG AGC GCT CTT TTG TGC GCC CCC CCA CAT ATT TTG TGA GAT ATT TTG TGC GCT CTG TGC GCA CAG AGT GTC TCA CAT ATA TAA AAC ACA CAA AAG AGA GAG AGA GAA AAG AGT GTG TGG GGG GGT GTT TTT TTT TTG TGT GTC TCG CGT GTG TGC GCC CCT CTG TGG GGT GTT TTT TTG TGC GCC CCT CTG TGG GGC GCA CAC ACG CGA GAT ATA TAT ATT TTA TAC ACG CGC GCA CAC ACA CAA AAC ACT CTA TAA AAT ATG TGG GGT GTG TGA GAC ACT CTT TTT TTT TTT TTG TGC GCA CAT ATT TTT TTC TCT CTT TTA TAC ACC CCT CTA TAG AGA GAG AGT GTT TTT TTT TTT TTA TAG AGT GTG TGC GCA CAG AGT GTT TTG TGG GGT GTA TAA AAC ACA CAT ATC TCT CTG TGT GTT TTA TAC ACA CAC ACA CAC ACC CCA CAT ATC TCA CAA AAA AAA AAC ACT CTT TTA TAT ATA TAG AGA GAG AGT GTA TAC ACA CAC ACT CTG TGA GAC ACT CTT TTT TTG TGC GCA CAA AAC ACA CAT ATC TCA CAG AGC GCT CTT TTG TGT GTG TGT GTT TTT TTT TTG TGG GGC GCT CTG TGC GCT CTG TGA GAA AAT ATG TGT GTA TAC ACA CAA AAT ATT TTT TTT TTT TTA TAA AAA AAG AGA GAT ATG TGC GCT CTT TTC TCT CTG TGG GGT GTA TAA AAG AGC GCC CCA CAG AGT GTA TAC ACC CCA CAT ATA TAT ATT TTG TGT GTT TTA TAT ATG TGA GAT ATA TAC ACC CCA CAA AAT ATG TGT GTA TAC ACT CTA TAG AGA GAA AAG AGG GGT GTT TTC TCT CTG TGT GTT TTG TGC GCT CTT TTA TAT ATG TGA GAA AAA AAG AGT GTT TTT TTA TAC ACG CGC GCC CCC CCT CTG TGA GAC ACA CAC ACA CAC ACG CGT GTT TTA TAT ATG TGT GTG TGC GCT CTC TCA CAT ATG TGG GGA GAT ATG TGG GGC GCT CTC TCT CTA TAT ATT TTA TAT ATT TTC TCA CAA AAT ATT TTT TTC TCC CCT CTA TAA AAC ACA CAC ACC CCT CTA TAC ACC CCT CTT TTG TGA GAA AAG AGG GGT GTT TTC TCT CTG TGT GTT TTA TAG AGA GAG AGT GTG TGG GGT GTA TAA AAC ACA CAA AAC ACT CTT TTT TTT TTG TGA GAT ATT TTC TCT CTG TGA GAG AGT GTA TAC ACT CTG TGT GTA TAG AGG GGC GCA CAC ACG CGG GGC GCA CAC ACT CTT TTG TGT GTG TGA GAA AAA AAG AGA GAT ATC TCA CAG AGA GAA AAG AGC GCT CTG TGG GGT GTG TGT GTT TTT TTG TGT GTG TGT GTA TAT ATC TCT CTA TAC ACT CTA TAG AGT GTG TGG GGT GTA TAG AGA GAT ATG TGG GGG GGT GTA TAC ACT CTT TTA TAA AAC ACA CAA AAT ATG TGA GAT ATT TTA TAT ATT TTA TAC ACA CAG AGA GAT ATC TCT CTT TTT TTA TAC ACC CCA CAG AGG GGA GAG AGT GTT TTT TTT TTC TCT CTG TGT GTG TGG GGT GTG TGT GTA TAG AGA GAT ATG TGC GCT CTG TGT GTA TAA AAA AAT ATT TTT TTA TAC ACT CTT TTA TAC ACT CTA TAA AAT ATA TAT ATG TGT GTT TTT TTA TAC ACA CAC ACC CCA CAC ACT CTA TAA AAT ATT TTC TCA CAA AAC ACC CCT CTA TAT ATT TTG TGG GGT GTG TGC GCT CTT TTT TTG TGG GGA GAC ACA CAT ATA TAT ATC TCA CAG AGC GCA CAT ATC TCT CTA TAT ATA TAG AGT GTA TAG AGC GCT CTG TGG GGT GTG TGG GGT GTA TAT ATT TTG TGT GTA TAG AGC GCT CTA TAT ATC TCG CGT GTA TAG AGT GTA TAA AAC ACA CAT ATG TGC GCC CCT CTT TTG TGC GCC CCT CTA TAC ACT CTA TAT ATT TTT TTT TTA TAT ATG TGA GAG AGG GGT GTT TTT TTA TAG AGA GAA AAG AGA GAG AGC GCT CTT TTT TTT TTG TGG GGT GTG TGA GAA AAT ATA TAC ACA CAG AGT GTC TCA CAT ATG TGT GTA TAG AGT GTT TTG TGC GCC CCT CTT TTT TTA TAA AAT ATA TAC ACT CTT TTT TTA TAC ACT CTA TAT ATT TTC TCC CCT CTT TTA TAT ATG TGT GTC TCA CAT ATT TTC TCA CAC ACT CTG TGT GTA TAC ACT CTC TCT CTG TGT GTT TTT TTA TAA AAC ACA CAC ACC CCA CAG AGT GTT TTT TTA TAC ACT CTC TCA CAT ATT TTC TCT CTT TTA TAC ACC CCT CTG TGG GGT GTG TGT GTT TTT TTA TAT ATT TTC TCT CTG TGT GTT TTA TAT ATT TTT TTA TAC ACT CTT TTG TGT GTA TAC ACT CTT TTG TGA GAC ACA CAT ATT TTT TTT TTA TAT ATC TCT CTT TTA TAC ACT CTA TAA AAT ATG TGA GAT ATG TGT GTT TTT TTC TCT CTT TTT TTT TTT TTT TTA TAG AGC GCA CAC ACA CAT ATA TAT ATT TTC TCA CAG AGT GTG TGG GGA GAT ATG TGG GGT GTT TTA TAT ATG TGT GTT TTC TCA CAC ACA CAC ACC CCT CTT TTT TTA TAG AGT GTA TAC ACC CCT CTT TTT TTC TCT CTG TGG GGA GAT ATA TAA AAC ACA CAA AAT ATT TTG TGC GCT CTT TTA TAT ATA TAT ATC TCA CAT ATT TTT TTG TGT GTA TAT ATT TTT TTC TCC CCA CAC ACA CAA AAA AAG AGC GCA CAT ATT TTT TTC TCT CTA TAT ATT TTG TGG GGT GTT TTC TCT CTT TTT TTA TAG AGT GTA TAA AAT ATT TTA TAC ACC CCT CTA TAA AAA AAG AGA GAG AGA GAC ACG CGT GTG TGT GTA TAG AGT GTC TCT CTT TTT TTA TAA AAT ATG TGG GGT GTG TGT GTT TTT TTC TCC CCT CTT TTT TTA TAG AGT GTA TAC ACT CTT TTT TTT TTG TGA GAA AAG AGA GAA AAG AGC GCT CTG TGC GCG CGC GCT CTG TGT GTG TGC GCA CAC ACC CCT CTT TTT TTT TTT TTG TGT GTT TTA TAA AAA AAT ATA TAA AAA AAG AGA GAA AAA AAT ATG TGT GTA TAT ATC TCT CTA TAA AAA AAG AGT GTT TTG TGC GCG CGT GTA TAG AGT GTG TGA GAT ATG TGT GTG TGC GCT CTA TAT ATT TTA TAC ACC CCT CTC TCT CTT TTA TAC ACG CGC GCA CAA AAT ATA TAT ATA TAA AAT ATA TAG AGA GAT ATA TAC ACT CTT TTA TAG AGC GCT CTC TCT CTT TTT TTA TAT ATA TAA AAT ATA TAA AAG AGT GTA TAC ACA CAA AAG AGT GTA TAT ATT TTT TTT TTA TAG AGT GTG TGG GGA GAG AGC GCA CAA AAT ATG TGG GGA GAT ATA TAC ACA CAA AAC ACT CTA TAG AGC GCT CTA TAC ACA CAG AGA GAG AGA GAA AAG AGC GCT CTG TGC GCT CTT TTG TGT GTT TTG TGT GTC TCA CAT ATC TCT CTC TCG CGC GCA CAA AAA AAG AGG GGC GCT CTC TCT CTC TCA CAA AAT ATG TGA GAC ACT CTT TTC TCA CAG AGT GTA TAA AAC ACT CTC TCA CAG AGG GGT GTT TTC TCT CTG TGA GAT ATG TGT GTT TTC TCT CTT TTT TTA TAC ACC CCA CAA AAC ACC CCA CAC ACC CCA CAC ACA CAA AAA AAC ACC CCT CTC TCT CTA TAT ATC TCA CAC ACC CCT CTC TCA CAG AGC GCT CTG TGT GTT TTT TTT TTG TGC GCA CAG AGA GAG AGT GTG TGG GGT GTT TTT TTT TTA TAG AGA GAA AAA AAA AAA AAT ATG TGG GGC GCA CAT ATT TTC TCC CCC CCA CAT ATC TCT CTG TGG GGT GTA TAA AAA AAG AGT GTT TTG TGA GAG AGG GGG GGT GTT TTG TGT GTA TAT ATG TGG GGT GTA TAC ACA CAA AAG AGT GTA TAA AAC ACT CTT TTG TGT GTG TGG GGT GTA TAC ACA CAA AAC ACT CTA TAC ACA CAC ACT CTT TTA TAA AAC ACG CGG GGT GTC TCT CTT TTT TTG TGG GGC GCT CTT TTG TGA GAT ATG TGA GAC ACG CGT GTA TAG AGT GTT TTT TTA TAC ACT CTG TGT GTC TCC CCA CAA AAG AGA GAC ACA CAT ATG TGT GTG TGA GAT ATC TCT CTG TGC GCA CAC ACC CCT CTC TCT CTG TGA GAA AAG AGA GAC ACA CAT ATG TGC GCT CTT TTA TAA AAC ACC CCC CCT CTA TAA AAT ATT TTA TAT ATG TGA GAA AAG AGA GAT ATT TTT TTA TAC ACT CTC TCA CAT ATT TTC TCG CGT GTA TAA AAG AGT GTC TCT CTA TAA AAT ATC TCA CAT ATA TAA AAT ATT TTT TTC TCT CTT TTG TGG GGT GTA TAC ACA CAG AGG GGC GCT CTG TGG GGT GTA TAA AAT ATG TGT GTT TTC TCA CAA AAC ACT CTC TCA CAG AGG GGG GGT GTT TTA TAT ATT TTG TGG GGA GAC ACA CAT ATT TTC TCT CTA TAT ATG TGC GCA CAA AAA AAA AAT ATT TTG TGT GTG TGT GTA TAC ACT CTT TTA TAA AAG AGC GCT CTT TTA TAA AAG AGG GGT GTT TTG TGA GAT ATA TAC ACA CAG AGC GCC CCA CAA AAT ATC TCC CCT CTA TAA AAG AGA GAC ACA CAC ACC CCT CTA TAA AAG AGT GTA TAT ATA TAA AAG AGT GTT TTT TTG TGT GTT TTC TCG CGC GCA CAT ATT TTC TCA CAA AAC ACC CCA CAG AGG GGA GAC ACA CAG AGA GAC ACT CTT TTT TTT TTT TTC TCA CAG AGT GTG TGT GTT TTA TAG AGC GCT CTT TTG TGT GTT TTA TAC ACA CAA AAT ATG TGG GGT GTT TTC TCA CAC ACC CCA CAT ATC TCT CTG TGG GGT GTG TGT GTT TTT TTA TAC ACC CCA CAA AAT ATG TGT GTG TGC GCT CTA TAT ATG TGA GAG AGG GGC GCC CCC CCA CAA AAT ATT TTT TTC TCA CAC ACT CTA TAT ATT TTA TAA AAG AGG GGG GGT GTT TTC TCA CAT ATT TTC TCC CCT CTT TTA TAA AAT ATG TGG GGT GTT TTC TCA CAT ATG TGT GTG TGG GGT GTA TAG AGT GTG TGT GTT TTG TGG GGT GTT TTT TTT TTA TAA AAC ACA CAT ATA TAG AGA GAT ATT TTA TAT ATG TGA GAC ACT CTG TGT GTG TGT GTC TCT CTC TCT CTT TTT TTT TTT TTG TGT GTT TTA TAC ACA CAT ATG TGC GCA CAC ACC CCA CAT ATA TAT ATG TGG GGA GAA AAT ATT TTA TAC ACC CCA CAA AAC ACT CTG TGG GGA GAG AGT GTT TTC TCA CAT ATG TGC GCT CTG TGG GGC GCA CAC ACA CAG AGA GAC ACT CTT TTA TAG AGA GAA AAG AGG GGT GTA TAA AAC ACT CTT TTT TTT TTA TAT ATG TGG GGA GAC ACC CCT CTT TTT TTT TTG TGT GTT TTG TGA GAC ACA CAG AGG GGC GCA CAA AAA AAC ACA CAG AGC GCA CAC ACA CAA AAG AGC GCA CAG AGC GCT CTG TGG GGT GTA TAC ACG CGG GGA GAC ACA CAC ACA CAA AAC ACT CTA TAT ATT TTA TAC ACA CAG AGT GTT TTA TAA AAT ATG TGT GTT TTT TTT TTA TAG AGC GCT CTT TTG TGG GGT GTT TTG TGT GTA TAC ACG CGC GCT CTG TGC GCT CTG TGT GTT TTA TAT ATA TAA AAA AAT ATG TGG GGA GAG AGA GAC ACA CAG AGG GGT GTG TGG GGT GTT TTT TTC TCT CTC TCA CAA AAT ATC TCG CGA GAT ATT TTT TTA TAC ACC CCA CAC ACA CAA AAC ACT CTC TCT CTT TTA TAA AAT ATG TGA GAC ACT CTT TTT TTA TAA AAC ACC CCT CTT TTG TGT GTG TGG GGC GCT CTA TAT ATG TGA GAA AAG AGT GTA TAC ACA CAA AAT ATT TTA TAT ATG TGA GAA AAC ACC CCT CTC TCT CTA TAA AAC ACA CAC ACA CAA AAG AGA GAC ACC CCA CAT ATG TGT GTT TTG TGA GAC ACA CAT ATA TAC ACT CTA TAG AGG GGA GAC ACC CCT CTC TCT CTT TTT TTC TCT CTG TGC GCT CTC TCA CAA AAA AAC ACT CTG TGG GGA GAA AAT ATT TTG TGC GCC CCG CGT GTT TTT TTT TTA TAG AGA GAT ATA TAT ATG TGT GTG TGT GTG TGC GCT CTT TTC TCA CAT ATT TTA TAA AAA AAA AAG AGA GAA AAT ATT TTA TAC ACT CTG TGC GCA CAA AAA AAA AAT ATG TGG GGT GTA TAT ATG TGA GAA AAT ATG TGG GGA GAC ACG CGT GTA TAC ACC CCA CAT ATA TAT ATT TTG TGG GGG GGT GTA TAG AGT GTG TGC GCT CTT TTT TTA TAT ATT TTA TAG AGA GAA AAG AGA GAT ATG TGA GAA AAT ATT TTT TTA TAC ACA CAC ACC CCT CTT TTT TTT TTG TGA GAT ATG TGT GTT TTG TGT GTT TTA TAG AGA GAC ACA CAA AAT ATG TGC GCT CTC TCA CAG AGG GGT GTG TGT GTT TTA TAC ACT CTT TTT TTC TCC CCA CAA AAA AAG AGT GTG TGC GCA CAG AGT GTG TGA GAA AAA AAA AAG AGA GAA AAC ACA CAA AAT ATC TCA CAA AAG AGG GGG GGT GTA TAC ACA CAC ACA CAC ACC CCA CAC ACT CTG TGG GGT GTT TTG TGT GTT TTA TAC ACT CTC TCA CAC ACA CAA AAT ATT TTT TTT TTG TGA GAC ACT CTT TTC TCA CAC ACT CTT TTT TTT TTA TAG AGT GTT TTT TTT TTA TAG AGT GTC TCC CCA CAG AGA GAG AGT GTA TAC ACT CTC TCA CAA AAT ATG TGG GGT GTC TCT CTT TTT TTG TGT GTT TTC TCT CTT TTT TTT TTT TTT TTT TTT TTG TGT GTA TAT ATG TGA GAA AAA AAA AAT ATG TGC GCC CCT CTT TTT TTT TTT TTA TAC ACC CCT CTT TTT TTT TTG TGC GCT CTA TAT ATG TGG GGG GGT GTA TAT ATT TTA TAT ATT TTG TGC GCT CTA TAT ATG TGT GTC TCT CTG TGC GCT CTT TTT TTT TTG TGC GCA CAA AAT ATG TGA GAT ATG TGT GTT TTT TTG TGT GTC TCA CAA AAA AAC ACA CAT ATA TAA AAG AGC GCA CAT ATG TGC GCA CAT ATT TTT TTC TCT CTC TCT CTG TGT GTT TTT TTG TGT GTT TTT TTT TTT TTG TGT GTT TTA TAC ACC CCT CTT TTC TCT CTC TCT CTT TTG TGC GCC CCA CAC ACT CTG TGT GTA TAG AGC GCT CTT TTA TAT ATT TTT TTT TTA TAA AAT ATA TAT ATG TGG GGT GTC TCT CTA TAT ATA TAT ATG TGC GCC CCT CTG TGC GCT CTA TAG AGT GTT TTG TGG GGG GGT GTG TGA GAT ATG TGC GCG CGT GTA TAT ATT TTA TAT ATG TGA GAC ACA CAT ATG TGG GGT GTT TTG TGG GGA GAT ATA TAT ATG TGG GGT GTT TTG TGA GAT ATA TAC ACT CTA TAG AGT GTT TTT TTG TGT GTC TCT CTG TGG GGT GTT TTT TTT TTA TAA AAG AGC GCT CTA TAA AAA AAA AAG AGA GAC ACT CTG TGT GTG TGT GTT TTA TAT ATG TGT GTA TAT ATG TGC GCA CAT ATC TCA CAG AGC GCT CTG TGT GTA TAG AGT GTG TGT GTT TTA TAC ACT CTA TAA AAT ATC TCC CCT CTT TTA TAT ATG TGA GAC ACA CAG AGC GCA CAA AAG AGA GAA AAC ACT CTG TGT GTG TGT GTA TAT ATG TGA GAT ATG TGA GAT ATG TGG GGT GTG TGC GCT CTA TAG AGG GGA GAG AGA GAG AGT GTG TGT GTG TGG GGA GAC ACA CAC ACT CTT TTA TAT ATG TGA GAA AAT ATG TGT GTC TCT CTT TTG TGA GAC ACA CAC ACT CTC TCG CGT GTT TTT TTA TAT ATA TAA AAA AAG AGT GTT TTT TTA TAT ATT TTA TAT ATG TGG GGT GTA TAA AAT ATG TGC GCT CTT TTT TTA TAG AGA GAT ATC TCA CAA AAG AGC GCC CCA CAT ATT TTT TTC TCC CCA CAT ATG TGT GTG TGG GGG GGC GCT CTC TCT CTT TTA TAT ATA TAA AAT ATC TCT CTC TCT CTG TGT GTT TTA TAC ACT CTT TTC TCT CTA TAA AAC ACT CTA TAC ACT CTC TCA CAG AGG GGT GTG TGT GTA TAG AGT GTT TTA TAC ACA CAA AAC ACT CTG TGT GTC TCA CAT ATG TGT GTT TTT TTT TTT TTG TGG GGC GCC CCA CAG AGA GAG AGG GGT GTA TAT ATT TTG TGT GTT TTT TTT TTT TTA TAT ATG TGT GTG TGT GTG TGT GTT TTG TGA GAG AGT GTA TAT ATT TTG TGC GCC CCC CCT CTA TAT ATT TTT TTT TTC TCT CTT TTC TCA CAT ATA TAA AAC ACT CTG TGG GGT GTA TAA AAT ATA TAC ACA CAC ACT CTT TTC TCA CAG AGT GTG TGT GTA TAT ATA TAA AAT ATG TGC GCT CTA TAG AGT GTT TTT TTA TAT ATT TTG TGT GTT TTT TTC TCT CTT TTA TAG AGG GGC GCT CTA TAT ATT TTT TTT TTT TTG TGT GTA TAC ACT CTT TTG TGT GTT TTA TAC ACT CTT TTT TTG TGG GGC GCC CCT CTC TCT CTT TTT TTT TTG TGT GTT TTT TTA TAC ACT CTC TCA CAA AAC ACC CCG CGC GCT CTA TAC ACT CTT TTT TTA TAG AGA GAC ACT CTG TGA GAC ACT CTC TCT CTT TTG TGG GGT GTG TGT GTT TTT TTA TAT ATG TGA GAT ATT TTA TAC ACT CTT TTA TAG AGT GTT TTT TTC TCT CTA TAC ACA CAC ACA CAG AGG GGA GAG AGT GTT TTT TTA TAG AGA GAT ATA TAT ATA TAT ATG TGA GAA AAT ATT TTC TCA CAC ACA CAG AGG GGG GGA GAC ACT CTA TAC ACT CTC TCC CCC CCA CAC ACC CCC CCA CAA AAG AGA GAA AAT ATA TAG AGC GCA CAT ATA TAG AGA GAT ATG TGC GCC CCT CTT TTC TCA CAA AAA AAC ACT CTC TCA CAA AAC ACA CAT ATT TTA TAA AAA AAT ATT TTG TGT GTT TTG TGG GGG GGT GTG TGT GTT TTG TGG GGT GTG TGG GGC GCA CAA AAA AAC ACC CCT CTT TTG TGT GTA TAT ATC TCA CAA AAA AAG AGT GTA TAG AGC GCC CCA CAC ACT CTG TGT GTA TAC ACA CAG AGT GTC TCT CTA TAA AAA AAA AAT ATG TGT GTC TCA CAG AGA GAT ATG TGT GTA TAA AAA AAG AGT GTG TGC GCA CAC ACA CAT ATC TCA CAG AGT GTA TAG AGT GTC TCT CTT TTA TAC ACT CTC TCT CTC TCA CAG AGT GTT TTT TTT TTG TGC GCA CAA AAC ACA CAA AAC ACT CTC TCA CAG AGA GAG AGT GTA TAG AGA GAA AAT ATC TCA CAT ATC TCA CAT ATC TCT CTA TAA AAA AAT ATT TTG TGT GTG TGG GGG GGC GCT CTC TCA CAA AAT ATG TGT GTG TGT GTC TCC CCA CAG AGT GTT TTA TAC ACA CAC ACA CAA AAT ATG TGA GAC ACA CAT ATT TTC TCT CTC TCT CTT TTA TAG AGC GCT CTA TAA AAA AAG AGA GAT ATA TAC ACT CTA TAC ACT CTG TGA GAA AAG AGC GCC CCT CTT TTT TTG TGA GAA AAA AAA AAA AAA AAT ATG TGG GGT GTT TTT TTC TCA CAC ACT CTA TAC ACT CTT TTT TTC TCT CTG TGT GTT TTT TTT TTG TGC GCT CTT TTT TTC TCC CCA CAT ATG TGC GCA CAG AGG GGG GGT GTG TGC GCT CTG TGT GTA TAG AGA GAC ACA CAT ATA TAA AAA AAC ACA CAA AAG AGC GCT CTT TTT TTG TGT GTG TGA GAA AAG AGA GAA AAA AAT ATG TGC GCT CTG TGG GGA GAC ACA CAA AAC ACA CAG AGG GGG GGC GCA CAA AAC ACC CCT CTT TTA TAC ACA CAA AAG AGC GCT CTA TAT ATA TAG AGC GCC CCT CTC TCA CAG AGA GAG AGT GTT TTT TTA TAG AGT GTT TTC TCC CCC CCT CTT TTC TCC CCA CAT ATC TCA CAT ATA TAT ATG TGC GCA CAG AGC GCT CTT TTT TTT TTG TGC GCT CTA TAC ACT CTG TGC GCT CTC TCA CAA AAG AGA GAA AAG AGC GCT CTT TTA TAT ATG TGA GAG AGC GCA CAG AGG GGC GCT CTG TGT GTT TTG TGC GCT CTA TAA AAT ATG TGG GGT GTG TGA GAT ATT TTC TCT CTG TGA GAA AAG AGT GTT TTG TGT GTT TTC TCT CTT TTA TAA AAA AAA AAA AAG AGT GTT TTG TGA GAA AAG AGA GAA AAG AGT GTC TCT CTT TTT TTG TGA GAA AAT ATG TGT GTG TGG GGC GCT CTA TAA AAA AAT ATC TCT CTG TGA GAA AAT ATT TTT TTG TGA GAC ACC CCG CGT GTG TGA GAT ATG TGC GCA CAG AGC GCC CCA CAT ATG TGC GCA CAA AAC ACG CGT GTA TAA AAG AGT GTT TTG TGG GGA GAA AAA AAA AAG AGA GAT ATG TGG GGC GCT CTG TGA GAT ATC TCA CAA AAG AGC GCT CTA TAT ATG TGA GAC ACC CCC CCA CAA AAA AAT ATG TGT GTA TAT ATA TAA AAA AAC ACA CAG AGG GGC GCT CTA TAG AGA GAT ATC TCT CTG TGA GAG AGG GGA GAC ACA CAA AAG AGA GAG AGG GGG GGC GCA CAA AAA AAA AAG AGT GTT TTA TAC ACT CTA TAG AGT GTG TGC GCT CTA TAT ATG TGC GCA CAG AGA GAC ACA CAA AAT ATG TGC GCT CTT TTT TTT TTC TCA CAC ACT CTA TAT ATG TGC GCT CTT TTA TAG AGA GAA AAA AAG AGT GTT TTG TGG GGA GAT ATA TAA AAT ATG TGA GAT ATG TGC GCA CAC ACT CTC TCA CAA AAC ACA CAA AAC ACA CAT ATT TTA TAT ATC TCA CAA AAC ACA CAA AAT ATG TGC GCA CAA AAG AGA GAG AGA GAT ATG TGG GGT GTT TTG TGT GTG TGT GTT TTC TCC CCC CCT CTT TTG TGA GAA AAC ACA CAT ATA TAA AAT ATA TAC ACC CCT CTC TCT CTT TTA TAC ACA CAA AAC ACA CAG AGC GCA CAG AGC GCC CCA CAA AAA AAC ACT CTA TAA AAT ATG TGG GGT GTT TTG TGT GTC TCA CAT ATA TAC ACC CCA CAG AGA GAC ACT CTA TAT ATA TAA AAC ACA CAC ACA CAT ATA TAT ATA TAA AAA AAA AAA AAT ATA TAC ACG CGT GTG TGT GTG TGA GAT ATG TGG GGT GTA TAC ACA CAA AAC ACA CAT ATT TTT TTA TAC ACT CTT TTA TAT ATG TGC GCA CAT ATC TCA CAG AGC GCA CAT ATT TTG TGT GTG TGG GGG GGA GAA AAA AAT ATC TCC CCA CAA AAC ACA CAG AGG GGT GTT TTG TGT GTA TAG AGA GAT ATG TGC GCA CAG AGA GAT ATA TAG AGT GTA TAA AAA AAA AAT ATT TTG TGT GTT TTC TCA CAA AAC ACT CTT TTA TAG AGT GTG TGA GAA AAA AAT ATT TTA TAG AGT GTA TAT ATG TGG GGA GAC ACA CAA AAT ATT TTC TCA CAC ACC CCT CTA TAA AAT ATT TTT TTA TAG AGC GCA CAT ATG TGG GGC GCC CCT CTC TCT CTT TTA TAT ATT TTG TGT GTA TAA AAC ACA CAG AGC GCT CTT TTT TTA TAA AAG AGG GGG GGC GCC CCA CAA AAT ATT TTC TCT CTG TGC GCT CTG TGT GTC TCA CAA AAA AAT ATT TTA TAC ACA CAG AGA GAA AAT ATA TAA AAT ATG TGA GAG AGC GCT CTT TTA TAG AGT GTC TCC CCT CTG TGT GTT TTG TGC GCA CAC ACT CTA TAC ACG CGA GAC ACA CAG AGA GAT ATG TGT GTC TCT CTT TTG TGT GTG TGC GCT CTG TGC GCC CCG CGG GGT GTA TAC ACT CTA TAC ACA CAC ACA CAA AAA AAC ACT CTG TGC GCT CTT TTG TGC GCA CAC A...

In [ ]:
val kmers_df = k_mers.zipWithIndex.map({case ((a,b),c) => (a,b,c)}).toDF("genome", "label", "id").cache()

  

>     kmers_df: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [genome: string, label: string ... 1 more field]

In [ ]:
kmers_df.take(100)

  

>     res14: Array[org.apache.spark.sql.Row] = Array([ATG TGG GGA GAG AGA GAG AGC GCC CCT CTT TTG TGT GTC TCC CCC CCT CTG TGG GGT GTT TTT TTC TCA CAA AAC ACG CGA GAG AGA GAA AAA AAA AAC ACA CAC ACA CAC ACG CGT GTC TCC CCA CAA AAC ACT CTC TCA CAG AGT GTT TTT TTG TGC GCC CCT CTG TGT GTT TTT TTT TTA TAC ACA CAG AGG GGT GTT TTC TCG CGC GCG CGA GAC ACG CGT GTG TGC GCT CTC TCG CGT GTA TAC ACG CGT GTG TGG GGC GCT CTT TTT TTG TGG GGA GAG AGA GAC ACT CTC TCC CCG CGT GTG TGG GGA GAG AGG GGA GAG AGG GGT GTC TCT CTT TTA TAT ATC TCA CAG AGA GAG AGG GGC GCA CAC ACG CGT GTC TCA CAA AAC ACA CAT ATC TCT CTT TTA TAA AAA AAG AGA GAT ATG TGG GGC GCA CAC ACT CTT TTG TGT GTG TGG GGC GCT CTT TTA TAG AGT GTA TAG AGA GAA AAG AGT GTT TTG TGA GAA AAA AAA AAA AAG AGG GGC GCG CGT GTT TTT TTT TTG TGC GCC CCT CTC TCA CAA AAC ACT CTT TTG TGA GAA AAC ACA CAG AGC GCC CCC CCT CTA TAT ATG TGT GTG TGT GTT TTC TCA CAT ATC TCA CAA AAA AAC ACG CGT GTT TTC TCG CGG GGA GAT ATG TGC GCT CTC TCG CGA GAA AAC ACT CTG TGC GCA CAC ACC CCT CTC TCA CAT ATG TGG GGT GTC TCA CAT ATG TGT GTT TTA TAT ATG TGG GGT GTT TTG TGA GAG AGC GCT CTG TGG GGT GTA TAG AGC GCA CAG AGA GAA AAC ACT CTC TCG CGA GAA AAG AGG GGC GCA CAT ATT TTC TCA CAG AGT GTA TAC ACG CGG GGT GTC TCG CGT GTA TAG AGT GTG TGG GGT GTG TGA GAG AGA GAC ACA CAC ACT CTT TTG TGG GGT GTG TGT GTC TCC CCT CTT TTG TGT GTC TCC CCC CCT CTC TCA CAT ATG TGT GTG TGG GGG GGC GCG CGA GAA AAA AAT ATA TAC ACC CCA CAG AGT GTG TGG GGC GCT CTT TTA TAC ACC CCG CGC GCA CAA AAG AGG GGT GTT TTC TCT CTT TTC TCT CTT TTC TCG CGT GTA TAA AAG AGA GAA AAC ACG CGG GGT GTA TAA AAT ATA TAA AAA AAG AGG GGA GAG AGC GCT CTG TGG GGT GTG TGG GGC GCC CCA CAT ATA TAG AGT GTT TTA TAC ACG CGG GGC GCG CGC GCC CCG CGA GAT ATC TCT CTA TAA AAA AAG AGT GTC TCA CAT ATT TTT TTG TGA GAC ACT CTT TTA TAG AGG GGC GCG CGA GAC ACG CGA GAG AGC GCT CTT TTG TGG GGC GCA CAC ACT CTG TGA GAT ATC TCC CCT CTT TTA TAT ATG TGA GAA AAG AGA GAT ATT TTT TTT TTC TCA CAA AAG AGA GAA AAA AAA AAC ACT CTG TGG GGA GAA AAC ACA CAC ACT CTA TAA AAA AAC ACA CAT ATA TAG AGC GCA CAG AGT GTG TGG GGT GTG TGT GTT TTA TAC ACC CCC CCG CGT GTG TGA GAA AAC ACT CTC TCA CAT ATG TGC GCG CGT GTG TGA GAG AGC GCT CTT TTA TAA AAC ACG CGG GGA GAG AGG GGG GGG GGC GCA CAT ATA TAC ACA CAC ACT CTC TCG CGC GCT CTA TAT ATG TGT GTC TCG CGA GAT ATA TAA AAC ACA CAA AAC ACT CTT TTC TCT CTG TGT GTG TGG GGC GCC CCC CCT CTG TGA GAT ATG TGG GGC GCT CTA TAC ACC CCC CCT CTC TCT CTT TTG TGA GAG AGT GTG TGC GCA CAT ATT TTA TAA AAA AAG AGA GAC ACC CCT CTT TTC TCT CTA TAG AGC GCA CAC ACG CGT GTG TGC GCT CTG TGG GGT GTA TAA AAA AAG AGC GCT CTT TTC TCA CAT ATG TGC GCA CAC ACT CTT TTT TTG TGT GTC TCC CCG CGA GAA AAC ACA CAA AAC ACT CTG TGG GGA GAC ACT CTT TTT TTA TAT ATT TTG TGA GAC ACA CAC ACT CTA TAA AAG AGA GAG AGG GGG GGG GGT GTG TGT GTA TAT ATA TAC ACT CTG TGC GCT CTG TGC GCC CCG CGT GTG TGA GAA AAC ACA CAT ATG TGA GAG AGC GCA CAT ATG TGA GAA AAA AAT ATT TTG TGC GCT CTT TTG TGG GGT GTA TAC ACA CAC ACG CGG GGA GAA AAC ACG CGT GTT TTC TCT CTG TGA GAA AAA AAA AAG AGA GAG AGC GCT CTA TAT ATG TGA GAA AAT ATT TTG TGC GCA CAG AGA GAC ACA CAC ACC CCT CTT TTT TTT TTG TGA GAA AAA AAT ATT TTA TAA AAA AAT ATT TTG TGG GGC GCA CAA AAA AAG AGA GAA AAA AAT ATT TTT TTG TGA GAC ACA CAC ACC CCT CTT TTC TCA CAA AAT ATG TGG GGG GGG GGA GAA AAT ATG TGT GTC TCC CCA CAA AAA AAT ATT TTT TTT TTG TGT GTA TAT ATT TTT TTC TCC CCC CCT CTT TTA TAA AAA AAT ATT TTC TCC CCA CAT ATA TAA AAT ATC TCA CAA AAG AGA GAC ACT CTA TAT ATT TTC TCA CAA AAC ACC CCA CAA AAG AGG GGG GGT GTT TTG TGA GAA AAA AAA AAG AGA GAA AAA AAA AAA AAG AGC GCT CTT TTG TGA GAT ATG TGG GGC GCT CTT TTT TTA TAT ATG TGG GGG GGT GTA TAG AGA GAA AAT ATT TTC TCG CGA GAT ATC TCT CTG TGT GTC TCT CTA TAT ATC TCC CCA CAG AGT GTT TTG TGC GCG CGT GTC TCA CAC ACC CCA CAA AAA AAT ATG TGA GAA AAT ATG TGC GCA CAA AAC ACC CCA CAA AAA AAT ATG TGT GTG TGC GCC CCT CTT TTT TTC TCA CAA AAC ACT CTC TCT CTC TCA CAT ATG TGA GAA AAG AGT GTG TGT GTG TGA GAT ATC TCA CAT ATT TTG TGT GTG TGG GGT GTG TGA GAA AAA AAC ACT CTT TTC TCA CAT ATG TGG GGC GCA CAG AGA GAC ACG CGG GGG GGC GCG CGA GAT ATT TTT TTT TTG TGT GTT TTA TAA AAA AAG AGC GCC CCA CAC ACT CTT TTG TGC GCG CGA GAA AAT ATT TTT TTT TTG TGT GTG TGG GGC GCA CAC ACT CTG TGA GAG AGA GAA AAT ATT TTT TTG TGA GAC ACT CTA TAA AAA AAG AGA GAA AAG AGG GGT GTG TGC GCC CCA CAC ACT CTA TAC ACT CTT TTG TGT GTG TGG GGT GTT TTA TAC ACT CTT TTA TAC ACC CCC CCC CCA CAA AAA AAA AAT ATG TGC GCT CTG TGT GTT TTG TGT GTT TTA TAA AAA AAA AAT ATT TTT TTA TAT ATT TTG TGT GTC TCC CCA CAG AGC GCA CAT ATG TGT GTC TCA CAC ACA CAA AAT ATT TTC TCA CAG AGA GAA AAG AGT GTA TAG AGG GGA GAC ACC CCT CTG TGA GAG AGC GCA CAT ATA TAG AGT GTC TCT CTT TTG TGC GCC CCG CGA GAA AAT ATA TAC ACC CCA CAT ATA TAA AAT ATG TGA GAA AAT ATC TCT CTG TGG GGC GCT CTT TTG TGA GAA AAA AAA AAC ACC CCA CAT ATT TTC TCT CTT TTC TCG CGT GTA TAA AAG AGG GGG GGT GTG TGG GGT GTC TCG CGC GCA CAC ACT CTA TAT ATT TTG TGC GCC CCT CTT TTT TTG TGG GGA GAG AGG GGC GCT CTG TGT GTG TGT GTG TGT GTT TTC TCT CTC TCT CTT TTA TAT ATG TGT GTT TTG TGG GGT GTT TTG TGC GCC CCA CAT ATA TAA AAC ACA CAA AAG AGT GTG TGT GTG TGC GCC CCT CTA TAT ATT TTG TGG GGG GGT GTT TTC TCC CCA CAC ACG CGT GTG TGC GCT CTA TAG AGC GCG CGC GCT CTA TAA AAC ACA CAT ATA TAG AGG GGT GTT TTG TGT GTA TAA AAC ACC CCA CAT ATA TAC ACA CAG AGG GGT GTG TGT GTT TTG TGT GTT TTG TGG GGA GAG AGA GAA AAG AGG GGT GTT TTC TCC CCG CGA GAA AAG AGG GGT GTC TCT CTT TTA TAA AAT ATG TGA GAC ACA CAA AAC ACC CCT CTT TTC TCT CTT TTG TGA GAA AAA AAT ATA TAC ACT CTC TCC CCA CAA AAA AAA AAA AAG AGA GAG AGA GAA AAA AAG AGT GTC TCA CAA AAC ACA CAT ATC TCA CAA AAT ATA TAT ATT TTG TGT GTT TTG TGG GGT GTG TGA GAC ACT CTT TTT TTA TAA AAA AAC ACT CTT TTA TAA AAT ATG TGA GAA AAG AGA GAG AGA GAT ATC TCG CGC GCC CCA CAT ATT TTA TAT ATT TTT TTT TTG TGG GGC GCA CAT ATC TCT CTT TTT TTT TTT TTC TCT CTG TGC GCT CTT TTC TCC CCA CAC ACA CAA AAG AGT GTG TGC GCT CTT TTT TTT TTG TGT GTG TGG GGA GAA AAA AAC ACT CTG TGT GTG TGA GAA AAA AAG AGG GGT GTT TTT TTG TGG GGA GAT ATT TTA TAT ATA TAA AAA AAG AGC GCA CAT ATT TTC TCA CAA AAA AAC ACA CAA AAA AAT ATT TTG TGT GTT TTG TGA GAA AAT ATC TCC CCT CTG TGT GTG TGG GGT GTA TAA AAT ATT TTT TTT TTA TAA AAA AAG AGT GTT TTA TAC ACA CAA AAA AAA AAG AGG GGA GAA AAA AAA AAG AGC GCT CTA TAA AAA AAA AAA AAA AAG AGG GGT GTG TGC GCC CCT CTG TGG GGA GAA AAT ATA TAT ATT TTG TGG GGT GTG TGA GAA AAC ACA CAG AGA GAA AAA AAT ATC TCA CAA AAT ATA TAC ACT CTG TGA GAG AGT GTC TCC CCT CTC TCT CTT TTT TTA TAT ATG TGC GCA CAT ATT TTT TTG TGC GCA CAT ATC TCA CAG AGA GAG AGG GGC GCT CTG TGC GCT CTC TCG CGT GTG TGT GTT TTG TGT GTA TAC ACG CGA GAT ATC TCA CAA AAT ATT TTT TTT TTC TCT CTC TCC CCC CCG CGC GCA CAC ACT CTC TCT CTT TTG TGA GAA AAA AAC ACT CTG TGC GCT CTC TCA CAA AAA AAA AAT ATT TTC TCT CTG TGT GTG TGC GCG CGT GTG TGT GTT TTT TTT TTA TAC ACA CAG AGA GAA AAG AGG GGC GCC CCG CGC GCT CTA TAT ATA TAA AAC ACA CAA AAT ATA TAC ACT CTA TAG AGA GAT ATG TGG GGA GAA AAT ATT TTT TTC TCA CAC ACA CAG AGT GTA TAT ATT TTC TCA CAC ACT CTG TGA GAG AGA GAC ACT CTC TCA CAT ATT TTG TGA GAT ATG TGC GCT CTA TAT ATG TGA GAT ATG TGT GTT TTC TCA CAC ACA CAT ATC TCT CTG TGA GAT ATT TTT TTG TGG GGC GCT CTA TAC ACT CTA TAA AAC ACA CAA AAT ATC TCT CTA TAG AGT GTT TTG TGT GTA TAA AAT ATG TGG GGC GCC CCT CTA TAC ACA CAT ATT TTA TAC ACA CAG AGG GGT GTG TGG GGT GTG TGT GTT TTG TGT GTT TTC TCA CAG AGT GTT TTG TGA GAC ACT CTT TTC TCG CGC GCA CAG AGT GTG TGG GGC GCT CTA TAA AAC ACT CTA TAA AAC ACA CAT ATC TCT CTT TTT TTG TGG GGC GCA CAC ACT CTG TGT GTT TTT TTA TAT ATG TGA GAA AAA AAA AAA AAC ACT CTC TCA CAA AAA AAC ACC CCC CCG CGT GTC TCC CCT CTT TTG TGA GAT ATT TTG TGG GGC GCT CTT TTG TGA GAA AAG AGA GAG AGA GAA AAG AGT GTT TTT TTA TAA AAG AGG GGA GAA AAG AGG GGT GTG TGT GTA TAG AGA GAG AGT GTT TTT TTC TCT CTT TTA TAG AGA GAG AGA GAC ACG CGG GGT GTT TTG TGG GGG GGA GAA AAA AAT ATT TTG TGT GTT TTA TAA AAA AAT ATT TTT TTA TAT ATC TCT CTC TCA CAA AAC ACC CCT CTG TGT GTG TGC GCT CTT TTG TGT GTG TGA GAA AAA AAT ATT TTG TGT GTC TCG CGG GGT GTG TGG GGA GAC ACA CAA AAA AAT ATT TTG TGT GTC TCA CAC ACC CCT CTG TGT GTG TGC GCA CAA AAA AAG AGG GGA GAA AAA AAT ATT TTA TAA AAG AGG GGA GAG AGA GAG AGT GTG TGT GTT TTC TCA CAG AGA GAC ACA CAT ATT TTC TCT CTT TTT TTA TAA AAG AGC GCT CTT TTG TGT GTA TAA AAA AAT ATA TAA AAA AAT ATT TTT TTT TTT TTG TGG GGC GCT CTT TTT TTG TGT GTG TGT GTG TGC GCT CTG TGA GAC ACT CTC TCT CTA TAT ATC TCA CAT ATT TTA TAT ATT TTG TGG GGT GTG TGG GGA GAG AGC GCT CTA TAA AAA AAC ACT CTT TTA TAA AAA AAG AGC GCC CCT CTT TTG TGA GAA AAT ATT TTT TTA TAG AGG GGT GTG TGA GAA AAA AAC ACA CAT ATT TTT TTG TGT GTC TCA CAC ACG CGC GCA CAC ACT CTC TCA CAA AAA AAG AGG GGG GGA GAT ATT TTG TGT GTA TAC ACA CAG AGA GAA AAA AAG AGT GTG TGT GTG TGT GTT TTA TAA AAA AAT ATC TCC CCA CAG AGA GAG AGA GAA AAG AGA GAA AAA AAC ACT CTG TGG GGC GCC CCT CTA TAC ACT CTC TCA CAT ATG TGC GCC CCT CTC TCT CTA TAA AAA AAA AAG AGC GCC CCC CCC CCA CAA AAA AAA AAG AGA GAA AAA AAT ATT TTA TAT ATC TCT CTT TTC TCT CTT TTA TAG AGA GAG AGG GGG GGA GAG AGA GAA AAA AAC ACA CAC ACT CTT TTC TCC CCC CCA CAC ACA CAG AGA GAA AAG AGT GTG TGT GTT TTA TAA AAC ACA CAG AGA GAG AGG GGA GAA AAG AGT GTT TTG TGT GTC TCT CTT TTG TGA GAA AAA AAA AAC ACT CTG TGG GGT GTG TGA GAT ATT TTT TTA TAC ACA CAA AAC ACC CCA CAT ATT TTA TAG AGA GAA AAC ACA CAA AAC ACC CCT CTA TAC ACT CTA TAG AGT GTG TGA GAA AAG AGC GCT CTG TGT GTT TTG TGA GAA AAG AGC GCT CTC TCC CCA CAT ATT TTG TGG GGT GTT TTG TGG GGT GTA TAC ACA CAC ACC CCA CAG AGT GTT TTT TTG TGT GTA TAT ATT TTA TAA AAC ACG CGG GGG GGC GCT CTT TTA TAT ATG TGT GTT TTG TGC GCT CTC TCG CGA GAA AAA AAT ATC TCA CAA AAA AAG AGA GAC ACA CAC ACA CAG AGA GAA AAA AAA AAG AGT GTA TAC ACT CTG TGT GTG TGC GCC CCC CCT CTT TTG TGC GCA CAC ACC CCT CTA TAA AAT ATA TAT ATG TGA GAT ATG TGG GGT GTA TAA AAC ACA CAA AAA AAC ACA CAA AAT ATA TAC ACC CCT CTT TTC TCA CAC ACA CAC ACT CTC TCA CAA AAA AAG AGG GGC GCG CGG GGT GTG TGC GCA CAC ACC CCA CAA AAC ACA CAA AAA AAG AGG GGT GTT TTA TAC ACT CTT TTT TTT TTG TGG GGT GTG TGA GAT ATG TGA GAC ACA CAC ACT CTG TGT GTG TGA GAT ATA TAG AGA GAA AAG AGT GTG TGC GCA CAA AAG AGG GGT GTT TTA TAC ACA CAA AAG AGA GAG AGT GTG TGT GTG TGA GAA AAT ATA TAT ATC TCA CAC ACT CTT TTT TTT TTG TGA GAA AAC ACT CTT TTG TGA GAT ATG TGA GAA AAA AAG AGG GGA GAT ATT TTG TGA GAT ATA TAA AAA AAG AGT GTA TAC ACT CTT TTA TAA AAT ATG TGA GAG AGA GAA AAG AGT GTG TGC GCT CTC TCT CTG TGC GCC CCT CTA TAT ATA TAC ACA CAG AGT GTT TTG TGA GAA AAC ACT CTC TCG CGG GGT GTA TAC ACA CAG AGA GAA AAG AGT GTA TAA AAA AAT ATG TGA GAG AGT GTT TTC TCG CGC GCC CCT CTG TGT GTG TGT GTT TTG TGT GTG TGG GGC GCA CAG AGA GAT ATG TGC GCT CTG TGT GTC TCA CAT ATA TAA AAA AAA AAA AAC ACT CTT TTT TTG TGC GCA CAA AAC ACC CCA CAG AGT GTA TAT ATC TCT CTG TGA GAA AAT ATT TTA TAC ACT CTT TTA TAC ACA CAC ACC CCA CAC ACT CTG TGG GGG GGC GCA CAT ATT TTG TGA GAT ATT TTT TTA TAG AGA GAT ATG TGA GAG AGT GTG TGG GGA GAG AGT GTA TAT ATG TGG GGC GCT CTA TAC ACA CAT ATA TAC ACT CTA TAC ACT CTT TTA TAT ATT TTT TTG TGA GAT ATG TGA GAG AGT GTC TCT CTG TGG GGT GTG TGA GAG AGT GTT TTT TTA TAA AAA AAT ATT TTG TGG GGC GCT CTT TTC TCA CAC ACA CAT ATA TAT ATG TGT GTA TAT ATT TTG TGT GTT TTC TCT CTT TTT TTT TTT TTA TAC ACC CCC CCT CTC TCC CCA CAG AGA GAT ATG TGA GAG AGG GGA GAT ATG TGA GAA AAG AGA GAA AAG AGA GAA AAG AGG GGT GTG TGA GAT ATT TTG TGT GTG TGA GAA AAG AGA GAA AAG AGA GAA AAG AGA GAG AGT GTT TTT TTG TGA GAG AGC GCC CCA CAT ATC TCA CAA AAC ACT CTC TCA CAA AAT ATA TAT ATG TGA GAG AGT GTA TAT ATG TGG GGT GTA TAC ACT CTG TGA GAA AAG AGA GAT ATG TGA GAT ATT TTA TAC ACC CCA CAA AAG AGG GGT GTA TAA AAA AAC ACC CCT CTT TTT TTG TGG GGA GAA AAT ATT TTT TTG TGG GGT GTG TGC GCC CCA CAC ACT CTT TTC TCT CTG TGC GCT CTG TGC GCT CTC TCT CTT TTC TCA CAA AAC ACC CCT CTG TGA GAA AAG AGA GAA AAG AGA GAG AGC GCA CAA AAG AGA GAA AAG AGA GAA AAG AGA GAT ATT TTG TGG GGT GTT TTA TAG AGA GAT ATG TGA GAT ATG TGA GAT ATA TAG AGT GTC TCA CAA AAC ACA CAA AAA AAC ACT CTG TGT GTT TTG TGG GGT GTC TCA CAA AAC ACA CAA AAG AGA GAC ACG CGG GGC GCA CAG AGT GTG TGA GAG AGG GGA GAC ACA CAA AAT ATC TCA CAG AGA GAC ACA CAA AAC ACT CTA TAC ACT CTA TAT ATT TTC TCA CAA AAA AAC ACA CAA AAT ATT TTG TGT GTT TTG TGA GAG AGG GGT GTT TTC TCA CAA AAC ACC CCT CTC TCA CAA AAT ATT TTA TAG AGA GAG AGA GAT ATG TGG GGA GAA AAC ACT CTT TTA TAC ACA CAC ACC CCA CAG AGT GTT TTG TGT GTT TTC TCA CAG AGA GAC ACT CTA TAT ATT TTG TGA GAA AAG AGT GTG TGA GAA AAT ATA TAG AGT GTT TTT TTT TTA TAG AGT GTG TGG GGT GTT TTA TAT ATT TTT TTA TAA AAA AAA AAC ACT CTT TTA TAC ACT CTG TGA GAC ACA CAA AAT ATG TGT GTA TAT ATA TAC ACA CAT ATT TTA TAA AAA AAA AAA AAT ATG TGC GCA CAG AGA GAC ACA CAT ATT TTG TGT GTG TGG GGA GAA AAG AGA GAA AAG AGC GCT CTA TAA AAA AAA AAA AAG AGG GGT GTA TAA AAA AAA AAC ACC CCA CAA AAC ACA CAG AGT GTG TGG GGT GTT TTG TGT GTT TTA TAA AAT ATG TGC GCA CAG AGC GCC CCA CAA AAT ATG TGT GTT TTT TTA TAC ACC CCT CTT TTA TAA AAA AAC ACA CAT ATG TGG GGA GAG AGG GGA GAG AGG GGT GTG TGT GTT TTG TGC GCA CAG AGG GGA GAG AGC GCC CCT CTT TTA TAA AAA AAT ATA TAA AAG AGG GGC GCT CTA TAC ACT CTA TAA AAC ACA CAA AAT ATG TGC GCC CCA CAT ATG TGC GCA CAA AAG AGT GTT TTG TGA GAA AAT ATC TCT CTG TGA GAT ATG TGA GAT ATT TTA TAC ACA CAT ATA TAG AGC GCT CTA TAC ACT CTA TAA AAT ATG TGG GGA GAC ACC CCA CAC ACT CTT TTA TAA AAA AAG AGT GTG TGG GGG GGT GTG TGG GGT GTA TAG AGT GTT TTG TGT GTG TGT GTT TTT TTT TTA TAA AAG AGC GCG CGG GGA GAC ACA CAC ACA CAA AAT ATC TCT CTT TTG TGC GCT CTA TAA AAA AAC ACA CAC ACT CTG TGT GTC TCT CTT TTC TCA CAT ATG TGT GTT TTG TGT GTC TCG CGG GGC GCC CCC CCA CAA AAA AAT ATG TGT GTT TTA TAA AAC ACA CAA AAA AAG AGG GGT GTG TGA GAA AAG AGA GAC ACA CAT ATT TTC TCA CAA AAC ACT CTT TTC TCT CTT TTA TAA AAG AGA GAG AGT GTG TGC GCT CTT TTA TAT ATG TGA GAA AAA AAA AAT ATT TTT TTT TTA TAA AAT ATC TCA CAG AGC GCA CAC ACG CGA GAA AAG AGT GTT TTC TCT CTA TAC ACT CTT TTG TGC GCA CAC ACC CCA CAT ATT TTA TAT ATT TTA TAT ATC TCA CAG AGC GCT CTG TGG GGT GTA TAT ATT TTT TTT TTT TTG TGG GGT GTG TGC GCT CTG TGA GAC ACC CCC CCT CTA TAT ATA TAC ACA CAT ATT TTC TCT CTT TTT TTA TAA AAG AGA GAG AGT GTT TTT TTG TGT GTG TGT GTA TAG AGA GAT ATA TAC ACT CTG TGT GTT TTC TCG CGC GCA CAC ACA CAA AAA AAT ATG TGT GTC TCT CTA TAC ACT CTT TTA TAG AGC GCT CTG TGT GTC TCT CTT TTT TTG TGA GAT ATA TAA AAA AAA AAA AAT ATC TCT CTC TCT CTA TAT ATG TGA GAC ACA CAA AAA AAC ACT CTT TTG TGT GTT TTT TTC TCA CAA AAG AGC GCT CTT TTT TTT TTT TTG TGG GGA GAA AAA AAT ATG TGA GAA AAG AGA GAG AGT GTG TGA GAA AAA AAA AAG AGC GCA CAA AAG AGT GTT TTG TGA GAA AAC ACA CAA AAA AAA AAG AGA GAT ATC TCG CGC GCT CTG TGA GAG AGA GAT ATT TTC TCC CCT CTA TAA AAA AAG AGA GAG AGG GGA GAA AAG AGT GTT TTA TAA AAG AGC GCC CCA CAT ATT TTT TTA TAT ATA TAA AAC ACT CTG TGA GAA AAA AAG AGT GTA TAA AAA AAC ACC CCT CTT TTC TCA CAG AGT GTT TTG TGA GAA AAC ACA CAG AGA GAG AGA GAA AAA AAA AAC ACA CAA AAG AGA GAT ATG TGA GAT ATA TAA AAG AGA GAA AAA AAA AAT ATC TCA CAA AAA AAG AGC GCT CTT TTG TGT GTG TGT GTT TTG TGA GAA AAG AGA GAA AAG AGT GTT TTA TAC ACA CAA AAC ACA CAA AAC ACT CTC TCT CTG TGG GGA GAA AAG AGA GAA AAA AAC ACT CTA TAA AAG AGT GTT TTC TCC CCT CTC TCA CAC ACA CAG AGA GAA AAA AAA AAC ACT CTT TTG TGT GTT TTA TAC ACT CTT TTT TTA TAT ATA TAT ATT TTG TGA GAC ACA CAT ATT TTA TAA AAT ATG TGG GGC GCA CAA AAT ATC TCT CTT TTC TCA CAT ATC TCC CCA CAG AGA GAT ATT TTC TCT CTG TGC GCC CCA CAC ACT CTC TCT CTT TTG TGT GTT TTA TAG AGT GTG TGA GAC ACA CAT ATT TTG TGA GAC ACA CAT ATC TCA CAC ACT CTT TTT TTC TCT CTT TTA TAA AAA AAG AGA GAA AAA AAG AGA GAT ATG TGC GCT CTC TCC CCA CAT ATA TAT ATA TAT ATA TAG AGT GTG TGG GGG GGT GTG TGA GAT ATG TGT GTT TTG TGT GTT TTC TCA CAA AAG AGA GAG AGG GGG GGT GTG TGT GTT TTT TTT TTA TAA AAC ACT CTG TGC GCT CTG TGT GTG TGG GGT GTT TTA TAT ATA TAC ACC CCT CTA TAC ACT CTA TAA AAA AAA AAA AAG AGG GGC GCT CTG TGG GGT GTG TGG GGC GCA CAC ACT CTA TAC ACT CTG TGA GAA AAA AAT ATG TGC GCT CTA TAG AGC GCG CGA GAA AAA AAG AGC GCT CTT TTT TTG TGA GAG AGA GAA AAA AAA AAG AGT GTG TGC GCC CCA CAA AAC ACA CAG AGA GAC ACA CAA AAT ATT TTA TAT ATA TAT ATA TAA AAC ACC CCA CAC ACT CTT TTA TAC ACC CCC CCG CGG GGG GGT GTC TCA CAG AGG GGG GGT GTT TTT TTA TAA AAA AAT ATG TGG GGT GTT TTA TAC ACA CAC ACT CTG TGT GTA TAG AGA GAG AGG GGA GAG AGG GGC GCA CAA AAA AAG AGA GAC ACA CAG AGT GTG TGC GCT CTT TTA TAA AAA AAA AAA AAG AGT GTG TGT GTA TAA AAA AAA AAG AGT GTG TGC GCC CCT CTT TTT TTT TTA TAC ACA CAT ATT TTC TCT CTA TAC ACC CCA CAT ATC TCT CTA TAT ATT TTA TAT ATC TCT CTC TCT CTA TAA AAT ATG TGA GAG AGA GAA AAG AGC GCA CAA AAG AGA GAA AAA AAT ATT TTC TCT CTT TTG TGG GGA GAA AAC ACT CTG TGT GTT TTT TTC TCT CTT TTG TGG GGA GAA AAT ATT TTT TTG TGC GCG CGA GAG AGA GAA AAA AAT ATG TGC GCT CTT TTG TGC GCA CAC ACA CAT ATG TGC GCA CAG AGA GAA AAG AGA GAA AAA AAC ACA CAC ACG CGC GCA CAA AAA AAT ATT TTA TAA AAT ATG TGC GCC CCT CTG TGT GTC TCT CTG TGT GTG TGT GTG TGG GGA GAA AAA AAC ACT CTA TAA AAA AAG AGC GCC CCA CAT ATA TAG AGT GTT TTT TTC TCA CAA AAC ACT CTA TAT ATA TAC ACA CAG AGC GCG CGT GTA TAA AAA AAT ATA TAT ATA TAA AAG AGG GGG GGT GTA TAT ATT TTA TAA AAA AAA AAT ATA TAC ACA CAA AAG AGA GAG AGG GGG GGT GTG TGT GTG TGG GGT GTT TTG TGA GAT ATT TTA TAT ATG TGG GGT GTG TGC GCT CTA TAG AGA GAT ATT TTT TTT TTA TAC ACT CTT TTT TTT TTA TAC ACA CAC ACC CCA CAG AGT GTA TAA AAA AAA AAC ACA CAA AAC ACT CTG TGT GTA TAG AGC GCG CGT GTC TCA CAC ACT CTT TTA TAT ATC TCA CAA AAC ACA CAC ACA CAC ACT CTT TTA TAA AAC ACG CGA GAT ATC TCT CTA TAA AAA AAT ATG TGA GAA AAA AAC ACT CTC TCT CTT TTG TGT GTT TTA TAC ACA CAA AAT ATG TGC GCC CCA CAC ACT CTT TTG TGG GGC GCT CTA TAT ATG TGT GTA TAA AAC ACA CAC ACA CAT ATG TGG GGC GCT CTT TTA TAA AAA AAT ATT TTT TTG TGG GGA GAA AAG AGA GAA AAG AGC GCT CTG TGC GCT CTC TCG CGG GGT GTA TAT ATA TAT ATG TGA GAG AGA GAT ATC TCT CTC TCT CTC TCA CAA AAA AAG AGT GTG TGC GCC CCA CAG AGC GCT CTA TAC ACA CAG AGT GTT TTT TTC TCT CTG TGT GTT TTT TTC TCT CTT TTC TCA CAC ACC CCT CTG TGA GAT ATG TGC GCT CTG TGT GTT TTA TAC ACA CAG AGC GCG CGT GTA TAT ATA TAA AAT ATG TGG GGT GTT TTA TAT ATC TCT CTT TTA TAC ACT CTT TTC TCT CTT TTC TCT CTT TTC TCT CTA TAA AAA AAA AAC ACA CAC ACC CCT CTG TGA GAA AAG AGA GAA AAC ACA CAT ATT TTT TTT TTA TAT ATT TTG TGA GAA AAA AAC ACC CCA CAT ATC TCT CTC TCA CAC ACT CTT TTG TGC GCT CTG TGG GGT GTT TTC TCC CCT CTA TAT ATA TAA AAA AAG AGA GAT ATT TTG TGG GGT GTC TCC CCT CTA TAT ATT TTC TCT CTG TGG GGA GAC ACA CAA AAT ATC TCT CTA TAC ACA CAC ACA CAA AAC ACT CTA TAG AGG GGT GTA TAT ATA TAG AGA GAA AAT ATT TTT TTC TCT CTT TTA TAA AAG AGA GAG AGA GAG AGG GGT GTG TGA GAT ATA TAA AAA AAA AAG AGT GTG TGT GTA TAT ATA TAT ATT TTA TAC ACA CAC ACT CTA TAG AGT GTA TAA AAT ATC TCC CCT CTA TAC ACC CCA CAC ACA CAT ATT TTC TCC CCA CAC ACC CCT CTA TAG AGA GAT ATG TGG GGT GTG TGA GAA AAG AGT GTT TTA TAT ATC TCA CAC ACC CCT CTT TTT TTG TGA GAC ACA CAA AAT ATC TCT CTT TTA TAA AAG AGA GAC ACA CAC ACT CTT TTC TCT CTT TTT TTC TCT CTT TTT TTG TGA GAG AGA GAG AGA GAA AAG AGT GTG TGA GAG AGG GGA GAC ACT CTA TAT ATT TTA TAA AAG AGG GGT GTG TGT GTT TTT TTA TAC ACA CAA AAC ACA CAG AGT GTA TAG AGA GAC ACA CAA AAC ACA CAT ATT TTA TAA AAC ACC CCT CTC TCC CCA CAC ACA CAC ACG CGC GCA CAA AAG AGT GTT TTG TGT GTG TGG GGA GAC ACA CAT ATG TGT GTC TCA CAA AAT ATG TGA GAC ACA CAT ATA TAT ATG TGG GGA GAC ACA CAA AAC ACA CAG AGT GTT TTT TTG TGG GGT GTC TCC CCA CAA AAC ACT CTT TTA TAT ATT TTT TTG TGG GGA GAT ATG TGG GGA GAG AGC GCT CTG TGA GAT ATG TGT GTT TTA TAC ACT CTA TAA AAA AAA AAT ATA TAA AAA AAA AAC ACC CCT CTC TCA CAT ATA TAA AAT ATT TTC TCA CAC ACA CAT ATG TGA GAA AAG AGG GGT GTA TAA AAA AAA AAC ACA CAT ATT TTT TTT TTA TAT ATG TGT GTT TTT TTT TTA TAC ACC CCT CTA TAA AAT ATG TGA GAT ATG TGA GAC ACA CAC ACT CTC TCT CTA TAC ACG CGT GTG TGT GTT TTG TGA GAG AGG GGC GCT CTT TTT TTT TTG TGA GAG AGT GTA TAC ACT CTA TAC ACC CCA CAC ACA CAC ACA CAA AAC ACT CTG TGA GAT ATC TCC CCT CTA TAG AGT GTT TTT TTT TTC TCT CTG TGG GGG GGT GTA TAG AGG GGT GTA TAC ACA CAT ATG TGT GTC TCA CAG AGC GCA CAT ATT TTA TAA AAA AAT ATC TCA CAC ACA CAC ACT CTA TAA AAA AAA AAA AAG AGT GTG TGG GGA GAA AAA AAT ATA TAC ACC CCC CCA CAC ACA CAA AAG AGT GTT TTA TAA AAT ATG TGG GGT GTT TTT TTA TAA AAC ACT CTT TTC TCT CTA TAT ATT TTA TAA AAA AAT ATG TGG GGG GGC GCA CAG AGA GAT ATA TAA AAC ACA CAA AAC ACT CTG TGT GTT TTA TAT ATC TCT CTT TTG TGC GCC CCA CAC ACT CTG TGC GCA CAT ATT TTG TGT GTT TTA TAA AAC ACA CAC ACT CTC TCC CCA CAA AAC ACA CAA AAA AAT ATA TAG AGA GAG AGT GTT TTG TGA GAA AAG AGT GTT TTT TTA TAA AAT ATC TCC CCA CAC ACC CCT CTG TGC GCT CTC TCT CTA TAC ACA CAA AAG AGA GAT ATG TGC GCT CTT TTA TAT ATT TTA TAC ACA CAG AGA GAG AGC GCA CAA AAG AGG GGG GGC GCT CTG TGG GGT GTG TGA GAA AAG AGC GCT CTG TGC GCT CTA TAA AAC ACT CTT TTT TTT TTG TGT GTG TGC GCA CAC ACT CTT TTA TAT ATC TCT CTT TTA TAG AGC GCC CCT CTA TAC ACT CTG TGT GTA TAA AAT ATA TAA AAG AGA GAC ACA CAG AGT GTA TAG AGG GGT GTG TGA GAG AGT GTT TTA TAG AGG GGT GTG TGA GAT ATG TGT GTT TTA TAG AGA GAG AGA GAA AAA AAC ACA CAA AAT ATG TGA GAG AGT GTT TTA TAC ACT CTT TTG TGT GTT TTT TTC TCA CAA AAC ACA CAT ATG TGC GCC CCA CAA AAT ATT TTT TTA TAG AGA GAT ATT TTC TCT CTT TTG TGC GCA CAA AAA AAA AAG AGA GAG AGT GTC TCT CTT TTG TGA GAA AAC ACG CGT GTG TGG GGT GTG TGT GTG TGT GTA TAA AAA AAA AAC ACT CTT TTG TGT GTG TGG GGA GAC ACA CAA AAC ACA CAG AGC GCA CAG AGA GAC ACA CAA AAC ACC CCC CCT CTT TTA TAA AAG AGG GGG GGT GTG TGT GTA TAG AGA GAA AAG AGC GCT CTG TGT GTT TTA TAT ATG TGT GTA TAC ACA CAT ATG TGG GGG GGC GCA CAC ACA CAC ACT CTT TTT TTC TCT CTT TTA TAT ATG TGA GAA AAC ACA CAA AAT ATT TTT TTA TAA AAG AGA GAA AAA AAG AGG GGT GTG TGT GTT TTC TCA CAG AGA GAT ATA TAC ACC CCT CTT TTG TGT GTA TAC ACG CGT GTG TGT GTG TGG GGT GTA TAA AAA AAC ACA CAA AAG AGC GCT CTA TAC ACA CAA AAA AAA AAT ATA TAT ATC TCT CTA TAG AGT GTA TAC ACA CAA AAC ACA CAG AGG GGA GAG AGT GTC TCA CAC ACC CCT CTT TTT TTT TTG TGT GTT TTA TAT ATG TGA GAT ATG TGT GTC TCA CAG AGC GCA CAC ACC CCA CAC ACC CCT CTG TGC GCT CTC TCA CAG AGT GTA TAT ATG TGA GAA AAC ACT CTT TTA TAA AAG AGC GCA CAT ATG TGG GGT GTA TAC ACA CAT ATT TTT TTA TAC ACT CTT TTG TGT GTG TGC GCT CTA TAG AGT GTG TGA GAG AGT GTA TAC ACA CAC ACT CTG TGG GGT GTA TAA AAT ATT TTA TAC ACC CCA CAG AGT GTG TGT GTG TGG GGT GTC TCA CAC ACT CTA TAT ATA TAA AAA AAC ACA CAT ATA TAT ATA TAA AAC ACT CTT TTC TCT CTA TAA AAA AAG AGA GAA AAA AAC ACT CTT TTT TTG TGT GTA TAT ATT TTG TGC GCA CAT ATA TAG AGA GAC ACG CGG GGT GTG TGC GCT CTT TTT TTA TAC ACT CTT TTA TAC ACA CAA AAA AAG AGT GTC TCC CCT CTC TCA CAG AGA GAA AAT ATA TAC ACA CAA AAA AAG AGG GGT GTC TCC CCT CTA TAT ATT TTA TAC ACG CGG GGA GAT ATG TGT GTT TTT TTT TTC TCT CTA TAC ACA CAA AAA AAG AGA GAA AAA AAA AAC ACA CAG AGT GTT TTA TAC ACA CAC ACA CAA AAC ACA CAA AAC ACC CCA CAT ATA TAA AAA AAA AAC ACC CCA CAG AGT GTT TTA TAC ACT CTT TTA TAT ATA TAA AAA AAT ATT TTG TGG GGA GAT ATG TGG GGT GTG TGT GTT TTG TGT GTT TTT TTG TGT GTA TAC ACA CAG AGA GAA AAA AAT ATT TTG TGA GAC ACC CCC CCT CTA TAA AAG AGT GTT TTG TGG GGA GAC ACA CAA AAT ATT TTA TAT ATT TTA TAT ATA TAA AAG AGA GAA AAA AAG AGA GAC ACA CAA AAT ATT TTC TCT CTT TTA TAT ATT TTT TTC TCA CAC ACA CAG AGA GAG AGC GCA CAA AAC ACC CCA CAA AAT ATT TTG TGA GAT ATC TCT CTT TTG TGT GTA TAC ACC CCA CAA AAA AAC ACC CCA CAA AAC ACC CCA CAT ATA TAT ATC TCC CCA CAA AAA AAC ACG CGC GCA CAA AAG AGC GCT CTT TTC TCG CGA GAT ATA TAA AAT ATT TTT TTT TTA TAA AAG AGT GTT TTT TTG TGT GTA TAT ATG TGT GTG TGA GAT ATA TAA AAT ATA TAT ATC TCA CAA AAA AAT ATT TTT TTG TGC GCT CTG TGA GAT ATG TGA GAT ATT TTT TTA TAA AAA AAC ACC CCA CAG AGT GTT TTA TAA AAC ACT CTG TGG GGT GTT TTA TAT ATA TAA AAG AGA GAA AAA AAC ACC CCT CTG TGC GCT CTT TTC TCA CAA AAG AGA GAG AGA GAG AGC GCT CTT TTA TAA AAA AAG AGT GTT TTA TAC ACA CAT ATT TTT TTT TTT TTC TCC CCC CCT CTG TGA GAC ACT CTT TTA TAA AAA AAT ATG TGG GGT GTG TGA GAT ATG TGT GTG TGG GGT GTG TGG GGC GCT CTA TAT ATT TTG TGA GAT ATT TTA TAT ATA TAA AAA AAC ACA CAC ACT CTA TAC ACA CAC ACA CAC ACC CCC CCT CTC TCT CTT TTT TTT TTA TAA AAG AGA GAA AAA AAG AGG GGA GAG AGC GCT CTA TAA AAA AAT ATT TTG TGT GTT TTA TAC ACA CAT ATA TAA AAA AAC ACC CCT CTA TAT ATT TTG TGT GTT TTT TTG TGG GGC GCA CAT ATG TGT GTT TTA TAA AAC ACA CAA AAT ATG TGC GCA CAA AAC ACT CTA TAA AAT ATA TAA AAA AAG AGC GCC CCA CAC ACG CGT GTA TAT ATA TAA AAA AAC ACC CCA CAA AAA AAT ATA TAC ACC CCT CTG TGG GGT GTG TGT GTA TAT ATA TAC ACG CGT GTT TTG TGT GTC TCT CTT TTT TTG TGG GGA GAG AGC GCA CAC ACA CAA AAA AAA AAC ACC CCA CAG AGT GTT TTG TGA GAA AAA AAC ACA CAT ATC TCA CAA AAA AAT ATT TTC TCG CGT GTT TTT TTG TGA GAT ATG TGT GTA TAC ACT CTG TGA GAA AAG AGT GTC TCA CAG AGA GAG AGG GGA GAC ACG CGC GCG CGC GCA CAG AGG GGG GGA GAA AAT ATG TGG GGA GAT ATA TAA AAT ATC TCT CTT TTG TGC GCC CCT CTG TGC GCG CGA GAA AAG AGA GAT ATC TCT CTA TAA AAA AAA AAC ACC CCA CAG AGT GTC TCT CTC TCT CTG TGA GAA AAG AGA GAA AAG AGT GTA TAG AGT GTG TGG GGA GAA AAA AAA AAT ATC TCC CCT CTA TAC ACC CCA CAT ATA TAC ACA CAG AGA GAA AAA AAG AGA GAC ACG CGT GTT TTC TCT CTT TTG TGA GAG AGT GTG TGT GTA TAA AAT ATG TGT GTG TGA GAA AAA AAA AAC ACT CTA TAC ACC CCG CGA GAA AAG AGT GTT TTG TGT GTA TAG AGG GGA GAG AGA GAC ACA CAT ATT TTA TAT ATA TAC ACT CTT TTA TAA AAA AAC ACC CCA CAG AGC GCA CAA AAA AAT ATA TAA AAT ATA TAG AGT GTT TTT TTA TAA AAA AAA AAA AAT ATT TTA TAC ACA CAG AGA GAA AAG AGA GAG AGG GGT GTT TTG TGG GGC GCC CCA CAC ACA CAC ACA CAG AGA GAT ATC TCT CTA TAA AAT ATG TGG GGC GCT CTG TGC GCT CTT TTA TAT ATG TGT GTA TAG AGA GAC ACA CAA AAT ATT TTC TCT CTA TAG AGT GTC TCT CTT TTA TAC ACT CTA TAT ATT TTA TAA AAG AGA GAA AAA AAC ACC CCT CTA TAA AAT ATG TGA GAA AAT ATT TTA TAT ATC TCT CTA TAG AGA GAG AGT GTA TAT ATT TTA TAG AGG GGT GTT TTT TTG TGA GAA AAA AAA AAC ACC CCC CCT CTT TTG TGC GCT CTA TAC ACT CTC TCA CAT ATG TGG GGT GTT TTT TTA TAG AGC GCT CTG TGC GCT CTG TGT GTT TTA TAA AAT ATA TAG AGT GTG TGT GTC TCC CCC CCT CTT TTG TGG GGG GGA GAT ATA TAC ACT CTA TAT ATA TAG AGC GCT CTA TAA AAT ATT TTA TAT ATG TGC GCT CTA TAA AAG AGC GCC CCT CTT TTT TTT TTC TCT CTT TTA TAA AAC ACA CAA AAA AAG AGT GTT TTG TGT GTT TTA TAG AGT GTA TAC ACA CAA AAC ACT CTA TAC ACT CTA TAA AAC ACA CAT ATA TAG AGT GTT TTA TAC ACA CAC ACG CGG GGT GTG TGT GTT TTT TTA TAA AAA AAC ACC CCG CGT GTG TGT GTT TTT TTG TGT GTA TAC ACT CTA TAA AAT ATT TTA TAT ATA TAT ATG TGC GCC CCT CTT TTA TAT ATT TTT TTC TCT CTT TTT TTA TAC ACT CTT TTT TTA TAT ATT TTG TGC GCT CTA TAC ACA CAA AAT ATT TTG TGT GTG TGT GTA TAC ACT CTT TTT TTT TTA TAC ACT CTA TAG AGA GAA AAG AGT GTA TAC ACA CAA AAA AAT ATT TTC TCT CTA TAG AGA GAA AAT ATT TTA TAA AAA AAG AGC GCA CAT ATC TCT CTA TAT ATG TGC GCC CCG CGA GAC ACT CTA TAC ACT CTA TAT ATA TAG AGC GCA CAA AAA AAG AGA GAA AAT ATA TAC ACT CTG TGT GTT TTA TAA AAG AGA GAG AGT GTG TGT GTC TCG CGG GGT GTA TAA AAA AAT ATT TTT TTT TTG TGT GTC TCT CTA TAG AGA GAG AGG GGC GCT CTT TTC TCA CAT ATT TTT TTA TAA AAT ATT TTA TAT ATT TTT TTG TGA GAA AAG AGT GTC TCA CAC ACC CCT CTA TAA AAT ATT TTT TTT TTT TTC TCT CTA TAA AAA AAC ACT CTG TGA GAT ATA TAA AAA AAT ATA TAT ATT TTA TAT ATA TAA AAT ATT TTT TTG TGG GGT GTT TTT TTT TTT TTA TAC ACT CTA TAT ATT TTA TAA AAG AGT GTG TGT GTT TTT TTG TGC GCC CCT CTA TAG AGG GGT GTT TTC TCT CTT TTT TTA TAA AAT ATC TCT CTA TAC ACT CTC TCA CAA AAC ACC CCG CGC GCT CTG TGC GCT CTT TTT TTA TAG AGG GGT GTG TGT GTT TTT TTT TTA TAA AAT ATG TGT GTC TCT CTA TAA AAT ATT TTT TTA TAG AGG GGC GCA CAT ATG TGC GCC CCT CTT TTC TCT CTT TTA TAC ACT CTG TGT GTA TAC ACT CTG TGG GGT GTT TTA TAC ACA CAG AGA GAG AGA GAA AAG AGG GGC GCT CTA TAT ATT TTT TTG TGA GAA AAC ACT CTC TCT CTA TAC ACT CTA TAA AAT ATG TGT GTC TCA CAC ACT CTA TAT ATT TTG TGC GCA CAA AAC ACC CCT CTA TAC ACT CTG TGT GTA TAC ACT CTG TGG GGT GTT TTC TCT CTA TAT ATA TAC ACC CCT CTT TTG TGT GTA TAG AGT GTG TGT GTT TTT TTG TGT GTC TCT CTT TTA TAG AGT GTG TGG GGT GTT TTT TTA TAG AGA GAT ATT TTC TCT CTT TTT TTA TAG AGA GAC ACA CAC ACC CCT CTA TAT ATC TCC CCT CTT TTC TCT CTT TTT TTA TAG AGA GAA AAA AAC ACT CTA TAT ATA TAC ACA CAA AAA AAT ATT TTA TAC ACC CCA CAT ATT TTT TTC TCA CAT ATC TCT CTT TTT TTT TTA TAA AAA AAT ATG TGG GGG GGA GAT ATT TTT TTA TAA AAC ACT CTG TGC GCT CTT TTT TTT TTG TGG GGC GCT CTT TTA TAG AGT GTT TTG TGC GCA CAG AGA GAG AGT GTG TGG GGT GTT TTT TTT TTT TTG TGG GGC GCA CAT ATA TAT ATA TAT ATT TTC TCT CTT TTT TTT TTC TCA CAC ACT CTA TAG AGG GGT GTT TTT TTT TTT TTC TCT CTA TAT ATG TGT GTA TAC ACT CTT TTG TGG GGA GAT ATT TTG TGG GGC GCT CTG TGC GCA CAA AAT ATC TCA CAT ATG TGC GCA CAA AAT ATT TTG TGT GTT TTT TTT TTT TTC TCA CAG AGC GCT CTA TAT ATT TTT TTT TTG TGC GCA CAG AGT GTA TAC ACA CAT ATT TTT TTT TTA TAT ATT TTA TAG AGT GTA TAA AAT ATT TTC TCT CTT TTG TGG GGC GCT CTT TTA TAT ATG TGT GTG TGG GGT GTT TTA TAA AAT ATA TAA AAT ATT TTA TAA AAT ATC TCT CTT TTG TGT GTA TAC ACA CAA AAA AAT ATG TGG GGC GCC CCC CCC CCG CGA GAT ATT TTT TTC TCA CAG AGC GCT CTA TAT ATG TGG GGT GTT TTA TAG AGA GAA AAT ATG TGT GTA TAC ACA CAT ATC TCT CTT TTC TCT CTT TTT TTG TGC GCA CAT ATC TCA CAT ATT TTT TTT TTA TAT ATT TTA TAT ATG TGT GTA TAT ATG TGG GGA GAA AAA AAA AAG AGT GTT TTA TAT ATG TGT GTG TGC GCA CAT ATG TGT GTT TTG TGT GTA TAG AGA GAC ACG CGG GGT GTT TTG TGT GTA TAA AAT ATT TTC TCA CAT ATC TCA CAA AAC ACT CTT TTG TGT GTA TAT ATG TGA GAT ATG TGT GTG TGT GTT TTA TAC ACA CAA AAA AAC ACG CGT GTA TAA AAT ATA TAG AGA GAG AGC GCA CAA AAC ACA CAA AAG AGA GAG AGT GTC TCG CGA GAA AAT ATG TGT GTA TAC ACA CAA AAC ACT CTA TAT ATT TTG TGT GTT TTA TAA AAT ATG TGG GGT GTG TGT GTT TTA TAG AGA GAA AAG AGG GGT GTC TCC CCT CTT TTT TTT TTA TAT ATG TGT GTC TCT CTA TAT ATG TGC GCT CTA TAA AAT ATG TGG GGA GAG AGG GGT GTA TAA AAA AAG AGG GGC GCT CTT TTT TTT TTG TGC GCA CAA AAA AAC ACT CTA TAC ACA CAC ACA CAA AAT ATT TTG TGG GGA GAA AAT ATT TTG TGT GTG TGT GTT TTA TAA AAT ATT TTG TGT GTG TGA GAT ATA TAC ACA CAT ATT TTC TCT CTG TGT GTG TGC GCT CTG TGG GGT GTA TAG AGT GTA TAC ACA CAT ATT TTT TTA TAT ATT TTA TAG AGT GTG TGA GAT ATG TGA GAA AAG AGT GTT TTG TGC GCG CGA GAG AGA GAG AGA GAC ACT CTT TTG TGT GTC TCA CAC ACT CTA TAC ACA CAG AGT GTT TTT TTA TAA AAA AAA AAG AGA GAC ACC CCA CAA AAT ATA TAA AAA AAT ATC TCC CCT CTA TAC ACT CTG TGA GAC ACC CCA CAG AGT GTC TCT CTT TTC TCT CTT TTA TAC ACA CAT ATC TCG CGT GTT TTG TGA GAT ATA TAG AGT GTG TGT GTT TTA TAC ACA CAG AGT GTG TGA GAA AAG AGA GAA AAT ATG TGG GGT GTT TTC TCC CCA CAT ATC TCC CCA CAT ATC TCT CTT TTT TTA TAC ACT CTT TTT TTG TGA GAT ATA TAA AAA AAG AGC GCT CTG TGG GGT GTC TCA CAA AAA AAA AAG AGA GAC ACT CTT TTA TAT ATG TGA GAA AAA AAG AGA GAC ACA CAT ATT TTC TCT CTC TCT CTC TCT CTC TCT CTC TCA CAT ATT TTT TTT TTG TGT GTT TTA TAA AAC ACT CTT TTA TAG AGA GAC ACA CAA AAC ACC CCT CTG TGA GAG AGA GAG AGC GCT CTA TAA AAT ATA TAA AAC ACA CAC ACT CTA TAA AAA AAG AGG GGT GTT TTC TCA CAT ATT TTG TGC GCC CCT CTA TAT ATT TTA TAA AAT ATG TGT GTT TTA TAT ATA TAG AGT GTT TTT TTT TTT TTG TGA GAT ATG TGG GGT GTA TAA AAA AAT ATC TCA CAA AAA AAA AAT ATG TGT GTG TGA GAA AAG AGA GAA AAT ATC TCA CAT ATC TCT CTG TGC GCA CAA AAA AAA AAT ATC TCA CAG AGC GCG CGT GTC TCT CTG TGT GTT TTT TTA TAC ACT CTA TAC ACA CAG AGT GTC TCA CAG AGC GCT CTT TTA TAT ATG TGT GTG TGT GTC TCA CAA AAC ACC CCT CTA TAT ATA TAC ACT CTG TGT GTT TTA TAC ACT CTA TAG AGA GAT ATC TCA CAG AGG GGC GCA CAT ATT TTA TAG AGT GTG TGT GTC TCT CTG TGA GAT ATG TGT GTT TTG TGG GGT GTG TGA GAT ATA TAG AGT GTG TGC GCG CGG GGA GAA AAG AGT GTT TTG TGC GCA CAG AGT GTT TTA TAA AAA AAA AAT ATG TGT GTT TTT TTG TGA GAT ATG TGC GCT CTT TTA TAC ACG CGT GTT TTA TAA AAT ATA TAC ACG CGT GTT TTT TTT TTC TCA CAT ATC TCA CAA AAC ACT CTT TTT TTT TTA TAA AAC ACG CGT GTA TAC ACC CCA CAA AAT ATG TGG GGA GAA AAA AAA AAA AAC ACT CTC TCA CAA AAA AAA AAC ACA CAC ACT CTA TAG AGT GTT TTG TGC GCA CAA AAC ACT CTG TGC GCA CAG AGA GAA AAG AGC GCT CTG TGA GAA AAC ACT CTT TTG TGC GCA CAA AAA AAG AGA GAA AAT ATG TGT GTG TGT GTC TCC CCT CTT TTA TAG AGA GAC ACA CAA AAT ATG TGT GTC TCT CTT TTA TAT ATC TCT CTA TAC ACT CTT TTT TTT TTA TAT ATT TTT TTC TCA CAG AGC GCA CAG AGC GCT CTC TCG CGG GGC GCA CAA AAG AGG GGG GGT GTT TTT TTG TGT GTT TTG TGA GAT ATT TTC TCA CAG AGA GAT ATG TGT GTA TAG AGA GAA AAA AAC ACT CTA TAA AAA AAG AGA GAT ATG TGT GTT TTG TGT GTT TTG TGA GAA AAT ATG TGT GTC TCT CTT TTA TAA AAA AAT ATT TTG TGT GTC TCA CAC ACA CAT ATC TCA CAA AAT ATC TCT CTG TGA GAC ACA CAT ATA TAG AGA GAA AAG AGT GTT TTA TAC ACT CTG TGG GGC GCG CGA GAT ATA TAG AGT GTT TTG TGT GTA TAA AAT ATA TAA AAC ACT CTA TAT ATA TAT ATG TGC GCT CTC TCA CAC ACC CCT CTA TAT ATA TAA AAC ACA CAA AAA AAG AGT GTT TTG TGA GAA AAA AAA AAC ACA CAT ATG TGA GAC ACA CAC ACC CCC CCC CCG CGT GTG TGA GAC ACC CCT CTT TTG TGG GGT GTG TGC GCT CTT TTG TGT GTA TAT ATT TTG TGA GAC ACT CTG TGT GTA TAG AGT GTG TGC GCG CGC GCG CGT GTC TCA CAT ATA TAT ATT TTA TAA AAT ATG TGC GCG CGC GCA CAG AGG GGT GTA TAG AGC GCA CAA AAA AAA AAA AAG AGT GTC TCA CAC ACA CAA AAC ACA CAT ATT TTG TGC GCT CTT TTT TTG TGA GAT ATA TAT ATG TGG GGA GAA AAC ACG CGT GTT TTA TAA AAA AAG AGA GAT ATT TTT TTC TCA CAT ATG TGT GTC TCA CAT ATT TTG TGT GTC TCT CTG TGA GAA AAC ACA CAA AAC ACT CTA TAC ACG CGA GAA AAA AAA AAC ACA CAA AAA AAT ATA TAC ACG CGT GTA TAG AGT GTG TGC GCT CTG TGC GCT CTA TAA AAA AAA AAA AAG AGA GAA AAT ATA TAA AAC ACT CTT TTA TAC ACC CCT CTT TTT TTT TTA TAA AAG AGT GTT TTG TGA GAC ACA CAT ATG TGT GTG TGC GCA CAA AAC ACT CTA TAC ACT CTA TAG AGA GAC ACA CAA AAG AGT GTT TTG TGT GTT TTA TAA AAT ATG TGT GTT TTG TGT GTA TAA AAC ACA CAA AAC ACA CAA AAA AAG AGA GAT ATA TAG AGC GCA CAC ACT CTT TTA TAA AAG AGG GGG GGT GTG TGG GGT GTA TAA AAA AAA AAT ATT TTG TGT GTT TTA TAA AAT ATA TAA AAT ATT TTG TGG GGT GTT TTG TGA GAA AAG AGC GCA CAG AGT GTT TTA TAA AAT ATT TTA TAA AAA AAG AGT GTT TTA TAC ACA CAC ACT CTT TTG TGT GTG TGT GTT TTC TCC CCT CTT TTT TTT TTT TTG TGT GTT TTG TGC GCT CTG TGC GCT CTA TAT ATT TTT TTT TTC TCT CTA TAT ATT TTT TTA TAA AAT ATA TAA AAC ACA CAC ACC CCT CTG TGT GTT TTC TCA CAT ATG TGT GTC TCA CAT ATG TGT GTC TCT CTA TAA AAA AAC ACA CAT ATA TAC ACT CTG TGA GAC ACT CTT TTT TTT TTC TCA CAA AAG AGT GTG TGA GAA AAA AAT ATC TCA CAT ATA TAG AGG GGA GAT ATA TAC ACA CAA AAG AGG GGC GCT CTA TAT ATT TTG TGA GAT ATG TGG GGT GTG TGG GGT GTG TGT GTC TCA CAC ACT CTC TCG CGT GTG TGA GAC ACA CAT ATA TAG AGC GCA CAT ATC TCT CTA TAC ACA CAG AGA GAT ATA TAC ACT CTT TTG TGT GTT TTT TTT TTG TGC GCT CTA TAA AAC ACA CAA AAA AAC ACA CAT ATG TGC GCT CTG TGA GAT ATT TTT TTT TTG TGA GAC ACA CAC ACA CAT ATG TGG GGT GTT TTT TTA TAG AGC GCC CCA CAG AGC GCG CGT GTG TGG GGT GTG TGG GGT GTA TAG AGT GTT TTA TAT ATA TAC ACT CTA TAA AAT ATG TGA GAC ACA CAA AAA AAG AGC GCT CTT TTG TGC GCC CCC CCA CAT ATT TTG TGA GAT ATT TTG TGC GCT CTG TGC GCA CAG AGT GTC TCA CAT ATA TAA AAC ACA CAA AAG AGA GAG AGA GAA AAG AGT GTG TGG GGG GGT GTT TTT TTT TTG TGT GTC TCG CGT GTG TGC GCC CCT CTG TGG GGT GTT TTT TTG TGC GCC CCT CTG TGG GGC GCA CAC ACG CGA GAT ATA TAT ATT TTA TAC ACG CGC GCA CAC ACA CAA AAC ACT CTA TAA AAT ATG TGG GGT GTG TGA GAC ACT CTT TTT TTT TTT TTG TGC GCA CAT ATT TTT TTC TCT CTT TTA TAC ACC CCT CTA TAG AGA GAG AGT GTT TTT TTT TTT TTA TAG AGT GTG TGC GCA CAG AGT GTT TTG TGG GGT GTA TAA AAC ACA CAT ATC TCT CTG TGT GTT TTA TAC ACA CAC ACA CAC ACC CCA CAT ATC TCA CAA AAA AAA AAC ACT CTT TTA TAT ATA TAG AGA GAG AGT GTA TAC ACA CAC ACT CTG TGA GAC ACT CTT TTT TTG TGC GCA CAA AAC ACA CAT ATC TCA CAG AGC GCT CTT TTG TGT GTG TGT GTT TTT TTT TTG TGG GGC GCT CTG TGC GCT CTG TGA GAA AAT ATG TGT GTA TAC ACA CAA AAT ATT TTT TTT TTT TTA TAA AAA AAG AGA GAT ATG TGC GCT CTT TTC TCT CTG TGG GGT GTA TAA AAG AGC GCC CCA CAG AGT GTA TAC ACC CCA CAT ATA TAT ATT TTG TGT GTT TTA TAT ATG TGA GAT ATA TAC ACC CCA CAA AAT ATG TGT GTA TAC ACT CTA TAG AGA GAA AAG AGG GGT GTT TTC TCT CTG TGT GTT TTG TGC GCT CTT TTA TAT ATG TGA GAA AAA AAG AGT GTT TTT TTA TAC ACG CGC GCC CCC CCT CTG TGA GAC ACA CAC ACA CAC ACG CGT GTT TTA TAT ATG TGT GTG TGC GCT CTC TCA CAT ATG TGG GGA GAT ATG TGG GGC GCT CTC TCT CTA TAT ATT TTA TAT ATT TTC TCA CAA AAT ATT TTT TTC TCC CCT CTA TAA AAC ACA CAC ACC CCT CTA TAC ACC CCT CTT TTG TGA GAA AAG AGG GGT GTT TTC TCT CTG TGT GTT TTA TAG AGA GAG AGT GTG TGG GGT GTA TAA AAC ACA CAA AAC ACT CTT TTT TTT TTG TGA GAT ATT TTC TCT CTG TGA GAG AGT GTA TAC ACT CTG TGT GTA TAG AGG GGC GCA CAC ACG CGG GGC GCA CAC ACT CTT TTG TGT GTG TGA GAA AAA AAG AGA GAT ATC TCA CAG AGA GAA AAG AGC GCT CTG TGG GGT GTG TGT GTT TTT TTG TGT GTG TGT GTA TAT ATC TCT CTA TAC ACT CTA TAG AGT GTG TGG GGT GTA TAG AGA GAT ATG TGG GGG GGT GTA TAC ACT CTT TTA TAA AAC ACA CAA AAT ATG TGA GAT ATT TTA TAT ATT TTA TAC ACA CAG AGA GAT ATC TCT CTT TTT TTA TAC ACC CCA CAG AGG GGA GAG AGT GTT TTT TTT TTC TCT CTG TGT GTG TGG GGT GTG TGT GTA TAG AGA GAT ATG TGC GCT CTG TGT GTA TAA AAA AAT ATT TTT TTA TAC ACT CTT TTA TAC ACT CTA TAA AAT ATA TAT ATG TGT GTT TTT TTA TAC ACA CAC ACC CCA CAC ACT CTA TAA AAT ATT TTC TCA CAA AAC ACC CCT CTA TAT ATT TTG TGG GGT GTG TGC GCT CTT TTT TTG TGG GGA GAC ACA CAT ATA TAT ATC TCA CAG AGC GCA CAT ATC TCT CTA TAT ATA TAG AGT GTA TAG AGC GCT CTG TGG GGT GTG TGG GGT GTA TAT ATT TTG TGT GTA TAG AGC GCT CTA TAT ATC TCG CGT GTA TAG AGT GTA TAA AAC ACA CAT ATG TGC GCC CCT CTT TTG TGC GCC CCT CTA TAC ACT CTA TAT ATT TTT TTT TTA TAT ATG TGA GAG AGG GGT GTT TTT TTA TAG AGA GAA AAG AGA GAG AGC GCT CTT TTT TTT TTG TGG GGT GTG TGA GAA AAT ATA TAC ACA CAG AGT GTC TCA CAT ATG TGT GTA TAG AGT GTT TTG TGC GCC CCT CTT TTT TTA TAA AAT ATA TAC ACT CTT TTT TTA TAC ACT CTA TAT ATT TTC TCC CCT CTT TTA TAT ATG TGT GTC TCA CAT ATT TTC TCA CAC ACT CTG TGT GTA TAC ACT CTC TCT CTG TGT GTT TTT TTA TAA AAC ACA CAC ACC CCA CAG AGT GTT TTT TTA TAC ACT CTC TCA CAT ATT TTC TCT CTT TTA TAC ACC CCT CTG TGG GGT GTG TGT GTT TTT TTA TAT ATT TTC TCT CTG TGT GTT TTA TAT ATT TTT TTA TAC ACT CTT TTG TGT GTA TAC ACT CTT TTG TGA GAC ACA CAT ATT TTT TTT TTA TAT ATC TCT CTT TTA TAC ACT CTA TAA AAT ATG TGA GAT ATG TGT GTT TTT TTC TCT CTT TTT TTT TTT TTT TTA TAG AGC GCA CAC ACA CAT ATA TAT ATT TTC TCA CAG AGT GTG TGG GGA GAT ATG TGG GGT GTT TTA TAT ATG TGT GTT TTC TCA CAC ACA CAC ACC CCT CTT TTT TTA TAG AGT GTA TAC ACC CCT CTT TTT TTC TCT CTG TGG GGA GAT ATA TAA AAC ACA CAA AAT ATT TTG TGC GCT CTT TTA TAT ATA TAT ATC TCA CAT ATT TTT TTG TGT GTA TAT ATT TTT TTC TCC CCA CAC ACA CAA AAA AAG AGC GCA CAT ATT TTT TTC TCT CTA TAT ATT TTG TGG GGT GTT TTC TCT CTT TTT TTA TAG AGT GTA TAA AAT ATT TTA TAC ACC CCT CTA TAA AAA AAG AGA GAG AGA GAC ACG CGT GTG TGT GTA TAG AGT GTC TCT CTT TTT TTA TAA AAT ATG TGG GGT GTG TGT GTT TTT TTC TCC CCT CTT TTT TTA TAG AGT GTA TAC ACT CTT TTT TTT TTG TGA GAA AAG AGA GAA AAG AGC GCT CTG TGC GCG CGC GCT CTG TGT GTG TGC GCA CAC ACC CCT CTT TTT TTT TTT TTG TGT GTT TTA TAA AAA AAT ATA TAA AAA AAG AGA GAA AAA AAT ATG TGT GTA TAT ATC TCT CTA TAA AAA AAG AGT GTT TTG TGC GCG CGT GTA TAG AGT GTG TGA GAT ATG TGT GTG TGC GCT CTA TAT ATT TTA TAC ACC CCT CTC TCT CTT TTA TAC ACG CGC GCA CAA AAT ATA TAT ATA TAA AAT ATA TAG AGA GAT ATA TAC ACT CTT TTA TAG AGC GCT CTC TCT CTT TTT TTA TAT ATA TAA AAT ATA TAA AAG AGT GTA TAC ACA CAA AAG AGT GTA TAT ATT TTT TTT TTA TAG AGT GTG TGG GGA GAG AGC GCA CAA AAT ATG TGG GGA GAT ATA TAC ACA CAA AAC ACT CTA TAG AGC GCT CTA TAC ACA CAG AGA GAG AGA GAA AAG AGC GCT CTG TGC GCT CTT TTG TGT GTT TTG TGT GTC TCA CAT ATC TCT CTC TCG CGC GCA CAA AAA AAG AGG GGC GCT CTC TCT CTC TCA CAA AAT ATG TGA GAC ACT CTT TTC TCA CAG AGT GTA TAA AAC ACT CTC TCA CAG AGG GGT GTT TTC TCT CTG TGA GAT ATG TGT GTT TTC TCT CTT TTT TTA TAC ACC CCA CAA AAC ACC CCA CAC ACC CCA CAC ACA CAA AAA AAC ACC CCT CTC TCT CTA TAT ATC TCA CAC ACC CCT CTC TCA CAG AGC GCT CTG TGT GTT TTT TTT TTG TGC GCA CAG AGA GAG AGT GTG TGG GGT GTT TTT TTT TTA TAG AGA GAA AAA AAA AAA AAT ATG TGG GGC GCA CAT ATT TTC TCC CCC CCA CAT ATC TCT CTG TGG GGT GTA TAA AAA AAG AGT GTT TTG TGA GAG AGG GGG GGT GTT TTG TGT GTA TAT ATG TGG GGT GTA TAC ACA CAA AAG AGT GTA TAA AAC ACT CTT TTG TGT GTG TGG GGT GTA TAC ACA CAA AAC ACT CTA TAC ACA CAC ACT CTT TTA TAA AAC ACG CGG GGT GTC TCT CTT TTT TTG TGG GGC GCT CTT TTG TGA GAT ATG TGA GAC ACG CGT GTA TAG AGT GTT TTT TTA TAC ACT CTG TGT GTC TCC CCA CAA AAG AGA GAC ACA CAT ATG TGT GTG TGA GAT ATC TCT CTG TGC GCA CAC ACC CCT CTC TCT CTG TGA GAA AAG AGA GAC ACA CAT ATG TGC GCT CTT TTA TAA AAC ACC CCC CCT CTA TAA AAT ATT TTA TAT ATG TGA GAA AAG AGA GAT ATT TTT TTA TAC ACT CTC TCA CAT ATT TTC TCG CGT GTA TAA AAG AGT GTC TCT CTA TAA AAT ATC TCA CAT ATA TAA AAT ATT TTT TTC TCT CTT TTG TGG GGT GTA TAC ACA CAG AGG GGC GCT CTG TGG GGT GTA TAA AAT ATG TGT GTT TTC TCA CAA AAC ACT CTC TCA CAG AGG GGG GGT GTT TTA TAT ATT TTG TGG GGA GAC ACA CAT ATT TTC TCT CTA TAT ATG TGC GCA CAA AAA AAA AAT ATT TTG TGT GTG TGT GTA TAC ACT CTT TTA TAA AAG AGC GCT CTT TTA TAA AAG AGG GGT GTT TTG TGA GAT ATA TAC ACA CAG AGC GCC CCA CAA AAT ATC TCC CCT CTA TAA AAG AGA GAC ACA CAC ACC CCT CTA TAA AAG AGT GTA TAT ATA TAA AAG AGT GTT TTT TTG TGT GTT TTC TCG CGC GCA CAT ATT TTC TCA CAA AAC ACC CCA CAG AGG GGA GAC ACA CAG AGA GAC ACT CTT TTT TTT TTT TTC TCA CAG AGT GTG TGT GTT TTA TAG AGC GCT CTT TTG TGT GTT TTA TAC ACA CAA AAT ATG TGG GGT GTT TTC TCA CAC ACC CCA CAT ATC TCT CTG TGG GGT GTG TGT GTT TTT TTA TAC ACC CCA CAA AAT ATG TGT GTG TGC GCT CTA TAT ATG TGA GAG AGG GGC GCC CCC CCA CAA AAT ATT TTT TTC TCA CAC ACT CTA TAT ATT TTA TAA AAG AGG GGG GGT GTT TTC TCA CAT ATT TTC TCC CCT CTT TTA TAA AAT ATG TGG GGT GTT TTC TCA CAT ATG TGT GTG TGG GGT GTA TAG AGT GTG TGT GTT TTG TGG GGT GTT TTT TTT TTA TAA AAC ACA CAT ATA TAG AGA GAT ATT TTA TAT ATG TGA GAC ACT CTG TGT GTG TGT GTC TCT CTC TCT CTT TTT TTT TTT TTG TGT GTT TTA TAC ACA CAT ATG TGC GCA CAC ACC CCA CAT ATA TAT ATG TGG GGA GAA AAT ATT TTA TAC ACC CCA CAA AAC ACT CTG TGG GGA GAG AGT GTT TTC TCA CAT ATG TGC GCT CTG TGG GGC GCA CAC ACA CAG AGA GAC ACT CTT TTA TAG AGA GAA AAG AGG GGT GTA TAA AAC ACT CTT TTT TTT TTA TAT ATG TGG GGA GAC ACC CCT CTT TTT TTT TTG TGT GTT TTG TGA GAC ACA CAG AGG GGC GCA CAA AAA AAC ACA CAG AGC GCA CAC ACA CAA AAG AGC GCA CAG AGC GCT CTG TGG GGT GTA TAC ACG CGG GGA GAC ACA CAC ACA CAA AAC ACT CTA TAT ATT TTA TAC ACA CAG AGT GTT TTA TAA AAT ATG TGT GTT TTT TTT TTA TAG AGC GCT CTT TTG TGG GGT GTT TTG TGT GTA TAC ACG CGC GCT CTG TGC GCT CTG TGT GTT TTA TAT ATA TAA AAA AAT ATG TGG GGA GAG AGA GAC ACA CAG AGG GGT GTG TGG GGT GTT TTT TTC TCT CTC TCA CAA AAT ATC TCG CGA GAT ATT TTT TTA TAC ACC CCA CAC ACA CAA AAC ACT CTC TCT CTT TTA TAA AAT ATG TGA GAC ACT CTT TTT TTA TAA AAC ACC CCT CTT TTG TGT GTG TGG GGC GCT CTA TAT ATG TGA GAA AAG AGT GTA TAC ACA CAA AAT ATT TTA TAT ATG TGA GAA AAC ACC CCT CTC TCT CTA TAA AAC ACA CAC ACA CAA AAG AGA GAC ACC CCA CAT ATG TGT GTT TTG TGA GAC ACA CAT ATA TAC ACT CTA TAG AGG GGA GAC ACC CCT CTC TCT CTT TTT TTC TCT CTG TGC GCT CTC TCA CAA AAA AAC ACT CTG TGG GGA GAA AAT ATT TTG TGC GCC CCG CGT GTT TTT TTT TTA TAG AGA GAT ATA TAT ATG TGT GTG TGT GTG TGC GCT CTT TTC TCA CAT ATT TTA TAA AAA AAA AAG AGA GAA AAT ATT TTA TAC ACT CTG TGC GCA CAA AAA AAA AAT ATG TGG GGT GTA TAT ATG TGA GAA AAT ATG TGG GGA GAC ACG CGT GTA TAC ACC CCA CAT ATA TAT ATT TTG TGG GGG GGT GTA TAG AGT GTG TGC GCT CTT TTT TTA TAT ATT TTA TAG AGA GAA AAG AGA GAT ATG TGA GAA AAT ATT TTT TTA TAC ACA CAC ACC CCT CTT TTT TTT TTG TGA GAT ATG TGT GTT TTG TGT GTT TTA TAG AGA GAC ACA CAA AAT ATG TGC GCT CTC TCA CAG AGG GGT GTG TGT GTT TTA TAC ACT CTT TTT TTC TCC CCA CAA AAA AAG AGT GTG TGC GCA CAG AGT GTG TGA GAA AAA AAA AAG AGA GAA AAC ACA CAA AAT ATC TCA CAA AAG AGG GGG GGT GTA TAC ACA CAC ACA CAC ACC CCA CAC ACT CTG TGG GGT GTT TTG TGT GTT TTA TAC ACT CTC TCA CAC ACA CAA AAT ATT TTT TTT TTG TGA GAC ACT CTT TTC TCA CAC ACT CTT TTT TTT TTA TAG AGT GTT TTT TTT TTA TAG AGT GTC TCC CCA CAG AGA GAG AGT GTA TAC ACT CTC TCA CAA AAT ATG TGG GGT GTC TCT CTT TTT TTG TGT GTT TTC TCT CTT TTT TTT TTT TTT TTT TTT TTG TGT GTA TAT ATG TGA GAA AAA AAA AAT ATG TGC GCC CCT CTT TTT TTT TTT TTA TAC ACC CCT CTT TTT TTT TTG TGC GCT CTA TAT ATG TGG GGG GGT GTA TAT ATT TTA TAT ATT TTG TGC GCT CTA TAT ATG TGT GTC TCT CTG TGC GCT CTT TTT TTT TTG TGC GCA CAA AAT ATG TGA GAT ATG TGT GTT TTT TTG TGT GTC TCA CAA AAA AAC ACA CAT ATA TAA AAG AGC GCA CAT ATG TGC GCA CAT ATT TTT TTC TCT CTC TCT CTG TGT GTT TTT TTG TGT GTT TTT TTT TTT TTG TGT GTT TTA TAC ACC CCT CTT TTC TCT CTC TCT CTT TTG TGC GCC CCA CAC ACT CTG TGT GTA TAG AGC GCT CTT TTA TAT ATT TTT TTT TTA TAA AAT ATA TAT ATG TGG GGT GTC TCT CTA TAT ATA TAT ATG TGC GCC CCT CTG TGC GCT CTA TAG AGT GTT TTG TGG GGG GGT GTG TGA GAT ATG TGC GCG CGT GTA TAT ATT TTA TAT ATG TGA GAC ACA CAT ATG TGG GGT GTT TTG TGG GGA GAT ATA TAT ATG TGG GGT GTT TTG TGA GAT ATA TAC ACT CTA TAG AGT GTT TTT TTG TGT GTC TCT CTG TGG GGT GTT TTT TTT TTA TAA AAG AGC GCT CTA TAA AAA AAA AAG AGA GAC ACT CTG TGT GTG TGT GTT TTA TAT ATG TGT GTA TAT ATG TGC GCA CAT ATC TCA CAG AGC GCT CTG TGT GTA TAG AGT GTG TGT GTT TTA TAC ACT CTA TAA AAT ATC TCC CCT CTT TTA TAT ATG TGA GAC ACA CAG AGC GCA CAA AAG AGA GAA AAC ACT CTG TGT GTG TGT GTA TAT ATG TGA GAT ATG TGA GAT ATG TGG GGT GTG TGC GCT CTA TAG AGG GGA GAG AGA GAG AGT GTG TGT GTG TGG GGA GAC ACA CAC ACT CTT TTA TAT ATG TGA GAA AAT ATG TGT GTC TCT CTT TTG TGA GAC ACA CAC ACT CTC TCG CGT GTT TTT TTA TAT ATA TAA AAA AAG AGT GTT TTT TTA TAT ATT TTA TAT ATG TGG GGT GTA TAA AAT ATG TGC GCT CTT TTT TTA TAG AGA GAT ATC TCA CAA AAG AGC GCC CCA CAT ATT TTT TTC TCC CCA CAT ATG TGT GTG TGG GGG GGC GCT CTC TCT CTT TTA TAT ATA TAA AAT ATC TCT CTC TCT CTG TGT GTT TTA TAC ACT CTT TTC TCT CTA TAA AAC ACT CTA TAC ACT CTC TCA CAG AGG GGT GTG TGT GTA TAG AGT GTT TTA TAC ACA CAA AAC ACT CTG TGT GTC TCA CAT ATG TGT GTT TTT TTT TTT TTG TGG GGC GCC CCA CAG AGA GAG AGG GGT GTA TAT ATT TTG TGT GTT TTT TTT TTT TTA TAT ATG TGT GTG TGT GTG TGT GTT TTG TGA GAG AGT GTA TAT ATT TTG TGC GCC CCC CCT CTA TAT ATT TTT TTT TTC TCT CTT TTC TCA CAT ATA TAA AAC ACT CTG TGG GGT GTA TAA AAT ATA TAC ACA CAC ACT CTT TTC TCA CAG AGT GTG TGT GTA TAT ATA TAA AAT ATG TGC GCT CTA TAG AGT GTT TTT TTA TAT ATT TTG TGT GTT TTT TTC TCT CTT TTA TAG AGG GGC GCT CTA TAT ATT TTT TTT TTT TTG TGT GTA TAC ACT CTT TTG TGT GTT TTA TAC ACT CTT TTT TTG TGG GGC GCC CCT CTC TCT CTT TTT TTT TTG TGT GTT TTT TTA TAC ACT CTC TCA CAA AAC ACC CCG CGC GCT CTA TAC ACT CTT TTT TTA TAG AGA GAC ACT CTG TGA GAC ACT CTC TCT CTT TTG TGG GGT GTG TGT GTT TTT TTA TAT ATG TGA GAT ATT TTA TAC ACT CTT TTA TAG AGT GTT TTT TTC TCT CTA TAC ACA CAC ACA CAG AGG GGA GAG AGT GTT TTT TTA TAG AGA GAT ATA TAT ATA TAT ATG TGA GAA AAT ATT TTC TCA CAC ACA CAG AGG GGG GGA GAC ACT CTA TAC ACT CTC TCC CCC CCA CAC ACC CCC CCA CAA AAG AGA GAA AAT ATA TAG AGC GCA CAT ATA TAG AGA GAT ATG TGC GCC CCT CTT TTC TCA CAA AAA AAC ACT CTC TCA CAA AAC ACA CAT ATT TTA TAA AAA AAT ATT TTG TGT GTT TTG TGG GGG GGT GTG TGT GTT TTG TGG GGT GTG TGG GGC GCA CAA AAA AAC ACC CCT CTT TTG TGT GTA TAT ATC TCA CAA AAA AAG AGT GTA TAG AGC GCC CCA CAC ACT CTG TGT GTA TAC ACA CAG AGT GTC TCT CTA TAA AAA AAA AAT ATG TGT GTC TCA CAG AGA GAT ATG TGT GTA TAA AAA AAG AGT GTG TGC GCA CAC ACA CAT ATC TCA CAG AGT GTA TAG AGT GTC TCT CTT TTA TAC ACT CTC TCT CTC TCA CAG AGT GTT TTT TTT TTG TGC GCA CAA AAC ACA CAA AAC ACT CTC TCA CAG AGA GAG AGT GTA TAG AGA GAA AAT ATC TCA CAT ATC TCA CAT ATC TCT CTA TAA AAA AAT ATT TTG TGT GTG TGG GGG GGC GCT CTC TCA CAA AAT ATG TGT GTG TGT GTC TCC CCA CAG AGT GTT TTA TAC ACA CAC ACA CAA AAT ATG TGA GAC ACA CAT ATT TTC TCT CTC TCT CTT TTA TAG AGC GCT CTA TAA AAA AAG AGA GAT ATA TAC ACT CTA TAC ACT CTG TGA GAA AAG AGC GCC CCT CTT TTT TTG TGA GAA AAA AAA AAA AAA AAT ATG TGG GGT GTT TTT TTC TCA CAC ACT CTA TAC ACT CTT TTT TTC TCT CTG TGT GTT TTT TTT TTG TGC GCT CTT TTT TTC TCC CCA CAT ATG TGC GCA CAG AGG GGG GGT GTG TGC GCT CTG TGT GTA TAG AGA GAC ACA CAT ATA TAA AAA AAC ACA CAA AAG AGC GCT CTT TTT TTG TGT GTG TGA GAA AAG AGA GAA AAA AAT ATG TGC GCT CTG TGG GGA GAC ACA CAA AAC ACA CAG AGG GGG GGC GCA CAA AAC ACC CCT CTT TTA TAC ACA CAA AAG AGC GCT CTA TAT ATA TAG AGC GCC CCT CTC TCA CAG AGA GAG AGT GTT TTT TTA TAG AGT GTT TTC TCC CCC CCT CTT TTC TCC CCA CAT ATC TCA CAT ATA TAT ATG TGC GCA CAG AGC GCT CTT TTT TTT TTG TGC GCT CTA TAC ACT CTG TGC GCT CTC TCA CAA AAG AGA GAA AAG AGC GCT CTT TTA TAT ATG TGA GAG AGC GCA CAG AGG GGC GCT CTG TGT GTT TTG TGC GCT CTA TAA AAT ATG TGG GGT GTG TGA GAT ATT TTC TCT CTG TGA GAA AAG AGT GTT TTG TGT GTT TTC TCT CTT TTA TAA AAA AAA AAA AAG AGT GTT TTG TGA GAA AAG AGA GAA AAG AGT GTC TCT CTT TTT TTG TGA GAA AAT ATG TGT GTG TGG GGC GCT CTA TAA AAA AAT ATC TCT CTG TGA GAA AAT ATT TTT TTG TGA GAC ACC CCG CGT GTG TGA GAT ATG TGC GCA CAG AGC GCC CCA CAT ATG TGC GCA CAA AAC ACG CGT GTA TAA AAG AGT GTT TTG TGG GGA GAA AAA AAA AAG AGA GAT ATG TGG GGC GCT CTG TGA GAT ATC TCA CAA AAG AGC GCT CTA TAT ATG TGA GAC ACC CCC CCA CAA AAA AAT ATG TGT GTA TAT ATA TAA AAA AAC ACA CAG AGG GGC GCT CTA TAG AGA GAT ATC TCT CTG TGA GAG AGG GGA GAC ACA CAA AAG AGA GAG AGG GGG GGC GCA CAA AAA AAA AAG AGT GTT TTA TAC ACT CTA TAG AGT GTG TGC GCT CTA TAT ATG TGC GCA CAG AGA GAC ACA CAA AAT ATG TGC GCT CTT TTT TTT TTC TCA CAC ACT CTA TAT ATG TGC GCT CTT TTA TAG AGA GAA AAA AAG AGT GTT TTG TGG GGA GAT ATA TAA AAT ATG TGA GAT ATG TGC GCA CAC ACT CTC TCA CAA AAC ACA CAA AAC ACA CAT ATT TTA TAT ATC TCA CAA AAC ACA CAA AAT ATG TGC GCA CAA AAG AGA GAG AGA GAT ATG TGG GGT GTT TTG TGT GTG TGT GTT TTC TCC CCC CCT CTT TTG TGA GAA AAC ACA CAT ATA TAA AAT ATA TAC ACC CCT CTC TCT CTT TTA TAC ACA CAA AAC ACA CAG AGC GCA CAG AGC GCC CCA CAA AAA AAC ACT CTA TAA AAT ATG TGG GGT GTT TTG TGT GTC TCA CAT ATA TAC ACC CCA CAG AGA GAC ACT CTA TAT ATA TAA AAC ACA CAC ACA CAT ATA TAT ATA TAA AAA AAA AAA AAT ATA TAC ACG CGT GTG TGT GTG TGA GAT ATG TGG GGT GTA TAC ACA CAA AAC ACA CAT ATT TTT TTA TAC ACT CTT TTA TAT ATG TGC GCA CAT ATC TCA CAG AGC GCA CAT ATT TTG TGT GTG TGG GGG GGA GAA AAA AAT ATC TCC CCA CAA AAC ACA CAG AGG GGT GTT TTG TGT GTA TAG AGA GAT ATG TGC GCA CAG AGA GAT ATA TAG AGT GTA TAA AAA AAA AAT ATT TTG TGT GTT TTC TCA CAA AAC ACT CTT TTA TAG AGT GTG TGA GAA AAA AAT ATT TTA TAG AGT GTA TAT ATG TGG GGA GAC ACA CAA AAT ATT TTC TCA CAC ACC CCT CTA TAA AAT ATT TTT TTA TAG AGC GCA CAT ATG TGG GGC GCC CCT CTC TCT CTT TTA TAT ATT TTG TGT GTA TAA AAC ACA CAG AGC GCT CTT TTT TTA TAA AAG AGG GGG GGC GCC CCA CAA AAT ATT TTC TCT CTG TGC GCT CTG TGT GTC TCA CAA AAA AAT ATT TTA TAC ACA CAG AGA GAA AAT ATA TAA AAT ATG TGA GAG AGC GCT CTT TTA TAG AGT GTC TCC CCT CTG TGT GTT TTG TGC GCA CAC ACT CTA TAC ACG CGA GAC ACA CAG AGA GAT ATG TGT GTC TCT CTT TTG TGT GTG TGC GCT CTG TGC GCC CCG CGG GGT GTA TAC ACT CTA TAC ACA CAC ACA CAA AAA AAC ACT CTG TGC GCT CTT TTG TGC G...

In [ ]:
import org.apache.spark.ml.feature.RegexTokenizer

// Set params for RegexTokenizer
val tokenizer = new RegexTokenizer()
.setPattern("\\W") // break by white space character(s)  
.setInputCol("genome") // name of the input column
.setOutputCol("tokens") // name of the output column

// Tokenize document
val tokenized_df_train = tokenizer.transform(kmers_df)

  

>     import org.apache.spark.ml.feature.RegexTokenizer
>     tokenizer: org.apache.spark.ml.feature.RegexTokenizer = RegexTokenizer: uid=regexTok_c06b3a0249a7, minTokenLength=1, gaps=true, pattern=\W, toLowercase=true
>     tokenized_df_train: org.apache.spark.sql.DataFrame = [genome: string, label: string ... 2 more fields]

In [ ]:

import org.apache.spark.sql.functions._

val distinct_df = tokenized_df_train.withColumn("tokens", explode(col("tokens"))).agg(collect_set("tokens").alias("distinct_tokens"))

  

>     import org.apache.spark.sql.functions._
>     distinct_df: org.apache.spark.sql.DataFrame = [distinct_tokens: array<string>]

In [ ]:
display(distinct_df.select("distinct_tokens"))

  

#### Take the complete sequences and get the samples without headers

In [ ]:
val filtered_s = data.filter( _.split("\n").head.split('|').last == "complete")
val samples = filtered_s.map( _.split("\n").tail.mkString("")).cache()
samples.count

  

>     filtered_s: org.apache.spark.rdd.RDD[String] = MapPartitionsRDD[71810] at filter at command-2185505652307367:1
>     samples: org.apache.spark.rdd.RDD[String] = MapPartitionsRDD[71811] at map at command-2185505652307367:2
>     res2: Long = 30816

In [ ]:
samples.take(1)

  

>     res3: Array[String] = Array(...)

In [ ]:
val genome_length_per_s = samples.map(_.length())

  

>     genome_length_per_s: org.apache.spark.rdd.RDD[Int] = MapPartitionsRDD[2398] at map at command-2185505652307369:1

In [ ]:
genome_length_per_s.stats

  

>     res7: org.apache.spark.util.StatCounter = (count: 30816, mean: 29810.095437, stdev: 174.662405, max: 30119.000000, min: 2859.000000)

  

#### Extract (overlapping \[since yet do not know which parts corresponds to coding regions\]) 3-mers

In [ ]:
def subsequence_map( sequence:String, k:Int ): Map[String, Int] = {
  def helper(seq:String, acc:Map[String, Int]): Map[String, Int] = {
    if (seq.length < k ) {
      return acc
    }
    else {
      val sub = seq.substring(0,k)
      if (acc.contains(sub)) {
        val acc2 = acc + ( sub -> (acc(sub) + 1))
        helper(seq.tail, acc2)
      }
      else {
        val acc2 = acc + ( sub -> 1)
        helper(seq.tail, acc2)
      }
    }
  }
  return helper(sequence, Map())
}

  

>     subsequence_map: (sequence: String, k: Int)Map[String,Int]

  

A tail recursive overlapping subsequence function

In [ ]:
def subsequence_str( sequence:String, k:Int ): String = {
  def helper(seq:String, acc:String): String = {
    if (seq.length < k ) {
      return acc
    }
    else {
      val sub = seq.substring(0,k)
//       helper(seq.substring(k), acc + sub + " ") aaabbb
      helper(seq.tail, acc + sub + " ")
    }
  }
  return helper(sequence, "")
}

  

>     subsequence_str: (sequence: String, k: Int)String

In [ ]:
val k = 3
val k_mers = samples.map( seq => subsequence_str(seq, k) ).cache()

  

>     k: Int = 3
>     k_mers: org.apache.spark.rdd.RDD[String] = MapPartitionsRDD[71917] at map at command-4060521384082286:2

In [ ]:
k_mers.take(1)

  

>     res15: Array[String] = Array("ATT TTA TAA AAA AAG AGG GGT GTT TTT TTA TAT ATA TAC ACC CCT CTT TTC TCC CCC CCA CAG AGG GGT GTA TAA AAC ACA CAA AAA AAC ACC CCA CAA AAC ACC CCA CAA AAC ACT CTT TTT TTC TCG CGA GAT ATC TCT CTC TCT CTT TTG TGT GTA TAG AGA GAT ATC TCT CTG TGT GTT TTC TCT CTC TCT CTA TAA AAA AAC ACG CGA GAA AAC ACT CTT TTT TTA TAA AAA AAA AAT ATC TCT CTG TGT GTG TGT GTG TGG GGC GCT CTG TGT GTC TCA CAC ACT CTC TCG CGG GGC GCT CTG TGC GCA CAT ATG TGC GCT CTT TTA TAG AGT GTG TGC GCA CAC ACT CTC TCA CAC ACG CGC GCA CAG AGT GTA TAT ATA TAA AAT ATT TTA TAA AAT ATA TAA AAC ACT CTA TAA AAT ATT TTA TAC ACT CTG TGT GTC TCG CGT GTT TTG TGA GAC ACA CAG AGG GGA GAC ACA CAC ACG CGA GAG AGT GTA TAA AAC ACT CTC TCG CGT GTC TCT CTA TAT ATC TCT CTT TTC TCT CTG TGC GCA CAG AGG GGC GCT CTG TGC GCT CTT TTA TAC ACG CGG GGT GTT TTT TTC TCG CGT GTC TCC CCG CGT GTG TGT GTT TTG TGC GCA CAG AGC GCC CCG CGA GAT ATC TCA CAT ATC TCA CAG AGC GCA CAC ACA CAT ATC TCT CTA TAG AGG GGT GTT TTT TTC TCG CGT GTC TCC CCG CGG GGG GGT GTG TGT GTG TGA GAC ACC CCG CGA GAA AAA AAG AGG GGT GTA TAA AAG AGA GAT ATG TGG GGA GAG AGA GAG AGC GCC CCT CTT TTG TGT GTC TCC CCC CCT CTG TGG GGT GTT TTT TTC TCA CAA AAC ACG CGA GAG AGA GAA AAA AAA AAC ACA CAC ACA CAC ACG CGT GTC TCC CCA CAA AAC ACT CTC TCA CAG AGT GTT TTT TTG TGC GCC CCT CTG TGT GTT TTT TTT TTA TAC ACA CAG AGG GGT GTT TTC TCG CGC GCG CGA GAC ACG CGT GTG TGC GCT CTC TCG CGT GTA TAC ACG CGT GTG TGG GGC GCT CTT TTT TTG TGG GGA GAG AGA GAC ACT CTC TCC CCG CGT GTG TGG GGA GAG AGG GGA GAG AGG GGT GTC TCT CTT TTA TAT ATC TCA CAG AGA GAG AGG GGC GCA CAC ACG CGT GTC TCA CAA AAC ACA CAT ATC TCT CTT TTA TAA AAA AAG AGA GAT ATG TGG GGC GCA CAC ACT CTT TTG TGT GTG TGG GGC GCT CTT TTA TAG AGT GTA TAG AGA GAA AAG AGT GTT TTG TGA GAA AAA AAA AAA AAG AGG GGC GCG CGT GTT TTT TTT TTG TGC GCC CCT CTC TCA CAA AAC ACT CTT TTG TGA GAA AAC ACA CAG AGC GCC CCC CCT CTA TAT ATG TGT GTG TGT GTT TTC TCA CAT ATC TCA CAA AAA AAC ACG CGT GTT TTC TCG CGG GGA GAT ATG TGC GCT CTC TCG CGA GAA AAC ACT CTG TGC GCA CAC ACC CCT CTC TCA CAT ATG TGG GGT GTC TCA CAT ATG TGT GTT TTA TAT ATG TGG GGT GTT TTG TGA GAG AGC GCT CTG TGG GGT GTA TAG AGC GCA CAG AGA GAA AAC ACT CTC TCG CGA GAA AAG AGG GGC GCA CAT ATT TTC TCA CAG AGT GTA TAC ACG CGG GGT GTC TCG CGT GTA TAG AGT GTG TGG GGT GTG TGA GAG AGA GAC ACA CAC ACT CTT TTG TGG GGT GTG TGT GTC TCC CCT CTT TTG TGT GTC TCC CCC CCT CTC TCA CAT ATG TGT GTG TGG GGG GGC GCG CGA GAA AAA AAT ATA TAC ACC CCA CAG AGT GTG TGG GGC GCT CTT TTA TAC ACC CCG CGC GCA CAA AAG AGG GGT GTT TTC TCT CTT TTC TCT CTT TTC TCG CGT GTA TAA AAG AGA GAA AAC ACG CGG GGT GTA TAA AAT ATA TAA AAA AAG AGG GGA GAG AGC GCT CTG TGG GGT GTG TGG GGC GCC CCA CAT ATA TAG AGT GTT TTA TAC ACG CGG GGC GCG CGC GCC CCG CGA GAT ATC TCT CTA TAA AAA AAG AGT GTC TCA CAT ATT TTT TTG TGA GAC ACT CTT TTA TAG AGG GGC GCG CGA GAC ACG CGA GAG AGC GCT CTT TTG TGG GGC GCA CAC ACT CTG TGA GAT ATC TCC CCT CTT TTA TAT ATG TGA GAA AAG AGA GAT ATT TTT TTT TTC TCA CAA AAG AGA GAA AAA AAA AAC ACT CTG TGG GGA GAA AAC ACA CAC ACT CTA TAA AAA AAC ACA CAT ATA TAG AGC GCA CAG AGT GTG TGG GGT GTG TGT GTT TTA TAC ACC CCC CCG CGT GTG TGA GAA AAC ACT CTC TCA CAT ATG TGC GCG CGT GTG TGA GAG AGC GCT CTT TTA TAA AAC ACG CGG GGA GAG AGG GGG GGG GGC GCA CAT ATA TAC ACA CAC ACT CTC TCG CGC GCT CTA TAT ATG TGT GTC TCG CGA GAT ATA TAA AAC ACA CAA AAC ACT CTT TTC TCT CTG TGT GTG TGG GGC GCC CCC CCT CTG TGA GAT ATG TGG GGC GCT CTA TAC ACC CCC CCT CTC TCT CTT TTG TGA GAG AGT GTG TGC GCA CAT ATT TTA TAA AAA AAG AGA GAC ACC CCT CTT TTC TCT CTA TAG AGC GCA CAC ACG CGT GTG TGC GCT CTG TGG GGT GTA TAA AAA AAG AGC GCT CTT TTC TCA CAT ATG TGC GCA CAC ACT CTT TTT TTG TGT GTC TCC CCG CGA GAA AAC ACA CAA AAC ACT CTG TGG GGA GAC ACT CTT TTT TTA TAT ATT TTG TGA GAC ACA CAC ACT CTA TAA AAG AGA GAG AGG GGG GGG GGT GTG TGT GTA TAT ATA TAC ACT CTG TGC GCT CTG TGC GCC CCG CGT GTG TGA GAA AAC ACA CAT ATG TGA GAG AGC GCA CAT ATG TGA GAA AAA AAT ATT TTG TGC GCT CTT TTG TGG GGT GTA TAC ACA CAC ACG CGG GGA GAA AAC ACG CGT GTT TTC TCT CTG TGA GAA AAA AAA AAG AGA GAG AGC GCT CTA TAT ATG TGA GAA AAT ATT TTG TGC GCA CAG AGA GAC ACA CAC ACC CCT CTT TTT TTT TTG TGA GAA AAA AAT ATT TTA TAA AAA AAT ATT TTG TGG GGC GCA CAA AAA AAG AGA GAA AAA AAT ATT TTT TTG TGA GAC ACA CAC ACC CCT CTT TTC TCA CAA AAT ATG TGG GGG GGG GGA GAA AAT ATG TGT GTC TCC CCA CAA AAA AAT ATT TTT TTT TTG TGT GTA TAT ATT TTT TTC TCC CCC CCT CTT TTA TAA AAA AAT ATT TTC TCC CCA CAT ATA TAA AAT ATC TCA CAA AAG AGA GAC ACT CTA TAT ATT TTC TCA CAA AAC ACC CCA CAA AAG AGG GGG GGT GTT TTG TGA GAA AAA AAA AAG AGA GAA AAA AAA AAA AAG AGC GCT CTT TTG TGA GAT ATG TGG GGC GCT CTT TTT TTA TAT ATG TGG GGG GGT GTA TAG AGA GAA AAT ATT TTC TCG CGA GAT ATC TCT CTG TGT GTC TCT CTA TAT ATC TCC CCA CAG AGT GTT TTG TGC GCG CGT GTC TCA CAC ACC CCA CAA AAA AAT ATG TGA GAA AAT ATG TGC GCA CAA AAC ACC CCA CAA AAA AAT ATG TGT GTG TGC GCC CCT CTT TTT TTC TCA CAA AAC ACT CTC TCT CTC TCA CAT ATG TGA GAA AAG AGT GTG TGT GTG TGA GAT ATC TCA CAT ATT TTG TGT GTG TGG GGT GTG TGA GAA AAA AAC ACT CTT TTC TCA CAT ATG TGG GGC GCA CAG AGA GAC ACG CGG GGG GGC GCG CGA GAT ATT TTT TTT TTG TGT GTT TTA TAA AAA AAG AGC GCC CCA CAC ACT CTT TTG TGC GCG CGA GAA AAT ATT TTT TTT TTG TGT GTG TGG GGC GCA CAC ACT CTG TGA GAG AGA GAA AAT ATT TTT TTG TGA GAC ACT CTA TAA AAA AAG AGA GAA AAG AGG GGT GTG TGC GCC CCA CAC ACT CTA TAC ACT CTT TTG TGT GTG TGG GGT GTT TTA TAC ACT CTT TTA TAC ACC CCC CCC CCA CAA AAA AAA AAT ATG TGC GCT CTG TGT GTT TTG TGT GTT TTA TAA AAA AAA AAT ATT TTT TTA TAT ATT TTG TGT GTC TCC CCA CAG AGC GCA CAT ATG TGT GTC TCA CAC ACA CAA AAT ATT TTC TCA CAG AGA GAA AAG AGT GTA TAG AGG GGA GAC ACC CCT CTG TGA GAG AGC GCA CAT ATA TAG AGT GTC TCT CTT TTG TGC GCC CCG CGA GAA AAT ATA TAC ACC CCA CAT ATA TAA AAT ATG TGA GAA AAT ATC TCT CTG TGG GGC GCT CTT TTG TGA GAA AAA AAA AAC ACC CCA CAT ATT TTC TCT CTT TTC TCG CGT GTA TAA AAG AGG GGG GGT GTG TGG GGT GTC TCG CGC GCA CAC ACT CTA TAT ATT TTG TGC GCC CCT CTT TTT TTG TGG GGA GAG AGG GGC GCT CTG TGT GTG TGT GTG TGT GTT TTC TCT CTC TCT CTT TTA TAT ATG TGT GTT TTG TGG GGT GTT TTG TGC GCC CCA CAT ATA TAA AAC ACA CAA AAG AGT GTG TGT GTG TGC GCC CCT CTA TAT ATT TTG TGG GGG GGT GTT TTC TCC CCA CAC ACG CGT GTG TGC GCT CTA TAG AGC GCG CGC GCT CTA TAA AAC ACA CAT ATA TAG AGG GGT GTT TTG TGT GTA TAA AAC ACC CCA CAT ATA TAC ACA CAG AGG GGT GTG TGT GTT TTG TGT GTT TTG TGG GGA GAG AGA GAA AAG AGG GGT GTT TTC TCC CCG CGA GAA AAG AGG GGT GTC TCT CTT TTA TAA AAT ATG TGA GAC ACA CAA AAC ACC CCT CTT TTC TCT CTT TTG TGA GAA AAA AAT ATA TAC ACT CTC TCC CCA CAA AAA AAA AAA AAG AGA GAG AGA GAA AAA AAG AGT GTC TCA CAA AAC ACA CAT ATC TCA CAA AAT ATA TAT ATT TTG TGT GTT TTG TGG GGT GTG TGA GAC ACT CTT TTT TTA TAA AAA AAC ACT CTT TTA TAA AAT ATG TGA GAA AAG AGA GAG AGA GAT ATC TCG CGC GCC CCA CAT ATT TTA TAT ATT TTT TTT TTG TGG GGC GCA CAT ATC TCT CTT TTT TTT TTT TTC TCT CTG TGC GCT CTT TTC TCC CCA CAC ACA CAA AAG AGT GTG TGC GCT CTT TTT TTT TTG TGT GTG TGG GGA GAA AAA AAC ACT CTG TGT GTG TGA GAA AAA AAG AGG GGT GTT TTT TTG TGG GGA GAT ATT TTA TAT ATA TAA AAA AAG AGC GCA CAT ATT TTC TCA CAA AAA AAC ACA CAA AAA AAT ATT TTG TGT GTT TTG TGA GAA AAT ATC TCC CCT CTG TGT GTG TGG GGT GTA TAA AAT ATT TTT TTT TTA TAA AAA AAG AGT GTT TTA TAC ACA CAA AAA AAA AAG AGG GGA GAA AAA AAA AAG AGC GCT CTA TAA AAA AAA AAA AAA AAG AGG GGT GTG TGC GCC CCT CTG TGG GGA GAA AAT ATA TAT ATT TTG TGG GGT GTG TGA GAA AAC ACA CAG AGA GAA AAA AAT ATC TCA CAA AAT ATA TAC ACT CTG TGA GAG AGT GTC TCC CCT CTC TCT CTT TTT TTA TAT ATG TGC GCA CAT ATT TTT TTG TGC GCA CAT ATC TCA CAG AGA GAG AGG GGC GCT CTG TGC GCT CTC TCG CGT GTG TGT GTT TTG TGT GTA TAC ACG CGA GAT ATC TCA CAA AAT ATT TTT TTT TTC TCT CTC TCC CCC CCG CGC GCA CAC ACT CTC TCT CTT TTG TGA GAA AAA AAC ACT CTG TGC GCT CTC TCA CAA AAA AAA AAT ATT TTC TCT CTG TGT GTG TGC GCG CGT GTG TGT GTT TTT TTT TTA TAC ACA CAG AGA GAA AAG AGG GGC GCC CCG CGC GCT CTA TAT ATA TAA AAC ACA CAA AAT ATA TAC ACT CTA TAG AGA GAT ATG TGG GGA GAA AAT ATT TTT TTC TCA CAC ACA CAG AGT GTA TAT ATT TTC TCA CAC ACT CTG TGA GAG AGA GAC ACT CTC TCA CAT ATT TTG TGA GAT ATG TGC GCT CTA TAT ATG TGA GAT ATG TGT GTT TTC TCA CAC ACA CAT ATC TCT CTG TGA GAT ATT TTT TTG TGG GGC GCT CTA TAC ACT CTA TAA AAC ACA CAA AAT ATC TCT CTA TAG AGT GTT TTG TGT GTA TAA AAT ATG TGG GGC GCC CCT CTA TAC ACA CAT ATT TTA TAC ACA CAG AGG GGT GTG TGG GGT GTG TGT GTT TTG TGT GTT TTC TCA CAG AGT GTT TTG TGA GAC ACT CTT TTC TCG CGC GCA CAG AGT GTG TGG GGC GCT CTA TAA AAC ACT CTA TAA AAC ACA CAT ATC TCT CTT TTT TTG TGG GGC GCA CAC ACT CTG TGT GTT TTT TTA TAT ATG TGA GAA AAA AAA AAA AAC ACT CTC TCA CAA AAA AAC ACC CCC CCG CGT GTC TCC CCT CTT TTG TGA GAT ATT TTG TGG GGC GCT CTT TTG TGA GAA AAG AGA GAG AGA GAA AAG AGT GTT TTT TTA TAA AAG AGG GGA GAA AAG AGG GGT GTG TGT GTA TAG AGA GAG AGT GTT TTT TTC TCT CTT TTA TAG AGA GAG AGA GAC ACG CGG GGT GTT TTG TGG GGG GGA GAA AAA AAT ATT TTG TGT GTT TTA TAA AAA AAT ATT TTT TTA TAT ATC TCT CTC TCA CAA AAC ACC CCT CTG TGT GTG TGC GCT CTT TTG TGT GTG TGA GAA AAA AAT ATT TTG TGT GTC TCG CGG GGT GTG TGG GGA GAC ACA CAA AAA AAT ATT TTG TGT GTC TCA CAC ACC CCT CTG TGT GTG TGC GCA CAA AAA AAG AGG GGA GAA AAA AAT ATT TTA TAA AAG AGG GGA GAG AGA GAG AGT GTG TGT GTT TTC TCA CAG AGA GAC ACA CAT ATT TTC TCT CTT TTT TTA TAA AAG AGC GCT CTT TTG TGT GTA TAA AAA AAT ATA TAA AAA AAT ATT TTT TTT TTT TTG TGG GGC GCT CTT TTT TTG TGT GTG TGT GTG TGC GCT CTG TGA GAC ACT CTC TCT CTA TAT ATC TCA CAT ATT TTA TAT ATT TTG TGG GGT GTG TGG GGA GAG AGC GCT CTA TAA AAA AAC ACT CTT TTA TAA AAA AAG AGC GCC CCT CTT TTG TGA GAA AAT ATT TTT TTA TAG AGG GGT GTG TGA GAA AAA AAC ACA CAT ATT TTT TTG TGT GTC TCA CAC ACG CGC GCA CAC ACT CTC TCA CAA AAA AAG AGG GGG GGA GAT ATT TTG TGT GTA TAC ACA CAG AGA GAA AAA AAG AGT GTG TGT GTG TGT GTT TTA TAA AAA AAT ATC TCC CCA CAG AGA GAG AGA GAA AAG AGA GAA AAA AAC ACT CTG TGG GGC GCC CCT CTA TAC ACT CTC TCA CAT ATG TGC GCC CCT CTC TCT CTA TAA AAA AAA AAG AGC GCC CCC CCC CCA CAA AAA AAA AAG AGA GAA AAA AAT ATT TTA TAT ATC TCT CTT TTC TCT CTT TTA TAG AGA GAG AGG GGG GGA GAG AGA GAA AAA AAC ACA CAC ACT CTT TTC TCC CCC CCA CAC ACA CAG AGA GAA AAG AGT GTG TGT GTT TTA TAA AAC ACA CAG AGA GAG AGG GGA GAA AAG AGT GTT TTG TGT GTC TCT CTT TTG TGA GAA AAA AAA AAC ACT CTG TGG GGT GTG TGA GAT ATT TTT TTA TAC ACA CAA AAC ACC CCA CAT ATT TTA TAG AGA GAA AAC ACA CAA AAC ACC CCT CTA TAC ACT CTA TAG AGT GTG TGA GAA AAG AGC GCT CTG TGT GTT TTG TGA GAA AAG AGC GCT CTC TCC CCA CAT ATT TTG TGG GGT GTT TTG TGG GGT GTA TAC ACA CAC ACC CCA CAG AGT GTT TTT TTG TGT GTA TAT ATT TTA TAA AAC ACG CGG GGG GGC GCT CTT TTA TAT ATG TGT GTT TTG TGC GCT CTC TCG CGA GAA AAA AAT ATC TCA CAA AAA AAG AGA GAC ACA CAC ACA CAG AGA GAA AAA AAA AAG AGT GTA TAC ACT CTG TGT GTG TGC GCC CCC CCT CTT TTG TGC GCA CAC ACC CCT CTA TAA AAT ATA TAT ATG TGA GAT ATG TGG GGT GTA TAA AAC ACA CAA AAA AAC ACA CAA AAT ATA TAC ACC CCT CTT TTC TCA CAC ACA CAC ACT CTC TCA CAA AAA AAG AGG GGC GCG CGG GGT GTG TGC GCA CAC ACC CCA CAA AAC ACA CAA AAA AAG AGG GGT GTT TTA TAC ACT CTT TTT TTT TTG TGG GGT GTG TGA GAT ATG TGA GAC ACA CAC ACT CTG TGT GTG TGA GAT ATA TAG AGA GAA AAG AGT GTG TGC GCA CAA AAG AGG GGT GTT TTA TAC ACA CAA AAG AGA GAG AGT GTG TGT GTG TGA GAA AAT ATA TAT ATC TCA CAC ACT CTT TTT TTT TTG TGA GAA AAC ACT CTT TTG TGA GAT ATG TGA GAA AAA AAG AGG GGA GAT ATT TTG TGA GAT ATA TAA AAA AAG AGT GTA TAC ACT CTT TTA TAA AAT ATG TGA GAG AGA GAA AAG AGT GTG TGC GCT CTC TCT CTG TGC GCC CCT CTA TAT ATA TAC ACA CAG AGT GTT TTG TGA GAA AAC ACT CTC TCG CGG GGT GTA TAC ACA CAG AGA GAA AAG AGT GTA TAA AAA AAT ATG TGA GAG AGT GTT TTC TCG CGC GCC CCT CTG TGT GTG TGT GTT TTG TGT GTG TGG GGC GCA CAG AGA GAT ATG TGC GCT CTG TGT GTC TCA CAT ATA TAA AAA AAA AAA AAC ACT CTT TTT TTG TGC GCA CAA AAC ACC CCA CAG AGT GTA TAT ATC TCT CTG TGA GAA AAT ATT TTA TAC ACT CTT TTA TAC ACA CAC ACC CCA CAC ACT CTG TGG GGG GGC GCA CAT ATT TTG TGA GAT ATT TTT TTA TAG AGA GAT ATG TGA GAG AGT GTG TGG GGA GAG AGT GTA TAT ATG TGG GGC GCT CTA TAC ACA CAT ATA TAC ACT CTA TAC ACT CTT TTA TAT ATT TTT TTG TGA GAT ATG TGA GAG AGT GTC TCT CTG TGG GGT GTG TGA GAG AGT GTT TTT TTA TAA AAA AAT ATT TTG TGG GGC GCT CTT TTC TCA CAC ACA CAT ATA TAT ATG TGT GTA TAT ATT TTG TGT GTT TTC TCT CTT TTT TTC TCT CTA TAC ACC CCC CCT CTC TCC CCA CAG AGA GAT ATG TGA GAG AGG GGA GAT ATG TGA GAA AAG AGA GAA AAG AGA GAA AAG AGG GGT GTG TGA GAT ATT TTG TGT GTG TGA GAA AAG AGA GAA AAG AGA GAA AAG AGA GAG AGT GTT TTT TTG TGA GAG AGC GCC CCA CAT ATC TCA CAA AAC ACT CTC TCA CAA AAT ATA TAT ATG TGA GAG AGT GTA TAT ATG TGG GGT GTA TAC ACT CTG TGA GAA AAG AGA GAT ATG TGA GAT ATT TTA TAC ACC CCA CAA AAG AGG GGT GTA TAA AAA AAC ACC CCT CTT TTT TTG TGG GGA GAA AAT ATT TTT TTG TGG GGT GTG TGC GCC CCA CAC ACT CTT TTC TCT CTG TGC GCT CTG TGC GCT CTC TCT CTT TTC TCA CAA AAC ACC CCT CTG TGA GAA AAG AGA GAA AAG AGA GAG AGC GCA CAA AAG AGA GAA AAG AGA GAA AAG AGA GAT ATT TTG TGG GGT GTT TTA TAG AGA GAT ATG TGA GAT ATG TGA GAT ATA TAG AGT GTC TCA CAA AAC ACA CAA AAA AAC ACT CTG TGT GTT TTG TGG GGT GTC TCA CAA AAC ACA CAA AAG AGA GAC ACG CGG GGC GCA CAG AGT GTG TGA GAG AGG GGA GAC ACA CAA AAT ATC TCA CAG AGA GAC ACA CAA AAC ACT CTA TAC ACT CTA TAT ATT TTC TCA CAA AAA AAC ACA CAA AAT ATT TTG TGT GTT TTG TGA GAG AGG GGT GTT TTC TCA CAA AAC ACC CCT CTC TCA CAA AAT ATT TTA TAG AGA GAG AGA GAT ATG TGG GGA GAA AAC ACT CTT TTA TAC ACA CAC ACC CCA CAG AGT GTT TTG TGT GTT TTC TCA CAG AGA GAC ACT CTA TAT ATT TTG TGA GAA AAG AGT GTG TGA GAA AAT ATA TAG AGT GTT TTT TTT TTA TAG AGT GTG TGG GGT GTT TTA TAT ATT TTT TTA TAA AAA AAA AAC ACT CTT TTA TAC ACT CTG TGA GAC ACA CAA AAT ATG TGT GTA TAT ATA TAC ACA CAT ATT TTA TAA AAA AAA AAA AAT ATG TGC GCA CAG AGA GAC ACA CAT ATT TTG TGT GTG TGG GGA GAA AAG AGA GAA AAG AGC GCT CTA TAA AAA AAA AAA AAG AGG GGT GTA TAA AAA AAA AAC ACC CCA CAA AAC ACA CAG AGT GTG TGG GGT GTT TTG TGT GTT TTA TAA AAT ATG TGC GCA CAG AGC GCC CCA CAA AAT ATG TGT GTT TTT TTA TAC ACC CCT CTT TTA TAA AAA AAC ACA CAT ATG TGG GGA GAG AGG GGA GAG AGG GGT GTG TGT GTT TTG TGC GCA CAG AGG GGA GAG AGC GCC CCT CTT TTA TAA AAA AAT ATA TAA AAG AGG GGC GCT CTA TAC ACT CTA TAA AAC ACA CAA AAT ATG TGC GCC CCA CAT ATG TGC GCA CAA AAG AGT GTT TTG TGA GAA AAT ATC TCT CTG TGA GAT ATG TGA GAT ATT TTA TAC ACA CAT ATA TAG AGC GCT CTA TAC ACT CTA TAA AAT ATG TGG GGA GAC ACC CCA CAC ACT CTT TTA TAA AAA AAG AGT GTG TGG GGG GGT GTG TGG GGT GTA TAG AGT GTT TTG TGT GTG TGT GTT TTT TTT TTA TAA AAG AGC GCG CGG GGA GAC ACA CAC ACA CAA AAT ATC TCT CTT TTG TGC GCT CTA TAA AAA AAC ACA CAC ACT CTG TGT GTC TCT CTT TTC TCA CAT ATG TGT GTT TTG TGT GTC TCG CGG GGC GCC CCC CCA CAA AAA AAT ATG TGT GTT TTA TAA AAC ACA CAA AAA AAG AGG GGT GTG TGA GAA AAG AGA GAC ACA CAT ATT TTC TCA CAA AAC ACT CTT TTC TCT CTT TTA TAA AAG AGA GAG AGT GTG TGC GCT CTT TTA TAT ATG TGA GAA AAA AAA AAT ATT TTT TTT TTA TAA AAT ATC TCA CAG AGC GCA CAC ACG CGA GAA AAG AGT GTT TTC TCT CTA TAC ACT CTT TTG TGC GCA CAC ACC CCA CAT ATT TTA TAT ATT TTA TAT ATC TCA CAG AGC GCT CTG TGG GGT GTA TAT ATT TTT TTT TTT TTG TGG GGT GTG TGC GCT CTG TGA GAC ACC CCC CCT CTA TAT ATA TAC ACA CAT ATT TTC TCT CTT TTT TTA TAA AAG AGA GAG AGT GTT TTT TTG TGT GTG TGT GTA TAG AGA GAT ATA TAC ACT CTG TGT GTT TTC TCG CGC GCA CAC ACA CAA AAA AAT ATG TGT GTC TCT CTA TAC ACT CTT TTA TAG AGC GCT CTG TGT GTC TCT CTT TTT TTG TGA GAT ATA TAA AAA AAA AAA AAT ATC TCT CTC TCT CTA TAT ATG TGA GAC ACA CAA AAA AAC ACT CTT TTG TGT GTT TTT TTC TCA CAA AAG AGC GCT CTT TTT TTT TTT TTG TGG GGA GAA AAA AAT ATG TGA GAA AAG AGA GAG AGT GTG TGA GAA AAA AAA AAG AGC GCA CAA AAG AGT GTT TTG TGA GAA AAC ACA CAA AAA AAA AAG AGA GAT ATC TCG CGC GCT CTG TGA GAG AGA GAT ATT TTC TCC CCT CTA TAA AAA AAG AGA GAG AGG GGA GAA AAG AGT GTT TTA TAA AAG AGC GCC CCA CAT ATT TTT TTA TAT ATA TAA AAC ACT CTG TGA GAA AAA AAG AGT GTA TAA AAA AAC ACC CCT CTT TTC TCA CAG AGT GTT TTG TGA GAA AAC ACA CAG AGA GAG AGA GAA AAA AAA AAC ACA CAA AAG AGA GAT ATG TGA GAT ATA TAA AAG AGA GAA AAA AAA AAT ATC TCA CAA AAA AAG AGC GCT CTT TTG TGT GTG TGT GTT TTG TGA GAA AAG AGA GAA AAG AGT GTT TTA TAC ACA CAA AAC ACA CAA AAC ACT CTC TCT CTG TGG GGA GAA AAG AGA GAA AAA AAC ACT CTA TAA AAG AGT GTT TTC TCC CCT CTC TCA CAC ACA CAG AGA GAA AAA AAA AAC ACT CTT TTG TGT GTT TTA TAC ACT CTT TTT TTA TAT ATA TAT ATT TTG TGA GAC ACA CAT ATT TTA TAA AAT ATG TGG GGC GCA CAA AAT ATC TCT CTT TTC TCA CAT ATC TCC CCA CAG AGA GAT ATT TTC TCT CTG TGC GCC CCA CAC ACT CTC TCT CTT TTG TGT GTT TTA TAG AGT GTG TGA GAC ACA CAT ATT TTG TGA GAC ACA CAT ATC TCA CAC ACT CTT TTT TTC TCT CTT TTA TAA AAA AAG AGA GAA AAA AAG AGA GAT ATG TGC GCT CTC TCC CCA CAT ATA TAT ATA TAT ATA TAG AGT GTG TGG GGG GGT GTG TGA GAT ATG TGT GTT TTG TGT GTT TTC TCA CAA AAG AGA GAG AGG GGG GGT GTG TGT GTT TTT TTT TTA TAA AAC ACT CTG TGC GCT CTG TGT GTG TGG GGT GTT TTA TAT ATA TAC ACC CCT CTA TAC ACT CTA TAA AAA AAA AAA AAG AGG GGC GCT CTG TGG GGT GTG TGG GGC GCA CAC ACT CTA TAC ACT CTG TGA GAA AAA AAT ATG TGC GCT CTA TAG AGC GCG CGA GAA AAA AAG AGC GCT CTT TTT TTG TGA GAG AGA GAA AAA AAA AAG AGT GTG TGC GCC CCA CAA AAC ACA CAG AGA GAC ACA CAA AAT ATT TTA TAT ATA TAT ATA TAA AAC ACC CCA CAC ACT CTT TTA TAC ACC CCC CCG CGG GGG GGT GTC TCA CAG AGG GGG GGT GTT TTT TTA TAA AAA AAT ATG TGG GGT GTT TTA TAC ACA CAC ACT CTG TGT GTA TAG AGA GAG AGG GGA GAG AGG GGC GCA CAA AAA AAG AGA GAC ACA CAG AGT GTG TGC GCT CTT TTA TAA AAA AAA AAA AAG AGT GTG TGT GTA TAA AAA AAA AAG AGT GTG TGC GCC CCT CTT TTT TTT TTA TAC ACA CAT ATT TTC TCT CTA TAC ACC CCA CAT ATC TCT CTA TAT ATT TTA TAT ATC TCT CTC TCT CTA TAA AAT ATG TGA GAG AGA GAA AAG AGC GCA CAA AAG AGA GAA AAA AAT ATT TTC TCT CTT TTG TGG GGA GAA AAC ACT CTG TGT GTT TTT TTC TCT CTT TTG TGG GGA GAA AAT ATT TTT TTG TGC GCG CGA GAG AGA GAA AAA AAT ATG TGC GCT CTT TTG TGC GCA CAC ACA CAT ATG TGC GCA CAG AGA GAA AAG AGA GAA AAA AAC ACA CAC ACG CGC GCA CAA AAA AAT ATT TTA TAA AAT ATG TGC GCC CCT CTG TGT GTC TCT CTG TGT GTG TGT GTG TGG GGA GAA AAA AAC ACT CTA TAA AAA AAG AGC GCC CCA CAT ATA TAG AGT GTT TTT TTC TCA CAA AAC ACT CTA TAT ATA TAC ACA CAG AGC GCG CGT GTA TAA AAA AAT ATA TAT ATA TAA AAG AGG GGG GGT GTA TAT ATT TTA TAA AAA AAA AAT ATA TAC ACA CAA AAG AGA GAG AGG GGG GGT GTG TGT GTG TGG GGT GTT TTG TGA GAT ATT TTA TAT ATG TGG GGT GTG TGC GCT CTA TAG AGA GAT ATT TTT TTT TTA TAC ACT CTT TTT TTT TTA TAC ACA CAC ACC CCA CAG AGT GTA TAA AAA AAA AAC ACA CAA AAC ACT CTG TGT GTA TAG AGC GCG CGT GTC TCA CAC ACT CTT TTA TAT ATC TCA CAA AAC ACA CAC ACA CAC ACT CTT TTA TAA AAC ACG CGA GAT ATC TCT CTA TAA AAA AAT ATG TGA GAA AAA AAC ACT CTC TCT CTT TTG TGT GTT TTA TAC ACA CAA AAT ATG TGC GCC CCA CAC ACT CTT TTG TGG GGC GCT CTA TAT ATG TGT GTA TAA AAC ACA CAC ACA CAT ATG TGG GGC GCT CTT TTA TAA AAA AAT ATT TTT TTG TGG GGA GAA AAG AGA GAA AAG AGC GCT CTG TGC GCT CTC TCG CGG GGT GTA TAT ATA TAT ATG TGA GAG AGA GAT ATC TCT CTC TCT CTC TCA CAA AAA AAG AGT GTG TGC GCC CCA CAG AGC GCT CTA TAC ACA CAG AGT GTT TTT TTC TCT CTG TGT GTT TTT TTC TCT CTT TTC TCA CAC ACC CCT CTG TGA GAT ATG TGC GCT CTG TGT GTT TTA TAC ACA CAG AGC GCG CGT GTA TAT ATA TAA AAT ATG TGG GGT GTT TTA TAT ATC TCT CTT TTA TAC ACT CTT TTC TCT CTT TTC TCT CTT TTC TCT CTA TAA AAA AAA AAC ACA CAC ACC CCT CTG TGA GAA AAG AGA GAA AAC ACA CAT ATT TTT TTT TTA TAT ATT TTG TGA GAA AAA AAC ACC CCA CAT ATC TCT CTC TCA CAC ACT CTT TTG TGC GCT CTG TGG GGT GTT TTC TCC CCT CTA TAT ATA TAA AAA AAG AGA GAT ATT TTG TGG GGT GTC TCC CCT CTA TAT ATT TTC TCT CTG TGG GGA GAC ACA CAA AAT ATC TCT CTA TAC ACA CAC ACA CAA AAC ACT CTA TAG AGG GGT GTA TAT ATA TAG AGA GAA AAT ATT TTT TTC TCT CTT TTA TAA AAG AGA GAG AGA GAG AGG GGT GTG TGA GAT ATA TAA AAA AAA AAG AGT GTG TGT GTA TAT ATA TAT ATT TTA TAC ACA CAC ACT CTA TAG AGT GTA TAA AAT ATC TCC CCT CTA TAC ACC CCA CAC ACA CAT ATT TTC TCC CCA CAC ACC CCT CTA TAG AGA GAT ATG TGG GGT GTG TGA GAA AAG AGT GTT TTA TAT ATC TCA CAC ACC CCT CTT TTT TTG TGA GAC ACA CAA AAT ATC TCT CTT TTA TAA AAG AGA GAC ACA CAC ACT CTT TTC TCT CTT TTT TTC TCT CTT TTT TTG TGA GAG AGA GAG AGA GAA AAG AGT GTG TGA GAG AGG GGA GAC ACT CTA TAT ATT TTA TAA AAG AGG GGT GTG TGT GTT TTT TTA TAC ACA CAA AAC ACA CAG AGT GTA TAG AGA GAC ACA CAA AAC ACA CAT ATT TTA TAA AAC ACC CCT CTC TCC CCA CAC ACA CAC ACG CGC GCA CAA AAG AGT GTT TTG TGT GTG TGG GGA GAC ACA CAT ATG TGT GTC TCA CAA AAT ATG TGA GAC ACA CAT ATA TAT ATG TGG GGA GAC ACA CAA AAC ACA CAG AGT GTT TTT TTG TGG GGT GTC TCC CCA CAA AAC ACT CTT TTA TAT ATT TTT TTG TGG GGA GAT ATG TGG GGA GAG AGC GCT CTG TGA GAT ATG TGT GTT TTA TAC ACT CTA TAA AAA AAA AAT ATA TAA AAA AAA AAC ACC CCT CTC TCA CAT ATA TAA AAT ATT TTC TCA CAC ACA CAT ATG TGA GAA AAG AGG GGT GTA TAA AAA AAA AAC ACA CAT ATT TTT TTT TTA TAT ATG TGT GTT TTT TTT TTA TAC ACC CCT CTA TAA AAT ATG TGA GAT ATG TGA GAC ACA CAC ACT CTC TCT CTA TAC ACG CGT GTG TGT GTT TTG TGA GAG AGG GGC GCT CTT TTT TTT TTG TGA GAG AGT GTA TAC ACT CTA TAC ACC CCA CAC ACA CAC ACA CAA AAC ACT CTG TGA GAT ATC TCC CCT CTA TAG AGT GTT TTT TTT TTC TCT CTG TGG GGG GGT GTA TAG AGG GGT GTA TAC ACA CAT ATG TGT GTC TCA CAG AGC GCA CAT ATT TTA TAA AAA AAT ATC TCA CAC ACA CAC ACT CTA TAA AAA AAA AAA AAG AGT GTG TGG GGA GAA AAA AAT ATA TAC ACC CCC CCA CAC ACA CAA AAG AGT GTT TTA TAA AAT ATG TGG GGT GTT TTT TTA TAA AAC ACT CTT TTC TCT CTA TAT ATT TTA TAA AAA AAT ATG TGG GGG GGC GCA CAG AGA GAT ATA TAA AAC ACA CAA AAC ACT CTG TGT GTT TTA TAT ATC TCT CTT TTG TGC GCC CCA CAC ACT CTG TGC GCA CAT ATT TTG TGT GTT TTA TAA AAC ACA CAC ACT CTC TCC CCA CAA AAC ACA CAA AAA AAT ATA TAG AGA GAG AGT GTT TTG TGA GAA AAG AGT GTT TTT TTA TAA AAT ATC TCC CCA CAC ACC CCT CTG TGC GCT CTC TCT CTA TAC ACA CAA AAG AGA GAT ATG TGC GCT CTT TTA TAT ATT TTA TAC ACA CAG AGA GAG AGC GCA CAA AAG AGG GGG GGC GCT CTG TGG GGT GTG TGA GAA AAG AGC GCT CTG TGC GCT CTA TAA AAC ACT CTT TTT TTT TTG TGT GTG TGC GCA CAC ACT CTT TTA TAT ATC TCT CTT TTA TAG AGC GCC CCT CTA TAC ACT CTG TGT GTA TAA AAT ATA TAA AAG AGA GAC ACA CAG AGT GTA TAG AGG GGT GTG TGA GAG AGT GTT TTA TAG AGG GGT GTG TGA GAT ATG TGT GTT TTA TAG AGA GAG AGA GAA AAA AAC ACA CAA AAT ATG TGA GAG AGT GTT TTA TAC ACT CTT TTG TGT GTT TTT TTC TCA CAA AAC ACA CAT ATG TGC GCC CCA CAA AAT ATT TTT TTA TAG AGA GAT ATT TTC TCT CTT TTG TGC GCA CAA AAA AAA AAG AGA GAG AGT GTC TCT CTT TTG TGA GAA AAC ACG CGT GTG TGG GGT GTG TGT GTG TGT GTA TAA AAA AAA AAC ACT CTT TTG TGT GTG TGG GGA GAC ACA CAA AAC ACA CAG AGC GCA CAG AGA GAC ACA CAA AAC ACC CCC CCT CTT TTA TAA AAG AGG GGG GGT GTG TGT GTA TAG AGA GAA AAG AGC GCT CTG TGT GTT TTA TAT ATG TGT GTA TAC ACA CAT ATG TGG GGG GGC GCA CAC ACA CAC ACT CTT TTT TTC TCT CTT TTA TAT ATG TGA GAA AAC ACA CAA AAT ATT TTT TTA TAA AAG AGA GAA AAA AAG AGG GGT GTG TGT GTT TTC TCA CAG AGA GAT ATA TAC ACC CCT CTT TTG TGT GTA TAC ACG CGT GTG TGT GTG TGG GGT GTA TAA AAA AAC ACA CAA AAG AGC GCT CTA TAC ACA CAA AAA AAA AAT ATA TAT ATC TCT CTA TAG AGT GTA TAC ACA CAA AAC ACA CAG AGG GGA GAG AGT GTC TCA CAC ACC CCT CTT TTT TTT TTG TGT GTT TTA TAT ATG TGA GAT ATG TGT GTC TCA CAG AGC GCA CAC ACC CCA CAC ACC CCT CTG TGC GCT CTC TCA CAG AGT GTA TAT ATG TGA GAA AAC ACT CTT TTA TAA AAG AGC GCA CAT ATG TGG GGT GTA TAC ACA CAT ATT TTT TTA TAC ACT CTT TTG TGT GTG TGC GCT CTA TAG AGT GTG TGA GAG AGT GTA TAC ACA CAC ACT CTG TGG GGT GTA TAA AAT ATT TTA TAC ACC CCA CAG AGT GTG TGT GTG TGG GGT GTC TCA CAC ACT CTA TAT ATA TAA AAA AAC ACA CAT ATA TAT ATA TAA AAC ACT CTT TTC TCT CTA TAA AAA AAG AGA GAA AAA AAC ACT CTT TTT TTG TGT GTA TAT ATT TTG TGC GCA CAT ATA TAG AGA GAC ACG CGG GGT GTG TGC GCT CTT TTT TTA TAC ACT CTT TTA TAC ACA CAA AAA AAG AGT GTC TCC CCT CTC TCA CAG AGA GAA AAT ATA TAC ACA CAA AAA AAG AGG GGT GTC TCC CCT CTA TAT ATT TTA TAC ACG CGG GGA GAT ATG TGT GTT TTT TTT TTC TCT CTA TAC ACA CAA AAA AAG AGA GAA AAA AAA AAC ACA CAG AGT GTT TTA TAC ACA CAC ACA CAA AAC ACA CAA AAC ACC CCA CAT ATA TAA AAA AAA AAC ACC CCA CAG AGT GTT TTA TAC ACT CTT TTA TAT ATA TAA AAA AAT ATT TTG TGG GGA GAT ATG TGG GGT GTG TGT GTT TTG TGT GTT TTT TTG TGT GTA TAC ACA CAG AGA GAA AAA AAT ATT TTG TGA GAC ACC CCC CCT CTA TAA AAG AGT GTT TTG TGG GGA GAC ACA CAA AAT ATT TTA TAT ATT TTA TAT ATA TAA AAG AGA GAA AAA AAG AGA GAC ACA CAA AAT ATT TTC TCT CTT TTA TAT ATT TTT TTC TCA CAC ACA CAG AGA GAG AGC GCA CAA AAC ACC CCA CAA AAT ATT TTG TGA GAT ATC TCT CTT TTG TGT GTA TAC ACC CCA CAA AAA AAC ACC CCA CAA AAC ACC CCA CAT ATA TAT ATC TCC CCA CAA AAA AAC ACG CGC GCA CAA AAG AGC GCT CTT TTC TCG CGA GAT ATA TAA AAT ATT TTT TTT TTA TAA AAG AGT GTT TTT TTG TGT GTA TAT ATG TGT GTG TGA GAT ATA TAA AAT ATA TAT ATC TCA CAA AAA AAT ATT TTT TTG TGC GCT CTG TGA GAT ATG TGA GAT ATT TTT TTA TAA AAA AAC ACC CCA CAG AGT GTT TTA TAA AAC ACT CTG TGG GGT GTT TTA TAT ATA TAA AAG AGA GAA AAA AAC ACC CCT CTG TGC GCT CTT TTC TCA CAA AAG AGA GAG AGA GAG AGC GCT CTT TTA TAA AAA AAG AGT GTT TTA TAC ACA CAT ATT TTT TTT TTT TTC TCC CCC CCT CTG TGA GAC ACT CTT TTA TAA AAA AAT ATG TGG GGT GTG TGA GAT ATG TGT GTG TGG GGT GTG TGG GGC GCT CTA TAT ATT TTG TGA GAT ATT TTA TAT ATA TAA AAA AAC ACA CAC ACT CTA TAC ACA CAC ACA CAC ACC CCC CCT CTC TCT CTT TTT TTT TTA TAA AAG AGA GAA AAA AAG AGG GGA GAG AGC GCT CTA TAA AAA AAT ATT TTG TGT GTT TTA TAC ACA CAT ATA TAA AAA AAC ACC CCT CTA TAT ATT TTG TGT GTT TTT TTG TGG GGC GCA CAT ATG TGT GTT TTA TAA AAC ACA CAA AAT ATG TGC GCA CAA AAC ACT CTA TAA AAT ATA TAA AAA AAG AGC GCC CCA CAC ACG CGT GTA TAT ATA TAA AAA AAC ACC CCA CAA AAA AAT ATA TAC ACC CCT CTG TGG GGT GTG TGT GTA TAT ATA TAC ACG CGT GTT TTG TGT GTC TCT CTT TTT TTG TGG GGA GAG AGC GCA CAC ACA CAA AAA AAA AAC ACC CCA CAG AGT GTT TTG TGA GAA AAA AAC ACA CAT ATC TCA CAA AAA AAT ATT TTC TCG CGT GTT TTT TTG TGA GAT ATG TGT GTA TAC ACT CTG TGA GAA AAG AGT GTC TCA CAG AGA GAG AGG GGA GAC ACG CGC GCG CGC GCA CAG AGG GGG GGA GAA AAT ATG TGG GGA GAT ATA TAA AAT ATC TCT CTT TTG TGC GCC CCT CTG TGC GCG CGA GAA AAG AGA GAT ATC TCT CTA TAA AAA AAA AAC ACC CCA CAG AGT GTC TCT CTC TCT CTG TGA GAA AAG AGA GAA AAG AGT GTA TAG AGT GTG TGG GGA GAA AAA AAA AAT ATC TCC CCT CTA TAC ACC CCA CAT ATA TAC ACA CAG AGA GAA AAA AAG AGA GAC ACG CGT GTT TTC TCT CTT TTG TGA GAG AGT GTG TGT GTA TAA AAT ATG TGT GTG TGA GAA AAA AAA AAC ACT CTA TAC ACC CCG CGA GAA AAG AGT GTT TTG TGT GTA TAG AGG GGA GAG AGA GAC ACA CAT ATT TTA TAT ATA TAC ACT CTT TTA TAA AAA AAC ACC CCA CAG AGC GCA CAA AAA AAT ATA TAA AAT ATA TAG AGT GTT TTT TTA TAA AAA AAA AAA AAT ATT TTA TAC ACA CAG AGA GAA AAG AGA GAG AGG GGT GTT TTG TGG GGC GCC CCA CAC ACA CAC ACA CAG AGA GAT ATC TCT CTA TAA AAT ATG TGG GGC GCT CTG TGC GCT CTT TTA TAT ATG TGT GTA TAG AGA GAC ACA CAA AAT ATT TTC TCT CTA TAG AGT GTC TCT CTT TTA TAC ACT CTA TAT ATT TTA TAA AAG AGA GAA AAA AAC ACC CCT CTA TAA AAT ATG TGA GAA AAT ATT TTA TAT ATC TCT CTA TAG AGA GAG AGT GTA TAT ATT TTA TAG AGG GGT GTT TTT TTG TGA GAA AAA AAA AAC ACC CCC CCT CTT TTG TGC GCT CTA TAC ACT CTC TCA CAT ATG TGG GGT GTT TTT TTA TAG AGC GCT CTG TGC GCT CTG TGT GTT TTA TAA AAT ATA TAG AGT GTG TGT GTC TCC CCC CCT CTT TTG TGG GGG GGA GAT ATA TAC ACT CTA TAT ATA TAG AGC GCT CTA TAA AAT ATT TTA TAT ATG TGC GCT CTA TAA AAG AGC GCC CCT CTT TTT TTT TTC TCT CTT TTA TAA AAC ACA CAA AAA AAG AGT GTT TTG TGT GTT TTA TAG AGT GTA TAC ACA CAA AAC ACT CTA TAC ACT CTA TAA AAC ACA CAT ATA TAG AGT GTT TTA TAC ACA CAC ACG CGG GGT GTG TGT GTT TTT TTA TAA AAA AAC ACC CCG CGT GTG TGT GTT TTT TTG TGT GTA TAC ACT CTA TAA AAT ATT TTA TAT ATA TAT ATG TGC GCC CCT CTT TTA TAT ATT TTT TTC TCT CTT TTT TTA TAC ACT CTT TTT TTA TAT ATT TTG TGC GCT CTA TAC ACA CAA AAT ATT TTG TGT GTG TGT GTA TAC ACT CTT TTT TTT TTA TAC ACT CTA TAG AGA GAA AAG AGT GTA TAC ACA CAA AAA AAT ATT TTC TCT CTA TAG AGA GAA AAT ATT TTA TAA AAA AAG AGC GCA CAT ATC TCT CTA TAT ATG TGC GCC CCG CGA GAC ACT CTA TAC ACT CTA TAT ATA TAG AGC GCA CAA AAA AAG AGA GAA AAT ATA TAC ACT CTG TGT GTT TTA TAA AAG AGA GAG AGT GTG TGT GTC TCG CGG GGT GTA TAA AAA AAT ATT TTT TTT TTG TGT GTC TCT CTA TAG AGA GAG AGG GGC GCT CTT TTC TCA CAT ATT TTT TTA TAA AAT ATT TTA TAT ATT TTT TTG TGA GAA AAG AGT GTC TCA CAC ACC CCT CTA TAA AAT ATT TTT TTT TTT TTC TCT CTA TAA AAA AAC ACT CTG TGA GAT ATA TAA AAA AAT ATA TAT ATT TTA TAT ATA TAA AAT ATT TTT TTG TGG GGT GTT TTT TTT TTT TTA TAC ACT CTA TAT ATT TTA TAA AAG AGT GTG TGT GTT TTT TTG TGC GCC CCT CTA TAG AGG GGT GTT TTC TCT CTT TTT TTA TAA AAT ATC TCT CTA TAC ACT CTC TCA CAA AAC ACC CCG CGC GCT CTG TGC GCT CTT TTT TTA TAG AGG GGT GTG TGT GTT TTT TTT TTA TAA AAT ATG TGT GTC TCT CTA TAA AAT ATT TTT TTA TAG AGG GGC GCA CAT ATG TGC GCC CCT CTT TTC TCT CTT TTA TAC ACT CTG TGT GTA TAC ACT CTG TGG GGT GTT TTA TAC ACA CAG AGA GAG AGA GAA AAG AGG GGC GCT CTA TAT ATT TTT TTG TGA GAA AAC ACT CTC TCT CTA TAC ACT CTA TAA AAT ATG TGT GTC TCA CAC ACT CTA TAT ATT TTG TGC GCA CAA AAC ACC CCT CTA TAC ACT CTG TGT GTA TAC ACT CTG TGG GGT GTT TTC TCT CTA TAT ATA TAC ACC CCT CTT TTG TGT GTA TAG AGT GTG TGT GTT TTT TTG TGT GTC TCT CTT TTA TAG AGT GTG TGG GGT GTT TTT TTA TAG AGA GAT ATT TTC TCT CTT TTT TTA TAG AGA GAC ACA CAC ACC CCT CTA TAT ATC TCC CCT CTT TTC TCT CTT TTT TTA TAG AGA GAA AAA AAC ACT CTA TAT ATA TAC ACA CAA AAA AAT ATT TTA TAC ACC CCA CAT ATT TTT TTC TCA CAT ATC TCT CTT TTT TTT TTA TAA AAA AAT ATG TGG GGG GGA GAT ATT TTT TTA TAA AAC ACT CTG TGC GCT CTT TTT TTT TTG TGG GGC GCT CTT TTA TAG AGT GTT TTG TGC GCA CAG AGA GAG AGT GTG TGG GGT GTT TTT TTT TTT TTG TGG GGC GCA CAT ATA TAT ATA TAT ATT TTC TCT CTT TTT TTT TTC TCA CAC ACT CTA TAG AGG GGT GTT TTT TTT TTT TTC TCT CTA TAT ATG TGT GTA TAC ACT CTT TTG TGG GGA GAT ATT TTG TGG GGC GCT CTG TGC GCA CAA AAT ATC TCA CAT ATG TGC GCA CAA AAT ATT TTG TGT GTT TTT TTT TTT TTC TCA CAG AGC GCT CTA TAT ATT TTT TTT TTG TGC GCA CAG AGT GTA TAC ACA CAT ATT TTT TTT TTA TAT ATT TTA TAG AGT GTA TAA AAT ATT TTC TCT CTT TTG TGG GGC GCT CTT TTA TAT ATG TGT GTG TGG GGT GTT TTA TAA AAT ATA TAA AAT ATT TTA TAA AAT ATC TCT CTT TTG TGT GTA TAC ACA CAA AAA AAT ATG TGG GGC GCC CCC CCC CCG CGA GAT ATT TTT TTC TCA CAG AGC GCT CTA TAT ATG TGG GGT GTT TTA TAG AGA GAA AAT ATG TGT GTA TAC ACA CAT ATC TCT CTT TTC TCT CTT TTT TTG TGC GCA CAT ATC TCA CAT ATT TTT TTT TTA TAT ATT TTA TAT ATG TGT GTA TAT ATG TGG GGA GAA AAA AAA AAG AGT GTT TTA TAT ATG TGT GTG TGC GCA CAT ATG TGT GTT TTG TGT GTA TAG AGA GAC ACG CGG GGT GTT TTG TGT GTA TAA AAT ATT TTC TCA CAT ATC TCA CAA AAC ACT CTT TTG TGT GTA TAT ATG TGA GAT ATG TGT GTG TGT GTT TTA TAC ACA CAA AAA AAC ACG CGT GTA TAA AAT ATA TAG AGA GAG AGC GCA CAA AAC ACA CAA AAG AGA GAG AGT GTC TCG CGA GAA AAT ATG TGT GTA TAC ACA CAA AAC ACT CTA TAT ATT TTG TGT GTT TTA TAA AAT ATG TGG GGT GTG TGT GTT TTA TAG AGA GAA AAG AGG GGT GTC TCC CCT CTT TTT TTT TTA TAT ATG TGT GTC TCT CTA TAT ATG TGC GCT CTA TAA AAT ATG TGG GGA GAG AGG GGT GTA TAA AAA AAG AGG GGC GCT CTT TTT TTT TTG TGC GCA CAA AAA AAC ACT CTA TAC ACA CAC ACA CAA AAT ATT TTG TGG GGA GAA AAT ATT TTG TGT GTG TGT GTT TTA TAA AAT ATT TTG TGT GTG TGA GAT ATA TAC ACA CAT ATT TTC TCT CTG TGT GTG TGC GCT CTG TGG GGT GTA TAG AGT GTA TAC ACA CAT ATT TTT TTA TAT ATT TTA TAG AGT GTG TGA GAT ATG TGA GAA AAG AGT GTT TTG TGC GCG CGA GAG AGA GAG AGA GAC ACT CTT TTG TGT GTC TCA CAC ACT CTA TAC ACA CAG AGT GTT TTT TTA TAA AAA AAA AAG AGA GAC ACC CCA CAA AAT ATA TAA AAA AAT ATC TCC CCT CTA TAC ACT CTG TGA GAC ACC CCA CAG AGT GTC TCT CTT TTC TCT CTT TTA TAC ACA CAT ATC TCG CGT GTT TTG TGA GAT ATA TAG AGT GTG TGT GTT TTA TAC ACA CAG AGT GTG TGA GAA AAG AGA GAA AAT ATG TGG GGT GTT TTC TCC CCA CAT ATC TCC CCA CAT ATC TCT CTT TTT TTA TAC ACT CTT TTT TTG TGA GAT ATA TAA AAA AAG AGC GCT CTG TGG GGT GTC TCA CAA AAA AAA AAG AGA GAC ACT CTT TTA TAT ATG TGA GAA AAA AAG AGA GAC ACA CAT ATT TTC TCT CTC TCT CTC TCT CTC TCT CTC TCA CAT ATT TTT TTT TTG TGT GTT TTA TAA AAC ACT CTT TTA TAG AGA GAC ACA CAA AAC ACC CCT CTG TGA GAG AGA GAG AGC GCT CTA TAA AAT ATA TAA AAC ACA CAC ACT CTA TAA AAA AAG AGG GGT GTT TTC TCA CAT ATT TTG TGC GCC CCT CTA TAT ATT TTA TAA AAT ATG TGT GTT TTA TAT ATA TAG AGT GTT TTT TTT TTT TTG TGA GAT ATG TGG GGT GTA TAA AAA AAT ATC TCA CAA AAA AAA AAT ATG TGT GTG TGA GAA AAG AGA GAA AAT ATC TCA CAT ATC TCT CTG TGC GCA CAA AAA AAA AAT ATC TCA CAG AGC GCG CGT GTC TCT CTG TGT GTT TTT TTA TAC ACT CTA TAC ACA CAG AGT GTC TCA CAG AGC GCT CTT TTA TAT ATG TGT GTG TGT GTC TCA CAA AAC ACC CCT CTA TAT ATA TAC ACT CTG TGT GTT TTA TAC ACT CTA TAG AGA GAT ATC TCA CAG AGG GGC GCA CAT ATT TTA TAG AGT GTG TGT GTC TCT CTG TGA GAT ATG TGT GTT TTG TGG GGT GTG TGA GAT ATA TAG AGT GTG TGC GCG CGG GGA GAA AAG AGT GTT TTG TGC GCA CAG AGT GTT TTA TAA AAA AAA AAT ATG TGT GTT TTT TTG TGA GAT ATG TGC GCT CTT TTA TAC ACG CGT GTT TTA TAA AAT ATA TAC ACG CGT GTT TTT TTT TTC TCA CAT ATC TCA CAA AAC ACT CTT TTT TTT TTA TAA AAC ACG CGT GTA TAC ACC CCA CAA AAT ATG TGG GGA GAA AAA AAA AAA AAC ACT CTC TCA CAA AAA AAA AAC ACA CAC ACT CTA TAG AGT GTT TTG TGC GCA CAA AAC ACT CTG TGC GCA CAG AGA GAA AAG AGC GCT CTG TGA GAA AAC ACT CTT TTG TGC GCA CAA AAA AAG AGA GAA AAT ATG TGT GTG TGT GTC TCC CCT CTT TTA TAG AGA GAC ACA CAA AAT ATG TGT GTC TCT CTT TTA TAT ATC TCT CTA TAC ACT CTT TTT TTT TTA TAT ATT TTT TTC TCA CAG AGC GCA CAG AGC GCT CTC TCG CGG GGC GCA CAA AAG AGG GGG GGT GTT TTT TTG TGT GTT TTG TGA GAT ATT TTC TCA CAG AGA GAT ATG TGT GTA TAG AGA GAA AAA AAC ACT CTA TAA AAA AAG AGA GAT ATG TGT GTT TTG TGT GTT TTG TGA GAA AAT ATG TGT GTC TCT CTT TTA TAA AAA AAT ATT TTG TGT GTC TCA CAC ACA CAT ATC TCA CAA AAT ATC TCT CTG TGA GAC ACA CAT ATA TAG AGA GAA AAG AGT GTT TTA TAC ACT CTG TGG GGC GCG CGA GAT ATA TAG AGT GTT TTG TGT GTA TAA AAT ATA TAA AAC ACT CTA TAT ATA TAT ATG TGC GCT CTC TCA CAC ACC CCT CTA TAT ATA TAA AAC ACA CAA AAA AAG AGT GTT TTG TGA GAA AAA AAA AAC ACA CAT ATG TGA GAC ACA CAC ACC CCC CCC CCG CGT GTG TGA GAC ACC CCT CTT TTG TGG GGT GTG TGC GCT CTT TTG TGT GTA TAT ATT TTG TGA GAC ACT CTG TGT GTA TAG AGT GTG TGC GCG CGC GCG CGT GTC TCA CAT ATA TAT ATT TTA TAA AAT ATG TGC GCG CGC GCA CAG AGG GGT GTA TAG AGC GCA CAA AAA AAA AAA AAG AGT GTC TCA CAC ACA CAA AAC ACA CAT ATT TTG TGC GCT CTT TTT TTG TGA GAT ATA TAT ATG TGG GGA GAA AAC ACG CGT GTT TTA TAA AAA AAG AGA GAT ATT TTT TTC TCA CAT ATG TGT GTC TCA CAT ATT TTG TGT GTC TCT CTG TGA GAA AAC ACA CAA AAC ACT CTA TAC ACG CGA GAA AAA AAA AAC ACA CAA AAA AAT ATA TAC ACG CGT GTA TAG AGT GTG TGC GCT CTG TGC GCT CTA TAA AAA AAA AAA AAG AGA GAA AAT ATA TAA AAC ACT CTT TTA TAC ACC CCT CTT TTT TTT TTA TAA AAG AGT GTT TTG TGA GAC ACA CAT ATG TGT GTG TGC GCA CAA AAC ACT CTA TAC ACT CTA TAG AGA GAC ACA CAA AAG AGT GTT TTG TGT GTT TTA TAA AAT ATG TGT GTT TTG TGT GTA TAA AAC ACA CAA AAC ACA CAA AAA AAG AGA GAT ATA TAG AGC GCA CAC ACT CTT TTA TAA AAG AGG GGG GGT GTG TGG GGT GTA TAA AAA AAA AAT ATT TTG TGT GTT TTA TAA AAT ATA TAA AAT ATT TTG TGG GGT GTT TTG TGA GAA AAG AGC GCA CAG AGT GTT TTA TAA AAT ATT TTA TAA AAA AAG AGT GTT TTA TAC ACA CAC ACT CTT TTG TGT GTG TGT GTT TTC TCC CCT CTT TTT TTT TTT TTG TGT GTT TTG TGC GCT CTG TGC GCT CTA TAT ATT TTT TTT TTC TCT CTA TAT ATT TTT TTA TAA AAT ATA TAA AAC ACA CAC ACC CCT CTG TGT GTT TTC TCA CAT ATG TGT GTC TCA CAT ATG TGT GTC TCT CTA TAA AAA AAC ACA CAT ATA TAC ACT CTG TGA GAC ACT CTT TTT TTT TTC TCA CAA AAG AGT GTG TGA GAA AAA AAT ATC TCA CAT ATA TAG AGG GGA GAT ATA TAC ACA CAA AAG AGG GGC GCT CTA TAT ATT TTG TGA GAT ATG TGG GGT GTG TGG GGT GTG TGT GTC TCA CAC ACT CTC TCG CGT GTG TGA GAC ACA CAT ATA TAG AGC GCA CAT ATC TCT CTA TAC ACA CAG AGA GAT ATA TAC ACT CTT TTG TGT GTT TTT TTT TTG TGC GCT CTA TAA AAC ACA CAA AAA AAC ACA CAT ATG TGC GCT CTG TGA GAT ATT TTT TTT TTG TGA GAC ACA CAC ACA CAT ATG TGG GGT GTT TTT TTA TAG AGC GCC CCA CAG AGC GCG CGT GTG TGG GGT GTG TGG GGT GTA TAG AGT GTT TTA TAT ATA TAC ACT CTA TAA AAT ATG TGA GAC ACA CAA AAA AAG AGC GCT CTT TTG TGC GCC CCC CCA CAT ATT TTG TGA GAT ATT TTG TGC GCT CTG TGC GCA CAG AGT GTC TCA CAT ATA TAA AAC ACA CAA AAG AGA GAG AGA GAA AAG AGT GTG TGG GGG GGT GTT TTT TTT TTG TGT GTC TCG CGT GTG TGC GCC CCT CTG TGG GGT GTT TTT TTG TGC GCC CCT CTG TGG GGC GCA CAC ACG CGA GAT ATA TAT ATT TTA TAC ACG CGC GCA CAC ACA CAA AAC ACT CTA TAA AAT ATG TGG GGT GTG TGA GAC ACT CTT TTT TTT TTT TTG TGC GCA CAT ATT TTT TTC TCT CTT TTA TAC ACC CCT CTA TAG AGA GAG AGT GTT TTT TTT TTT TTA TAG AGT GTG TGC GCA CAG AGT GTT TTG TGG GGT GTA TAA AAC ACA CAT ATC TCT CTG TGT GTT TTA TAC ACA CAC ACA CAC ACC CCA CAT ATC TCA CAA AAA AAA AAC ACT CTT TTA TAT ATA TAG AGA GAG AGT GTA TAC ACA CAC ACT CTG TGA GAC ACT CTT TTT TTG TGC GCA CAA AAC ACA CAT ATC TCA CAG AGC GCT CTT TTG TGT GTG TGT GTT TTT TTT TTG TGG GGC GCT CTG TGC GCT CTG TGA GAA AAT ATG TGT GTA TAC ACA CAA AAT ATT TTT TTT TTT TTA TAA AAA AAG AGA GAT ATG TGC GCT CTT TTC TCT CTG TGG GGT GTA TAA AAG AGC GCC CCA CAG AGT GTA TAC ACC CCA CAT ATA TAT ATT TTG TGT GTT TTA TAT ATG TGA GAT ATA TAC ACC CCA CAA AAT ATG TGT GTA TAC ACT CTA TAG AGA GAA AAG AGG GGT GTT TTC TCT CTG TGT GTT TTG TGC GCT CTT TTA TAT ATG TGA GAA AAA AAG AGT GTT TTT TTA TAC ACG CGC GCC CCC CCT CTG TGA GAC ACA CAC ACA CAC ACG CGT GTT TTA TAT ATG TGT GTG TGC GCT CTC TCA CAT ATG TGG GGA GAT ATG TGG GGC GCT CTC TCT CTA TAT ATT TTA TAT ATT TTC TCA CAA AAT ATT TTT TTC TCC CCT CTA TAA AAC ACA CAC ACC CCT CTA TAC ACC CCT CTT TTG TGA GAA AAG AGG GGT GTT TTC TCT CTG TGT GTT TTA TAG AGA GAG AGT GTG TGG GGT GTA TAA AAC ACA CAA AAC ACT CTT TTT TTT TTG TGA GAT ATT TTC TCT CTG TGA GAG AGT GTA TAC ACT CTG TGT GTA TAG AGG GGC GCA CAC ACG CGG GGC GCA CAC ACT CTT TTG TGT GTG TGA GAA AAA AAG AGA GAT ATC TCA CAG AGA GAA AAG AGC GCT CTG TGG GGT GTG TGT GTT TTT TTG TGT GTG TGT GTA TAT ATC TCT CTA TAC ACT CTA TAG AGT GTG TGG GGT GTA TAG AGA GAT ATG TGG GGG GGT GTA TAC ACT CTT TTA TAA AAC ACA CAA AAT ATG TGA GAT ATT TTA TAT ATT TTA TAC ACA CAG AGA GAT ATC TCT CTT TTT TTA TAC ACC CCA CAG AGG GGA GAG AGT GTT TTT TTT TTC TCT CTG TGT GTG TGG GGT GTG TGT GTA TAG AGA GAT ATG TGC GCT CTG TGT GTA TAA AAA AAT ATT TTT TTA TAC ACT CTT TTA TAC ACT CTA TAA AAT ATA TAT ATG TGT GTT TTT TTA TAC ACA CAC ACC CCA CAC ACT CTA TAA AAT ATT TTC TCA CAA AAC ACC CCT CTA TAT ATT TTG TGG GGT GTG TGC GCT CTT TTT TTG TGG GGA GAC ACA CAT ATA TAT ATC TCA CAG AGC GCA CAT ATC TCT CTA TAT ATA TAG AGT GTA TAG AGC GCT CTG TGG GGT GTG TGG GGT GTA TAT ATT TTG TGT GTA TAG AGC GCT CTA TAT ATC TCG CGT GTA TAG AGT GTA TAA AAC ACA CAT ATG TGC GCC CCT CTT TTG TGC GCC CCT CTA TAC ACT CTA TAT ATT TTT TTT TTA TAT ATG TGA GAG AGG GGT GTT TTT TTA TAG AGA GAA AAG AGA GAG AGC GCT CTT TTT TTT TTG TGG GGT GTG TGA GAA AAT ATA TAC ACA CAG AGT GTC TCA CAT ATG TGT GTA TAG AGT GTT TTG TGC GCC CCT CTT TTT TTA TAA AAT ATA TAC ACT CTT TTT TTA TAC ACT CTA TAT ATT TTC TCC CCT CTT TTA TAT ATG TGT GTC TCA CAT ATT TTC TCA CAC ACT CTG TGT GTA TAC ACT CTC TCT CTG TGT GTT TTT TTA TAA AAC ACA CAC ACC CCA CAG AGT GTT TTT TTA TAC ACT CTC TCA CAT ATT TTC TCT CTT TTA TAC ACC CCT CTG TGG GGT GTG TGT GTT TTT TTA TAT ATT TTC TCT CTG TGT GTT TTA TAT ATT TTT TTA TAC ACT CTT TTG TGT GTA TAC ACT CTT TTG TGA GAC ACA CAT ATT TTT TTT TTA TAT ATC TCT CTT TTA TAC ACT CTA TAA AAT ATG TGA GAT ATG TGT GTT TTT TTC TCT CTT TTT TTT TTT TTT TTA TAG AGC GCA CAC ACA CAT ATA TAT ATT TTC TCA CAG AGT GTG TGG GGA GAT ATG TGG GGT GTT TTA TAT ATG TGT GTT TTC TCA CAC ACA CAC ACC CCT CTT TTT TTA TAG AGT GTA TAC ACC CCT CTT TTT TTC TCT CTG TGG GGA GAT ATA TAA AAC ACA CAA AAT ATT TTG TGC GCT CTT TTA TAT ATA TAT ATC TCA CAT ATT TTT TTG TGT GTA TAT ATT TTT TTC TCC CCA CAC ACA CAA AAA AAG AGC GCA CAT ATT TTT TTC TCT CTA TAT ATT TTG TGG GGT GTT TTC TCT CTT TTT TTA TAG AGT GTA TAA AAT ATT TTA TAC ACC CCT CTA TAA AAA AAG AGA GAG AGA GAC ACG CGT GTG TGT GTA TAG AGT GTC TCT CTT TTT TTA TAA AAT ATG TGG GGT GTG TGT GTT TTT TTC TCC CCT CTT TTT TTA TAG AGT GTA TAC ACT CTT TTT TTT TTG TGA GAA AAG AGA GAA AAG AGC GCT CTG TGC GCG CGC GCT CTG TGT GTG TGC GCA CAC ACC CCT CTT TTT TTT TTT TTG TGT GTT TTA TAA AAA AAT ATA TAA AAA AAG AGA GAA AAA AAT ATG TGT GTA TAT ATC TCT CTA TAA AAA AAG AGT GTT TTG TGC GCG CGT GTA TAG AGT GTG TGA GAT ATG TGT GTG TGC GCT CTA TAT ATT TTA TAC ACC CCT CTC TCT CTT TTA TAC ACG CGC GCA CAA AAT ATA TAT ATA TAA AAT ATA TAG AGA GAT ATA TAC ACT CTT TTA TAG AGC GCT CTC TCT CTT TTT TTA TAT ATA TAA AAT ATA TAA AAG AGT GTA TAC ACA CAA AAG AGT GTA TAT ATT TTT TTT TTA TAG AGT GTG TGG GGA GAG AGC GCA CAA AAT ATG TGG GGA GAT ATA TAC ACA CAA AAC ACT CTA TAG AGC GCT CTA TAC ACA CAG AGA GAG AGA GAA AAG AGC GCT CTG TGC GCT CTT TTG TGT GTT TTG TGT GTC TCA CAT ATC TCT CTC TCG CGC GCA CAA AAA AAG AGG GGC GCT CTC TCT CTC TCA CAA AAT ATG TGA GAC ACT CTT TTC TCA CAG AGT GTA TAA AAC ACT CTC TCA CAG AGG GGT GTT TTC TCT CTG TGA GAT ATG TGT GTT TTC TCT CTT TTT TTA TAC ACC CCA CAA AAC ACC CCA CAC ACC CCA CAC ACA CAA AAA AAC ACC CCT CTC TCT CTA TAT ATC TCA CAC ACC CCT CTC TCA CAG AGC GCT CTG TGT GTT TTT TTT TTG TGC GCA CAG AGA GAG AGT GTG TGG GGT GTT TTT TTT TTA TAG AGA GAA AAA AAA AAA AAT ATG TGG GGC GCA CAT ATT TTC TCC CCC CCA CAT ATC TCT CTG TGG GGT GTA TAA AAA AAG AGT GTT TTG TGA GAG AGG GGG GGT GTT TTG TGT GTA TAT ATG TGG GGT GTA TAC ACA CAA AAG AGT GTA TAA AAC ACT CTT TTG TGT GTG TGG GGT GTA TAC ACA CAA AAC ACT CTA TAC ACA CAC ACT CTT TTA TAA AAC ACG CGG GGT GTC TCT CTT TTT TTG TGG GGC GCT CTT TTG TGA GAT ATG TGA GAC ACG CGT GTA TAG AGT GTT TTT TTA TAC ACT CTG TGT GTC TCC CCA CAA AAG AGA GAC ACA CAT ATG TGT GTG TGA GAT ATC TCT CTG TGC GCA CAC ACC CCT CTC TCT CTG TGA GAA AAG AGA GAC ACA CAT ATG TGC GCT CTT TTA TAA AAC ACC CCC CCT CTA TAA AAT ATT TTA TAT ATG TGA GAA AAG AGA GAT ATT TTT TTA TAC ACT CTC TCA CAT ATT TTC TCG CGT GTA TAA AAG AGT GTC TCT CTA TAA AAT ATC TCA CAT ATA TAA AAT ATT TTT TTC TCT CTT TTG TGG GGT GTA TAC ACA CAG AGG GGC GCT CTG TGG GGT GTA TAA AAT ATG TGT GTT TTC TCA CAA AAC ACT CTC TCA CAG AGG GGG GGT GTT TTA TAT ATT TTG TGG GGA GAC ACA CAT ATT TTC TCT CTA TAT ATG TGC GCA CAA AAA AAA AAT ATT TTG TGT GTG TGT GTA TAC ACT CTT TTA TAA AAG AGC GCT CTT TTA TAA AAG AGG GGT GTT TTG TGA GAT ATA TAC ACA CAG AGC GCC CCA CAA AAT ATC TCC CCT CTA TAA AAG AGA GAC ACA CAC ACC CCT CTA TAA AAG AGT GTA TAT ATA TAA AAG AGT GTT TTT TTG TGT GTT TTC TCG CGC GCA CAT ATT TTC TCA CAA AAC ACC CCA CAG AGG GGA GAC ACA CAG AGA GAC ACT CTT TTT TTT TTT TTC TCA CAG AGT GTG TGT GTT TTA TAG AGC GCT CTT TTG TGT GTT TTA TAC ACA CAA AAT ATG TGG GGT GTT TTC TCA CAC ACC CCA CAT ATC TCT CTG TGG GGT GTG TGT GTT TTT TTA TAC ACC CCA CAA AAT ATG TGT GTG TGC GCT CTA TAT ATG TGA GAG AGG GGC GCC CCC CCA CAA AAT ATT TTT TTC TCA CAC ACT CTA TAT ATT TTA TAA AAG AGG GGG GGT GTT TTC TCA CAT ATT TTC TCC CCT CTT TTA TAA AAT ATG TGG GGT GTT TTC TCA CAT ATG TGT GTG TGG GGT GTA TAG AGT GTG TGT GTT TTG TGG GGT GTT TTT TTT TTA TAA AAC ACA CAT ATA TAG AGA GAT ATT TTA TAT ATG TGA GAC ACT CTG TGT GTG TGT GTC TCT CTC TCT CTT TTT TTT TTT TTG TGT GTT TTA TAC ACA CAT ATG TGC GCA CAC ACC CCA CAT ATA TAT ATG TGG GGA GAA AAT ATT TTA TAC ACC CCA CAA AAC ACT CTG TGG GGA GAG AGT GTT TTC TCA CAT ATG TGC GCT CTG TGG GGC GCA CAC ACA CAG AGA GAC ACT CTT TTA TAG AGA GAA AAG AGG GGT GTA TAA AAC ACT CTT TTT TTT TTA TAT ATG TGG GGA GAC ACC CCT CTT TTT TTT TTG TGT GTT TTG TGA GAC ACA CAG AGG GGC GCA CAA AAA AAC ACA CAG AGC GCA CAC ACA CAA AAG AGC GCA CAG AGC GCT CTG TGG GGT GTA TAC ACG CGG GGA GAC ACA CAC ACA CAA AAC ACT CTA TAT ATT TTA TAC ACA CAG AGT GTT TTA TAA AAT ATG TGT GTT TTT TTT TTA TAG AGC GCT CTT TTG TGG GGT GTT TTG TGT GTA TAC ACG CGC GCT CTG TGC GCT CTG TGT GTT TTA TAT ATA TAA AAA AAT ATG TGG GGA GAG AGA GAC ACA CAG AGG GGT GTG TGG GGT GTT TTT TTC TCT CTC TCA CAA AAT ATC TCG CGA GAT ATT TTT TTA TAC ACC CCA CAC ACA CAA AAC ACT CTC TCT CTT TTA TAA AAT ATG TGA GAC ACT CTT TTT TTA TAA AAC ACC CCT CTT TTG TGT GTG TGG GGC GCT CTA TAT ATG TGA GAA AAG AGT GTA TAC ACA CAA AAT ATT TTA TAT ATG TGA GAA AAC ACC CCT CTC TCT CTA TAA AAC ACA CAC ACA CAA AAG AGA GAC ACC CCA CAT ATG TGT GTT TTG TGA GAC ACA CAT ATA TAC ACT CTA TAG AGG GGA GAC ACC CCT CTC TCT CTT TTT TTC TCT CTG TGC GCT CTC TCA CAA AAA AAC ACT CTG TGG GGA GAA AAT ATT TTG TGC GCC CCG CGT GTT TTT TTT TTA TAG AGA GAT ATA TAT ATG TGT GTG TGT GTG TGC GCT CTT TTC TCA CAT ATT TTA TAA AAA AAA AAG AGA GAA AAT ATT TTA TAC ACT CTG TGC GCA CAA AAA AAA AAT ATG TGG GGT GTA TAT ATG TGA GAA AAT ATG TGG GGA GAC ACG CGT GTA TAC ACC CCA CAT ATA TAT ATT TTG TGG GGG GGT GTA TAG AGT GTG TGC GCT CTT TTT TTA TAT ATT TTA TAG AGA GAA AAG AGA GAT ATG TGA GAA AAT ATT TTT TTA TAC ACA CAC ACC CCT CTT TTT TTT TTG TGA GAT ATG TGT GTT TTG TGT GTT TTA TAG AGA GAC ACA CAA AAT ATG TGC GCT CTC TCA CAG AGG GGT GTG TGT GTT TTA TAC ACT CTT TTT TTC TCC CCA CAA AAA AAG AGT GTG TGC GCA CAG AGT GTG TGA GAA AAA AAA AAG AGA GAA AAC ACA CAA AAT ATC TCA CAA AAG AGG GGG GGT GTA TAC ACA CAC ACA CAC ACC CCA CAC ACT CTG TGG GGT GTT TTG TGT GTT TTA TAC ACT CTC TCA CAC ACA CAA AAT ATT TTT TTT TTG TGA GAC ACT CTT TTC TCA CAC ACT CTT TTT TTT TTA TAG AGT GTT TTT TTT TTA TAG AGT GTC TCC CCA CAG AGA GAG AGT GTA TAC ACT CTC TCA CAA AAT ATG TGG GGT GTC TCT CTT TTT TTG TGT GTT TTC TCT CTT TTT TTT TTT TTT TTT TTT TTG TGT GTA TAT ATG TGA GAA AAA AAA AAT ATG TGC GCC CCT CTT TTT TTT TTT TTA TAC ACC CCT CTT TTT TTT TTG TGC GCT CTA TAT ATG TGG GGG GGT GTA TAT ATT TTA TAT ATT TTG TGC GCT CTA TAT ATG TGT GTC TCT CTG TGC GCT CTT TTT TTT TTG TGC GCA CAA AAT ATG TGA GAT ATG TGT GTT TTT TTG TGT GTC TCA CAA AAA AAC ACA CAT ATA TAA AAG AGC GCA CAT ATG TGC GCA CAT ATT TTT TTC TCT CTC TCT CTG TGT GTT TTT TTG TGT GTT TTT TTT TTT TTG TGT GTT TTA TAC ACC CCT CTT TTC TCT CTC TCT CTT TTG TGC GCC CCA CAC ACT CTG TGT GTA TAG AGC GCT CTT TTA TAT ATT TTT TTT TTA TAA AAT ATA TAT ATG TGG GGT GTC TCT CTA TAT ATA TAT ATG TGC GCC CCT CTG TGC GCT CTA TAG AGT GTT TTG TGG GGG GGT GTG TGA GAT ATG TGC GCG CGT GTA TAT ATT TTA TAT ATG TGA GAC ACA CAT ATG TGG GGT GTT TTG TGG GGA GAT ATA TAT ATG TGG GGT GTT TTG TGA GAT ATA TAC ACT CTA TAG AGT GTT TTT TTG TGT GTC TCT CTG TGG GGT GTT TTT TTT TTA TAA AAG AGC GCT CTA TAA AAA AAA AAG AGA GAC ACT CTG TGT GTG TGT GTT TTA TAT ATG TGT GTA TAT ATG TGC GCA CAT ATC TCA CAG AGC GCT CTG TGT GTA TAG AGT GTG TGT GTT TTA TAC ACT CTA TAA AAT ATC TCC CCT CTT TTA TAT ATG TGA GAC ACA CAG AGC GCA CAA AAG AGA GAA AAC ACT CTG TGT GTG TGT GTA TAT ATG TGA GAT ATG TGA GAT ATG TGG GGT GTG TGC GCT CTA TAG AGG GGA GAG AGA GAG AGT GTG TGT GTG TGG GGA GAC ACA CAC ACT CTT TTA TAT ATG TGA GAA AAT ATG TGT GTC TCT CTT TTG TGA GAC ACA CAC ACT CTC TCG CGT GTT TTT TTA TAT ATA TAA AAA AAG AGT GTT TTT TTA TAT ATT TTA TAT ATG TGG GGT GTA TAA AAT ATG TGC GCT CTT TTT TTA TAG AGA GAT ATC TCA CAA AAG AGC GCC CCA CAT ATT TTT TTC TCC CCA CAT ATG TGT GTG TGG GGG GGC GCT CTC TCT CTT TTA TAT ATA TAA AAT ATC TCT CTC TCT CTG TGT GTT TTA TAC ACT CTT TTC TCT CTA TAA AAC ACT CTA TAC ACT CTC TCA CAG AGG GGT GTG TGT GTA TAG AGT GTT TTA TAC ACA CAA AAC ACT CTG TGT GTC TCA CAT ATG TGT GTT TTT TTT TTT TTG TGG GGC GCC CCA CAG AGA GAG AGG GGT GTA TAT ATT TTG TGT GTT TTT TTT TTT TTA TAT ATG TGT GTG TGT GTG TGT GTT TTG TGA GAG AGT GTA TAT ATT TTG TGC GCC CCC CCT CTA TAT ATT TTT TTT TTC TCT CTT TTC TCA CAT ATA TAA AAC ACT CTG TGG GGT GTA TAA AAT ATA TAC ACA CAC ACT CTT TTC TCA CAG AGT GTG TGT GTA TAT ATA TAA AAT ATG TGC GCT CTA TAG AGT GTT TTT TTA TAT ATT TTG TGT GTT TTT TTC TCT CTT TTA TAG AGG GGC GCT CTA TAT ATT TTT TTT TTT TTG TGT GTA TAC ACT CTT TTG TGT GTT TTA TAC ACT CTT TTT TTG TGG GGC GCC CCT CTC TCT CTT TTT TTT TTG TGT GTT TTT TTA TAC ACT CTC TCA CAA AAC ACC CCG CGC GCT CTA TAC ACT CTT TTT TTA TAG AGA GAC ACT CTG TGA GAC ACT CTC TCT CTT TTG TGG GGT GTG TGT GTT TTT TTA TAT ATG TGA GAT ATT TTA TAC ACT CTT TTA TAG AGT GTT TTT TTC TCT CTA TAC ACA CAC ACA CAG AGG GGA GAG AGT GTT TTT TTA TAG AGA GAT ATA TAT ATA TAT ATG TGA GAA AAT ATT TTC TCA CAC ACA CAG AGG GGG GGA GAC ACT CTA TAC ACT CTC TCC CCC CCA CAC ACC CCC CCA CAA AAG AGA GAA AAT ATA TAG AGC GCA CAT ATA TAG AGA GAT ATG TGC GCC CCT CTT TTC TCA CAA AAA AAC ACT CTC TCA CAA AAC ACA CAT ATT TTA TAA AAA AAT ATT TTG TGT GTT TTG TGG GGG GGT GTG TGT GTT TTG TGG GGT GTG TGG GGC GCA CAA AAA AAC ACC CCT CTT TTG TGT GTA TAT ATC TCA CAA AAA AAG AGT GTA TAG AGC GCC CCA CAC ACT CTG TGT GTA TAC ACA CAG AGT GTC TCT CTA TAA AAA AAA AAT ATG TGT GTC TCA CAG AGA GAT ATG TGT GTA TAA AAA AAG AGT GTG TGC GCA CAC ACA CAT ATC TCA CAG AGT GTA TAG AGT GTC TCT CTT TTA TAC ACT CTC TCT CTC TCA CAG AGT GTT TTT TTT TTG TGC GCA CAA AAC ACA CAA AAC ACT CTC TCA CAG AGA GAG AGT GTA TAG AGA GAA AAT ATC TCA CAT ATC TCA CAT ATC TCT CTA TAA AAA AAT ATT TTG TGT GTG TGG GGG GGC GCT CTC TCA CAA AAT ATG TGT GTG TGT GTC TCC CCA CAG AGT GTT TTA TAC ACA CAC ACA CAA AAT ATG TGA GAC ACA CAT ATT TTC TCT CTC TCT CTT TTA TAG AGC GCT CTA TAA AAA AAG AGA GAT ATA TAC ACT CTA TAC ACT CTG TGA GAA AAG AGC GCC CCT CTT TTT TTG TGA GAA AAA AAA AAA AAA AAT ATG TGG GGT GTT TTT TTC TCA CAC ACT CTA TAC ACT CTT TTT TTC TCT CTG TGT GTT TTT TTT TTG TGC GCT CTT TTT TTC TCC CCA CAT ATG TGC GCA CAG AGG GGG GGT GTG TGC GCT CTG TGT GTA TAG AGA GAC ACA CAT ATA TAA AAA AAC ACA CAA AAG AGC GCT CTT TTT TTG TGT GTG TGA GAA AAG AGA GAA AAA AAT ATG TGC GCT CTG TGG GGA GAC ACA CAA AAC ACA CAG AGG GGG GGC GCA CAA AAC ACC CCT CTT TTA TAC ACA CAA AAG AGC GCT CTA TAT ATA TAG AGC GCC CCT CTC TCA CAG AGA GAG AGT GTT TTT TTA TAG AGT GTT TTC TCC CCC CCT CTT TTC TCC CCA CAT ATC TCA CAT ATA TAT ATG TGC GCA CAG AGC GCT CTT TTT TTT TTG TGC GCT CTA TAC ACT CTG TGC GCT CTC TCA CAA AAG AGA GAA AAG AGC GCT CTT TTA TAT ATG TGA GAG AGC GCA CAG AGG GGC GCT CTG TGT GTT TTG TGC GCT CTA TAA AAT ATG TGG GGT GTG TGA GAT ATT TTC TCT CTG TGA GAA AAG AGT GTT TTG TGT GTT TTC TCT CTT TTA TAA AAA AAA AAA AAG AGT GTT TTG TGA GAA AAG AGA GAA AAG AGT GTC TCT CTT TTT TTG TGA GAA AAT ATG TGT GTG TGG GGC GCT CTA TAA AAA AAT ATC TCT CTG TGA GAA AAT ATT TTT TTG TGA GAC ACC CCG CGT GTG TGA GAT ATG TGC GCA CAG AGC GCC CCA CAT ATG TGC GCA CAA AAC ACG CGT GTA TAA AAG AGT GTT TTG TGG GGA GAA AAA AAA AAG AGA GAT ATG TGG GGC GCT CTG TGA GAT ATC TCA CAA AAG AGC GCT CTA TAT ATG TGA GAC ACC CCC CCA CAA AAA AAT ATG TGT GTA TAT ATA TAA AAA AAC ACA CAG AGG GGC GCT CTA TAG AGA GAT ATC TCT CTG TGA GAG AGG GGA GAC ACA CAA AAG AGA GAG AGG GGG GGC GCA CAA AAA AAA AAG AGT GTT TTA TAC ACT CTA TAG AGT GTG TGC GCT CTA TAT ATG TGC GCA CAG AGA GAC ACA CAA AAT ATG TGC GCT CTT TTT TTT TTC TCA CAC ACT CTA TAT ATG TGC GCT CTT TTA TAG AGA GAA AAA AAG AGT GTT TTG TGG GGA GAT ATA TAA AAT ATG TGA GAT ATG TGC GCA CAC ACT CTC TCA CAA AAC ACA CAA AAC ACA CAT ATT TTA TAT ATC TCA CAA AAC ACA CAA AAT ATG TGC GCA CAA AAG AGA GAG AGA GAT ATG TGG GGT GTT TTG TGT GTG TGT GTT TTC TCC CCC CCT CTT TTG TGA GAA AAC ACA CAT ATA TAA AAT ATA TAC ACC CCT CTC TCT CTT TTA TAC ACA CAA AAC ACA CAG AGC GCA CAG AGC GCC CCA CAA AAA AAC ACT CTA TAA AAT ATG TGG GGT GTT TTG TGT GTC TCA CAT ATA TAC ACC CCA CAG AGA GAC ACT...

In [ ]:
k_mers.getNumPartitions

  

>     res9: Int = 21

In [ ]:
k_mers.count

  

#### Format data for apache.spark.mllib.clustering.LDA (adapted from the LDA course tutorial '034*LDA*20NewsGroupsSmall')

This takes long time (lazy evaluation \[after this caching will help in
later stages\])

In [ ]:
val kmers_df = k_mers.zipWithIndex.toDF("kmers", "id")

  

>     kmers_df: org.apache.spark.sql.DataFrame = [kmers: string, id: bigint]

In [ ]:
import org.apache.spark.ml.feature.RegexTokenizer

// Set params for RegexTokenizer
val tokenizer = new RegexTokenizer()
.setPattern("[\\W_]+") // break by white space character(s)  - try to remove emails and other patterns
.setInputCol("kmers") // name of the input column
.setOutputCol("tokens") // name of the output column

// Tokenize document
val tokenized_df = tokenizer.transform(kmers_df)

  

>     import org.apache.spark.ml.feature.RegexTokenizer
>     tokenizer: org.apache.spark.ml.feature.RegexTokenizer = RegexTokenizer: uid=regexTok_9eb145fc922f, minTokenLength=1, gaps=true, pattern=[\W_]+, toLowercase=true
>     tokenized_df: org.apache.spark.sql.DataFrame = [kmers: string, id: bigint ... 1 more field]

In [ ]:
display(tokenized_df.select("tokens"))

  

Note: Better to give a fixed vocabulary ( all possible combinations for
a k-mer) (!problem here too many unique terms!) // import
org.apache.spark.sql.functions.countDistinct // val unique*terms =
tokenized*df.select(countDistinct("tokens")).cache() //
unique\_terms.show()

In [ ]:
import org.apache.spark.ml.feature.CountVectorizer

// Set params for CountVectorizer
val vectorizer = new CountVectorizer()
.setInputCol("tokens")
.setOutputCol("features")
.setMinDF(1.0) 
.setVocabSize(3) // max 64=4*4*4 possible words for 3-mer (4 letter genome alphabet: A-T-G-C)
.fit(tokenized_df)

  

With a fixed vocabulary

In [ ]:
import org.apache.spark.ml.feature.CountVectorizerModel
// create a dictionary array from all possible k-mers 
val fixed_vocab = List.fill((k))(List("a","t","g","c")).flatten.combinations(k).flatMap(_.permutations).toArray.map(_.mkString("")) // https://stackoverflow.com/questions/38406959/creating-all-permutations-of-a-list-with-a-limited-range-in-scala
val fixed_vectorizer = new CountVectorizerModel(fixed_vocab)
.setInputCol("tokens")
.setOutputCol("features")// max 64=4*4*4 possible words for 3-mer (4 letter genome alphabet: A-T-G-C)

  

>     import org.apache.spark.ml.feature.CountVectorizerModel
>     fixed_vocab: Array[String] = Array(aaa, aat, ata, taa, aag, aga, gaa, aac, aca, caa, att, tat, tta, atg, agt, tag, tga, gat, gta, atc, act, tac, tca, cat, cta, agg, gag, gga, agc, acg, gac, gca, cag, cga, acc, cac, cca, ttt, ttg, tgt, gtt, ttc, tct, ctt, tgg, gtg, ggt, tgc, tcg, gtc, gct, ctg, cgt, tcc, ctc, cct, ggg, ggc, gcg, cgg, gcc, cgc, ccg, ccc)
>     fixed_vectorizer: org.apache.spark.ml.feature.CountVectorizerModel = CountVectorizerModel: uid=cntVecModel_72ffdd4ca377, vocabularySize=64

In [ ]:
val vocabList = fixed_vectorizer.vocabulary

  

>     vocabList: Array[String] = Array(aaa, aat, ata, taa, aag, aga, gaa, aac, aca, caa, att, tat, tta, atg, agt, tag, tga, gat, gta, atc, act, tac, tca, cat, cta, agg, gag, gga, agc, acg, gac, gca, cag, cga, acc, cac, cca, ttt, ttg, tgt, gtt, ttc, tct, ctt, tgg, gtg, ggt, tgc, tcg, gtc, gct, ctg, cgt, tcc, ctc, cct, ggg, ggc, gcg, cgg, gcc, cgc, ccg, ccc)

In [ ]:
vocabList.size

  

>     res30: Int = 64

In [ ]:
// Create vector of token counts
val countVectors = fixed_vectorizer.transform(tokenized_df).select("id", "features")

  

>     countVectors: org.apache.spark.sql.DataFrame = [id: bigint, features: vector]

In [ ]:
countVectors.take(10)

  

>     res7: Array[org.apache.spark.sql.Row] = Array([0,(64,[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63],[923.0,761.0,471.0,719.0,580.0,605.0,535.0,615.0,809.0,703.0,773.0,622.0,876.0,725.0,507.0,427.0,630.0,440.0,469.0,339.0,674.0,609.0,549.0,484.0,561.0,329.0,297.0,282.0,301.0,164.0,340.0,372.0,438.0,95.0,376.0,459.0,354.0,1004.0,817.0,858.0,700.0,518.0,542.0,738.0,554.0,552.0,454.0,547.0,113.0,269.0,521.0,495.0,171.0,209.0,287.0,344.0,134.0,223.0,88.0,76.0,187.0,97.0,74.0,116.0])], [1,(64,[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63],[884.0,749.0,466.0,704.0,575.0,599.0,529.0,606.0,800.0,697.0,765.0,616.0,865.0,718.0,498.0,417.0,624.0,433.0,459.0,328.0,667.0,604.0,543.0,478.0,551.0,317.0,287.0,273.0,294.0,159.0,335.0,365.0,430.0,88.0,369.0,450.0,347.0,995.0,814.0,844.0,694.0,509.0,528.0,729.0,552.0,540.0,449.0,538.0,106.0,264.0,512.0,486.0,166.0,205.0,281.0,340.0,130.0,219.0,87.0,72.0,182.0,95.0,69.0,112.0])], [2,(64,[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63],[884.0,749.0,466.0,704.0,575.0,599.0,529.0,606.0,800.0,697.0,765.0,616.0,865.0,718.0,498.0,417.0,624.0,433.0,459.0,328.0,667.0,604.0,543.0,478.0,551.0,317.0,287.0,273.0,294.0,159.0,335.0,365.0,430.0,88.0,369.0,450.0,347.0,995.0,814.0,844.0,694.0,509.0,528.0,729.0,552.0,540.0,449.0,538.0,106.0,264.0,512.0,486.0,166.0,205.0,281.0,340.0,130.0,219.0,87.0,72.0,182.0,95.0,69.0,112.0])], [3,(64,[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63],[884.0,749.0,466.0,704.0,575.0,599.0,529.0,606.0,800.0,697.0,765.0,616.0,865.0,718.0,498.0,417.0,624.0,433.0,459.0,328.0,667.0,604.0,543.0,478.0,551.0,317.0,287.0,273.0,294.0,159.0,335.0,365.0,430.0,88.0,369.0,450.0,347.0,995.0,814.0,844.0,694.0,509.0,528.0,729.0,552.0,540.0,449.0,538.0,106.0,264.0,512.0,486.0,166.0,205.0,281.0,340.0,130.0,219.0,87.0,72.0,182.0,95.0,69.0,112.0])], [4,(64,[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63],[884.0,749.0,466.0,704.0,575.0,599.0,529.0,606.0,800.0,697.0,765.0,616.0,865.0,718.0,498.0,417.0,624.0,433.0,459.0,328.0,667.0,604.0,543.0,478.0,551.0,317.0,287.0,273.0,294.0,159.0,335.0,365.0,430.0,88.0,369.0,450.0,347.0,995.0,814.0,844.0,694.0,509.0,528.0,729.0,552.0,540.0,449.0,538.0,106.0,264.0,512.0,486.0,166.0,205.0,281.0,340.0,130.0,219.0,87.0,72.0,182.0,95.0,69.0,112.0])], [5,(64,[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63],[890.0,759.0,471.0,716.0,579.0,602.0,534.0,611.0,804.0,699.0,772.0,621.0,874.0,718.0,506.0,427.0,625.0,435.0,467.0,337.0,675.0,607.0,547.0,484.0,559.0,326.0,294.0,280.0,300.0,164.0,339.0,371.0,434.0,94.0,371.0,456.0,351.0,1014.0,817.0,859.0,698.0,512.0,536.0,733.0,553.0,553.0,453.0,544.0,111.0,269.0,517.0,493.0,169.0,208.0,285.0,342.0,135.0,223.0,88.0,76.0,187.0,97.0,73.0,115.0])], [6,(64,[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63],[891.0,761.0,472.0,718.0,580.0,603.0,534.0,615.0,808.0,703.0,773.0,624.0,877.0,724.0,508.0,427.0,629.0,439.0,470.0,340.0,676.0,609.0,548.0,485.0,559.0,328.0,295.0,280.0,299.0,163.0,339.0,372.0,436.0,95.0,374.0,458.0,354.0,1009.0,818.0,858.0,701.0,517.0,544.0,738.0,554.0,551.0,455.0,546.0,113.0,269.0,518.0,494.0,170.0,208.0,288.0,341.0,135.0,223.0,88.0,76.0,186.0,96.0,73.0,116.0])], [7,(64,[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63],[894.0,761.0,471.0,718.0,579.0,604.0,535.0,614.0,807.0,702.0,773.0,623.0,877.0,724.0,508.0,428.0,629.0,439.0,470.0,340.0,676.0,609.0,548.0,485.0,559.0,328.0,296.0,281.0,299.0,163.0,339.0,372.0,436.0,95.0,374.0,458.0,354.0,1008.0,818.0,858.0,700.0,517.0,545.0,739.0,554.0,551.0,454.0,546.0,113.0,269.0,518.0,494.0,170.0,208.0,288.0,341.0,135.0,223.0,88.0,76.0,186.0,96.0,73.0,116.0])], [8,(64,[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63],[890.0,761.0,471.0,717.0,579.0,603.0,534.0,612.0,807.0,701.0,773.0,623.0,877.0,724.0,508.0,427.0,629.0,439.0,469.0,340.0,676.0,608.0,548.0,485.0,559.0,326.0,295.0,280.0,299.0,163.0,339.0,372.0,435.0,95.0,371.0,458.0,351.0,1008.0,818.0,858.0,700.0,516.0,545.0,738.0,554.0,551.0,453.0,546.0,113.0,269.0,518.0,494.0,170.0,207.0,288.0,340.0,135.0,223.0,88.0,76.0,186.0,96.0,73.0,115.0])], [9,(64,[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63],[893.0,761.0,472.0,720.0,580.0,604.0,534.0,614.0,807.0,701.0,774.0,624.0,878.0,723.0,507.0,426.0,629.0,440.0,469.0,339.0,673.0,610.0,549.0,482.0,561.0,327.0,296.0,279.0,302.0,165.0,338.0,372.0,439.0,96.0,375.0,458.0,352.0,1010.0,819.0,858.0,700.0,516.0,542.0,739.0,553.0,552.0,455.0,548.0,112.0,269.0,522.0,494.0,170.0,208.0,287.0,344.0,133.0,222.0,88.0,76.0,187.0,97.0,74.0,115.0])])

In [ ]:
import org.apache.spark.ml.linalg.Vector

val lda_countVector = countVectors.map { case Row(id: Long, countVector: Vector) => (id, countVector) }.cache()

  

>     import org.apache.spark.ml.linalg.Vector
>     lda_countVector: org.apache.spark.sql.Dataset[(Long, org.apache.spark.ml.linalg.Vector)] = [_1: bigint, _2: vector]

In [ ]:
// format: Array(id, (VocabSize, Array(indexedTokens), Array(Token Frequency)))
lda_countVector.take(1)

  

>     res9: Array[(Long, org.apache.spark.ml.linalg.Vector)] = Array((0,(64,[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63],[923.0,761.0,471.0,719.0,580.0,605.0,535.0,615.0,809.0,703.0,773.0,622.0,876.0,725.0,507.0,427.0,630.0,440.0,469.0,339.0,674.0,609.0,549.0,484.0,561.0,329.0,297.0,282.0,301.0,164.0,340.0,372.0,438.0,95.0,376.0,459.0,354.0,1004.0,817.0,858.0,700.0,518.0,542.0,738.0,554.0,552.0,454.0,547.0,113.0,269.0,521.0,495.0,171.0,209.0,287.0,344.0,134.0,223.0,88.0,76.0,187.0,97.0,74.0,116.0])))

In [ ]:
val num_topics = 10

  

>     num_topics: Int = 10

  

Create the LDA builder

In [ ]:
import org.apache.spark.mllib.clustering.{LDA, OnlineLDAOptimizer}

val lda = new LDA()
.setOptimizer("em")
.setK(num_topics)
.setMaxIterations(3000)
.setDocConcentration(-1) // use default values
.setTopicConcentration(-1) // use default values

  

>     import org.apache.spark.mllib.clustering.{LDA, OnlineLDAOptimizer}
>     lda: org.apache.spark.mllib.clustering.LDA = org.apache.spark.mllib.clustering.LDA@2e8d3712

In [ ]:
// convert ML vectors into MLlib vectors
val lda_countVector_mllib = lda_countVector.map { case (id, vector) => (id, org.apache.spark.mllib.linalg.Vectors.fromML(vector)) }.rdd

val ldaModel = lda.run(lda_countVector_mllib)

  

>     lda_countVector_mllib: org.apache.spark.rdd.RDD[(Long, org.apache.spark.mllib.linalg.Vector)] = MapPartitionsRDD[93148] at rdd at command-3263315706437130:2
>     ldaModel: org.apache.spark.mllib.clustering.LDAModel = org.apache.spark.mllib.clustering.DistributedLDAModel@268aa06a

In [ ]:
val topicIndices = ldaModel.describeTopics(maxTermsPerTopic = 5)

  

>     topicIndices: Array[(Array[Int], Array[Double])] = Array((Array(0, 37, 12, 39, 38),Array(0.03444157420861971, 0.03353058390335083, 0.02937025496124375, 0.028654480529187693, 0.026903904596751745)), (Array(37, 12, 39, 38, 8),Array(0.034078663146711055, 0.02941284929048858, 0.02892782473279161, 0.027681266906881698, 0.02724111987295051)), (Array(37, 0, 12, 39, 38),Array(0.03380183794696089, 0.03078133634600185, 0.02930460394790454, 0.02880617595155694, 0.027448513543661936)), (Array(0, 37, 12, 39, 38),Array(0.03349798117773547, 0.033441249365837186, 0.029392080599673208, 0.028613722928375124, 0.027213052436801832)), (Array(37, 0, 12, 39, 38),Array(0.03367357347540192, 0.03120523572244181, 0.029301414708581334, 0.028579419713923707, 0.02736404043038672)), (Array(37, 0, 12, 39, 38),Array(0.03368925977294659, 0.03204611683546861, 0.029357609171458507, 0.02845778079284791, 0.02720256582662721)), (Array(37, 12, 39, 38, 8),Array(0.033889085599252605, 0.02942589938085284, 0.029023693830823854, 0.027869173019970658, 0.027437465900291676)), (Array(37, 0, 12, 39, 38),Array(0.033678250321072634, 0.031989942130312234, 0.029257840330362158, 0.028785153938342835, 0.027031702120087073)), (Array(37, 12, 39, 38, 8),Array(0.03377437150391708, 0.02944048458003182, 0.028914324102090615, 0.027819163489517296, 0.027207388640435436)), (Array(37, 12, 39, 38, 8),Array(0.033821056385005495, 0.02935798283426454, 0.029084518199319884, 0.02781919932087112, 0.027149464861134)))

In [ ]:
val topics = topicIndices.map { case (terms, termWeights) =>
  terms.map(vocabList(_)).zip(termWeights)
}
println(s"$num_topics topics:")
topics.zipWithIndex.foreach { case (topic, i) =>
  println(s"TOPIC $i")
  topic.foreach { case (term, weight) => println(s"$term\t$weight") }
  println(s"==========")
}

  

>     10 topics:
>     TOPIC 0
>     aaa	0.03444157420861971
>     ttt	0.03353058390335083
>     tta	0.02937025496124375
>     tgt	0.028654480529187693
>     ttg	0.026903904596751745
>     ==========
>     TOPIC 1
>     ttt	0.034078663146711055
>     tta	0.02941284929048858
>     tgt	0.02892782473279161
>     ttg	0.027681266906881698
>     aca	0.02724111987295051
>     ==========
>     TOPIC 2
>     ttt	0.03380183794696089
>     aaa	0.03078133634600185
>     tta	0.02930460394790454
>     tgt	0.02880617595155694
>     ttg	0.027448513543661936
>     ==========
>     TOPIC 3
>     aaa	0.03349798117773547
>     ttt	0.033441249365837186
>     tta	0.029392080599673208
>     tgt	0.028613722928375124
>     ttg	0.027213052436801832
>     ==========
>     TOPIC 4
>     ttt	0.03367357347540192
>     aaa	0.03120523572244181
>     tta	0.029301414708581334
>     tgt	0.028579419713923707
>     ttg	0.02736404043038672
>     ==========
>     TOPIC 5
>     ttt	0.03368925977294659
>     aaa	0.03204611683546861
>     tta	0.029357609171458507
>     tgt	0.02845778079284791
>     ttg	0.02720256582662721
>     ==========
>     TOPIC 6
>     ttt	0.033889085599252605
>     tta	0.02942589938085284
>     tgt	0.029023693830823854
>     ttg	0.027869173019970658
>     aca	0.027437465900291676
>     ==========
>     TOPIC 7
>     ttt	0.033678250321072634
>     aaa	0.031989942130312234
>     tta	0.029257840330362158
>     tgt	0.028785153938342835
>     ttg	0.027031702120087073
>     ==========
>     TOPIC 8
>     ttt	0.03377437150391708
>     tta	0.02944048458003182
>     tgt	0.028914324102090615
>     ttg	0.027819163489517296
>     aca	0.027207388640435436
>     ==========
>     TOPIC 9
>     ttt	0.033821056385005495
>     tta	0.02935798283426454
>     tgt	0.029084518199319884
>     ttg	0.02781919932087112
>     aca	0.027149464861134
>     ==========
>     topics: Array[Array[(String, Double)]] = Array(Array((aaa,0.03444157420861971), (ttt,0.03353058390335083), (tta,0.02937025496124375), (tgt,0.028654480529187693), (ttg,0.026903904596751745)), Array((ttt,0.034078663146711055), (tta,0.02941284929048858), (tgt,0.02892782473279161), (ttg,0.027681266906881698), (aca,0.02724111987295051)), Array((ttt,0.03380183794696089), (aaa,0.03078133634600185), (tta,0.02930460394790454), (tgt,0.02880617595155694), (ttg,0.027448513543661936)), Array((aaa,0.03349798117773547), (ttt,0.033441249365837186), (tta,0.029392080599673208), (tgt,0.028613722928375124), (ttg,0.027213052436801832)), Array((ttt,0.03367357347540192), (aaa,0.03120523572244181), (tta,0.029301414708581334), (tgt,0.028579419713923707), (ttg,0.02736404043038672)), Array((ttt,0.03368925977294659), (aaa,0.03204611683546861), (tta,0.029357609171458507), (tgt,0.02845778079284791), (ttg,0.02720256582662721)), Array((ttt,0.033889085599252605), (tta,0.02942589938085284), (tgt,0.029023693830823854), (ttg,0.027869173019970658), (aca,0.027437465900291676)), Array((ttt,0.033678250321072634), (aaa,0.031989942130312234), (tta,0.029257840330362158), (tgt,0.028785153938342835), (ttg,0.027031702120087073)), Array((ttt,0.03377437150391708), (tta,0.02944048458003182), (tgt,0.028914324102090615), (ttg,0.027819163489517296), (aca,0.027207388640435436)), Array((ttt,0.033821056385005495), (tta,0.02935798283426454), (tgt,0.029084518199319884), (ttg,0.02781919932087112), (aca,0.027149464861134)))

  

#### Visualise Results

In [ ]:
// Zip topic terms with topic IDs
val termArray = topics.zipWithIndex

  

>     termArray: Array[(Array[(String, Double)], Int)] = Array((Array((cgg,0.018905359563726597), (gag,0.018826906708222287), (gtg,0.0186438929438026), (gac,0.018443438711730386), (gta,0.01797393755464917)),0), (Array((ttt,0.03079890216412217), (aaa,0.030263588699584752), (tta,0.029280678722729565), (tgt,0.02903206817540538), (aat,0.028541432285717528)),1), (Array((tgc,0.018469718157263258), (gat,0.018459202951584024), (gca,0.01840913919258279), (acg,0.01836400866492382), (cca,0.01782568599536628)),2), (Array((ttt,0.04243893197355697), (gtt,0.02830295111412854), (aca,0.027768763398499912), (aaa,0.027563616807062468), (atg,0.02747371354607066)),3), (Array((aca,0.02844561624204671), (att,0.021989498189039818), (atg,0.02176716316530825), (aat,0.021659121727235976), (ttt,0.02127340340892406)),4), (Array((agt,0.01916304020205914), (ttc,0.018636869201199755), (aac,0.01851411678212198), (tcg,0.01792018730249314), (gca,0.017895742081755303)),5), (Array((atc,0.0188580834704403), (ttc,0.01853923657230603), (tcc,0.018399492175984657), (gat,0.018206966483854614), (gcc,0.01785051584399567)),6), (Array((ccg,0.01927764640210291), (agg,0.018818711098401008), (cac,0.018668627272176476), (atg,0.018365138519811763), (gag,0.018129818102139303)),7), (Array((acg,0.020595149122383256), (tcg,0.019670086002129036), (act,0.01809843635176616), (cac,0.017979023705549213), (gct,0.017784281001887486)),8), (Array((tta,0.03363877892183009), (tgt,0.03314869000946389), (ttt,0.03246254151121533), (aaa,0.03167156490236555), (ttg,0.028947446411002883)),9))

In [ ]:
// Transform data into the form (term, probability, topicId)
val termRDD = sc.parallelize(termArray)
val termRDD2 =termRDD.flatMap( (x: (Array[(String, Double)], Int)) => {
  val arrayOfTuple = x._1
  val topicId = x._2
  arrayOfTuple.map(el => (el._1, el._2, topicId))
})

  

>     termRDD: org.apache.spark.rdd.RDD[(Array[(String, Double)], Int)] = ParallelCollectionRDD[4068] at parallelize at command-3263315706437135:2
>     termRDD2: org.apache.spark.rdd.RDD[(String, Double, Int)] = MapPartitionsRDD[4069] at flatMap at command-3263315706437135:3

In [ ]:
// Create DF with proper column names
val termDF = termRDD2.toDF.withColumnRenamed("_1", "term").withColumnRenamed("_2", "probability").withColumnRenamed("_3", "topicId")

  

>     termDF: org.apache.spark.sql.DataFrame = [term: string, probability: double ... 1 more field]

In [ ]:
// Create JSON data
val rawJson = termDF.toJSON.collect().mkString(",\n")

  

>     rawJson: String =
>     {"term":"cgg","probability":0.018905359563726597,"topicId":0},
>     {"term":"gag","probability":0.018826906708222287,"topicId":0},
>     {"term":"gtg","probability":0.0186438929438026,"topicId":0},
>     {"term":"gac","probability":0.018443438711730386,"topicId":0},
>     {"term":"gta","probability":0.01797393755464917,"topicId":0},
>     {"term":"ttt","probability":0.03079890216412217,"topicId":1},
>     {"term":"aaa","probability":0.030263588699584752,"topicId":1},
>     {"term":"tta","probability":0.029280678722729565,"topicId":1},
>     {"term":"tgt","probability":0.02903206817540538,"topicId":1},
>     {"term":"aat","probability":0.028541432285717528,"topicId":1},
>     {"term":"tgc","probability":0.018469718157263258,"topicId":2},
>     {"term":"gat","probability":0.018459202951584024,"topicId":2},
>     {"term":"gca","probability":0.01840913919258279,"topicId":2},
>     {"term":"acg","probability":0.01836400866492382,"topicId":2},
>     {"term":"cca","probability":0.01782568599536628,"topicId":2},
>     {"term":"ttt","probability":0.04243893197355697,"topicId":3},
>     {"term":"gtt","probability":0.02830295111412854,"topicId":3},
>     {"term":"aca","probability":0.027768763398499912,"topicId":3},
>     {"term":"aaa","probability":0.027563616807062468,"topicId":3},
>     {"term":"atg","probability":0.02747371354607066,"topicId":3},
>     {"term":"aca","probability":0.02844561624204671,"topicId":4},
>     {"term":"att","probability":0.021989498189039818,"topicId":4},
>     {"term":"atg","probability":0.02176716316530825,"topicId":4},
>     {"term":"aat","probability":0.021659121727235976,"topicId":4},
>     {"term":"ttt","probability":0.02127340340892406,"topicId":4},
>     {"term":"agt","probability":0.01916304020205914,"topicId":5},
>     {"term":"ttc","probability":0.018636869201199755,"topicId":5},
>     {"term":"aac","probability":0.01851411678212198,"topicId":5},
>     {"term":"tcg","probability":0.01792018730249314,"topicId":5},
>     {"term":"gca","probability":0.017895742081755303,"topicId":5},
>     {"term":"atc","probability":0.0188580834704403,"topicId":6},
>     {"term":"ttc","probability":0.01853923657230603,"topicId":6},
>     {"term":"tcc","probability":0.018399492175984657,"topicId":6},
>     {"term":"gat","probability":0.018206966483854614,"topicId":6},
>     {"term":"gcc","probability":0.01785051584399567,"topicId":6},
>     {"term":"ccg","probability":0.01927764640210291,"topicId":7},
>     {"term":"agg","probability":0.018818711098401008,"topicId":7},
>     {"term":"cac","probability":0.018668627272176476,"topicId":7},
>     {"term":"atg","probability":0.018365138519811763,"topicId":7},
>     {"term":"gag","probability":0.018129818102139303,"topicId":7},
>     {"term":"acg","probability":0.020595149122383256,"topicId":8},
>     {"term":"tcg","probability":0.019670086002129036,"topicId":8},
>     {"term":"act","probability":0.01809843635176616,"topicId":8},
>     {"term":"cac","probability":0.017979023705549213,"topicId":8},
>     {"term":"gct","probability":0.017784281001887486,"topicId":8},
>     {"term":"tta","probability":0.03363877892183009,"topicId":9},
>     {"term":"tgt","probability":0.03314869000946389,"topicId":9},
>     {"term":"ttt","probability":0.03246254151121533,"topicId":9},
>     {"term":"aaa","probability":0.03167156490236555,"topicId":9},
>     {"term":"ttg","probability":0.028947446411002883,"topicId":9}

In [ ]:
displayHTML(s"""
<!DOCTYPE html>
<meta charset="utf-8">
<style>

circle {
  fill: rgb(31, 119, 180);
  fill-opacity: 0.5;
  stroke: rgb(31, 119, 180);
  stroke-width: 1px;
}

.leaf circle {
  fill: #ff7f0e;
  fill-opacity: 1;
}

text {
  font: 14px sans-serif;
}

</style>
<body>
<script src="https://cdnjs.cloudflare.com/ajax/libs/d3/3.5.5/d3.min.js"></script>
<script>

var json = {
 "name": "data",
 "children": [
  {
     "name": "topics",
     "children": [
      ${rawJson}
     ]
    }
   ]
};

var r = 1000,
    format = d3.format(",d"),
    fill = d3.scale.category20c();

var bubble = d3.layout.pack()
    .sort(null)
    .size([r, r])
    .padding(1.5);

var vis = d3.select("body").append("svg")
    .attr("width", r)
    .attr("height", r)
    .attr("class", "bubble");

  
var node = vis.selectAll("g.node")
    .data(bubble.nodes(classes(json))
    .filter(function(d) { return !d.children; }))
    .enter().append("g")
    .attr("class", "node")
    .attr("transform", function(d) { return "translate(" + d.x + "," + d.y + ")"; })
    color = d3.scale.category20();
  
  node.append("title")
      .text(function(d) { return d.className + ": " + format(d.value); });

  node.append("circle")
      .attr("r", function(d) { return d.r; })
      .style("fill", function(d) {return color(d.topicName);});

var text = node.append("text")
    .attr("text-anchor", "middle")
    .attr("dy", ".3em")
    .text(function(d) { return d.className.substring(0, d.r / 3)});
  
  text.append("tspan")
      .attr("dy", "1.2em")
      .attr("x", 0)
      .text(function(d) {return Math.ceil(d.value * 10000) /10000; });

// Returns a flattened hierarchy containing all leaf nodes under the root.
function classes(root) {
  var classes = [];

  function recurse(term, node) {
    if (node.children) node.children.forEach(function(child) { recurse(node.term, child); });
    else classes.push({topicName: node.topicId, className: node.term, value: node.probability});
  }

  recurse(null, root);
  return {children: classes};
}
</script>
""")

In [ ]:
displayHTML(s"""
<!DOCTYPE html>
<meta charset="utf-8">
<style>

circle {
  fill: rgb(31, 119, 180);
  fill-opacity: 0.5;
  stroke: rgb(31, 119, 180);
  stroke-width: 1px;
}

.leaf circle {
  fill: #ff7f0e;
  fill-opacity: 1;
}

text {
  font: 14px sans-serif;
}

</style>
<body>
<script src="https://cdnjs.cloudflare.com/ajax/libs/d3/3.5.5/d3.min.js"></script>
<script>

var json = {
 "name": "data",
 "children": [
  {
     "name": "topics",
     "children": [
      ${rawJson}
     ]
    }
   ]
};

var r = 1000,
    format = d3.format(",d"),
    fill = d3.scale.category20c();

var bubble = d3.layout.pack()
    .sort(null)
    .size([r, r])
    .padding(1.5);

var vis = d3.select("body").append("svg")
    .attr("width", r)
    .attr("height", r)
    .attr("class", "bubble");

  
var node = vis.selectAll("g.node")
    .data(bubble.nodes(classes(json))
    .filter(function(d) { return !d.children; }))
    .enter().append("g")
    .attr("class", "node")
    .attr("transform", function(d) { return "translate(" + d.x + "," + d.y + ")"; })
    color = d3.scale.category20();
  
  node.append("title")
      .text(function(d) { return d.className + ": " + format(d.value); });

  node.append("circle")
      .attr("r", function(d) { return d.r; })
      .style("fill", function(d) {return color(d.topicName);});

var text = node.append("text")
    .attr("text-anchor", "middle")
    .attr("dy", ".3em")
    .text(function(d) { return d.className.substring(0, d.r / 3)});
  
  text.append("tspan")
      .attr("dy", "1.2em")
      .attr("x", 0)
      .text(function(d) {return Math.ceil(d.value * 10000) /10000; });

// Returns a flattened hierarchy containing all leaf nodes under the root.
function classes(root) {
  var classes = [];

  function recurse(term, node) {
    if (node.children) node.children.forEach(function(child) { recurse(node.term, child); });
    else classes.push({topicName: node.topicId, className: node.term, value: node.probability});
  }

  recurse(null, root);
  return {children: classes};
}
</script>
""")